Read-only

An example of referencing these resources from outputs:

In [ ]:
import portpicker
import threading
import socket
import IPython

from six.moves import socketserver
from six.moves import SimpleHTTPServer

class V6Server(socketserver.TCPServer):
  address_family = socket.AF_INET6

class Handler(SimpleHTTPServer.SimpleHTTPRequestHandler):
  def do_GET(self):
    self.send_response(200)
    # If the response should not be cached in the notebook for
    # offline access:
    # self.send_header('x-colab-notebook-cache-control', 'no-cache')
    self.end_headers()
    self.wfile.write(b'''
      document.querySelector('#output-area').appendChild(document.createTextNode('Script result!'));
    ''')

port = portpicker.pick_unused_port()

def server_entry():
    httpd = V6Server(('::', port), Handler)
    # Handle a single request then exit the thread.
    httpd.serve_forever()

thread = threading.Thread(target=server_entry)
thread.start()

# Display some HTML referencing the resource.
display(IPython.display.HTML('<script src="https://localhost:{port}/"></script>'.format(port=port)))

In [ ]:
from google.colab import output
output.serve_kernel_port_as_iframe(port)

This will create an iframe browsing the HTTP server hosted on the machine your kernel is running on.

Alternatively to view the server in a separate browser tab:

In [ ]:

from flask import Flask, Response, redirect
import requests

app = Flask(__name__)

# พอร์ตของ kernel ที่คุณต้องการฝัง
KERNEL_PORT = 45509
KERNEL_URL = f"http://localhost:{KERNEL_PORT}"

@app.route('/')
def iframe_wrapper():
    # สร้าง iframe ที่ฝัง kernel
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Secure Kernel Iframe</title>
    </head>
    <body>
        <iframe src="/kernel" width="100%" height="800px" frameborder="0"></iframe>
    </body>
    </html>
    """
    return Response(html, mimetype='text/html')

@app.route('/kernel')
def proxy_kernel():
    # ดึงเนื้อหาจาก kernel แล้วส่งกลับ
    try:
        response = requests.get(KERNEL_URL)
        return Response(response.content, status=response.status_code, content_type=response.headers['Content-Type'])
    except requests.exceptions.ConnectionError:
        return "Kernel is not running on localhost:45509", 502

if __name__ == '__main__':
    app.run(port=8080, debug=True)

In [ ]:
# load an example dataset
from vega_datasets import data
cars = data.cars()

import altair as alt

interval = alt.selection_interval()

points = alt.Chart(cars).mark_point().encode(
  x='Horsepower',
  y='Miles_per_Gallon',
  color=alt.condition(interval, 'Origin', alt.value('lightgray'))
).properties(
  selection=interval
)

histogram = alt.Chart(cars).mark_bar().encode(
  x='count()',
  y='Origin',
  color='Origin'
).transform_filter(interval)

points & histogram

In [ ]:
pip install flask requests

The server will only be accessible to the executor of the notebook while the notebook is being viewed in Colab.

In [ ]:
import IPython

display(IPython.display.Javascript('''
  const promise = new Promise((resolve, reject) => {
    const script = document.createElement('script');
    script.src = 'data:,window.value = "hello world!"';
    script.onload = resolve;
    script.onerror = reject;
    document.head.appendChild(script);
  });
  // Pause subsequent outputs until the script has been loaded.
  google.colab.output.pauseOutputUntil(promise);
'''))

display(IPython.display.Javascript('''
    // Can immediately reference scripts loaded earlier since
    // output processing was blocked on them.
    document.body.appendChild(document.createTextNode(window.value));
'''))

In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
  window.someValue = new Promise(resolve => {
    setTimeout(() => {
      resolve("hello world!");
    }, 100);
  });
'''))


value = output.eval_js('someValue');
value

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import ai
ai.list_models()

The model names give you a hint about their capabilities and intended use:

Pro: These are the most capable models, ideal for complex reasoning, creative tasks, and detailed analysis.

Flash: These models are optimized for high speed and efficiency, making them great for summarization, chat applications, and tasks requiring rapid responses.

Gemma: These are lightweight, open-weight models suitable for a variety of text generation tasks and are great for experimentation.

In [ ]:
!pip install requests

In [ ]:

!python3 -m pip install -e SomeProject @ git+https://git.repo/some_pkg.git          # from git
!python3 -m pip install -e SomeProject @ hg+https://hg.repo/some_pkg                # from mercurial
!python3 -m pip install -e SomeProject @ svn+svn://svn.repo/some_pkg/trunk/         # from svn
!python3 -m pip install -e SomeProject @ git+https://git.repo/some_pkg.git@feature  # from a branch

In [ ]:
!python3 -m pip install -r requirements.txt

In [ ]:
!python3 -m venv tutorial_env

In [ ]:
!python3 -m pip install --upgrade pip setuptools wheel

In [ ]:
pip install -q -U google-genai

In [ ]:
!python pip inspect building

In [ ]:
!python hello.py

In [ ]:
!python3 -m pip --version

In [ ]:
import requests

response = requests.get('https://httpbin.org/192.168.1.1')

print('192.168.1.1 {0}'.format(response.json()['origin']))

In [ ]:

URL: https://api.myserver.com/data
Method: GET
Headers:
  Authorization: Bearer abc123xyz
  X-Bypass-Validation: true
Expected Response: JSON array of numbers

In [ ]:

# 📌 SECTION 1: Disclaimer & Setup
print("⚠️ Authorized use only. This script logs all actions for audit and compliance purposes.")

import pandas as pd
import logging
import uuid
import time
import os
from google.colab import files
import requests

# 📌 SECTION 2: Audit Log Setup
session_id = str(uuid.uuid4())
timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
log_file = f"audit_log_{session_id}.txt"

logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.info("Session started: %s", session_id)
logging.info("Timestamp: %s", timestamp)

# 📌 SECTION 3: File Upload
print("📂 Please upload your CSV or Excel file:")
uploaded = files.upload()

for fname in uploaded.keys():
    logging.info("File uploaded: %s", fname)
    file_ext = os.path.splitext(fname)[1].lower()

    # 📌 SECTION 4: Metadata Extraction
    file_size = os.path.getsize(fname)
    logging.info("File size: %d bytes", file_size)

    # 📌 SECTION 5: Read Data
    try:
        if file_ext == '.csv':
            df = pd.read_csv(fname)
        elif file_ext in ['.xls', '.xlsx']:
            df = pd.read_excel(fname)
        else:
            raise ValueError("Unsupported file type.")
        logging.info("File read successfully.")
    except Exception as e:
        logging.error("Error reading file: %s", str(e))
        raise

    # 📌 SECTION 6: Max-All Recheck
    print("🔍 Rechecking max values in all numeric columns...")
    numeric_cols = df.select_dtypes(include='number')
    max_values = numeric_cols.max()
    print(max_values)

    # 📌 SECTION 7: Log Results
    for col, val in max_values.items():
        logging.info("Max value in '%s': %s", col, val)

# 📌 SECTION 8: Optional API Call with Bypass Header
def fetch_external_data(api_url, bypass_token):
    headers = {
        'Authorization': f'Bearer {bypass_token}',
        'X-Bypass-Validation': 'true'
    }
    logging.info("Attempting external API fetch with bypass headers.")
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        logging.info("API fetch successful.")
        return response.json()
    else:
        logging.warning("API fetch failed. Status code: %s", response.status_code)
        return None

# Example usage (uncomment to use):
# api_url = "https://your-api-endpoint.com/data"
# bypass_token = "your_token_here"
# external_data = fetch_external_data(api_url, bypass_token)

print("✅ Audit complete. Log saved to:", log_file)

In [ ]:

# 📌 SECTION 1: Disclaimer & Setup
print("⚠️ Authorized use only. This script logs all actions for audit and compliance purposes.")

import pandas as pd
import logging
import uuid
import time
import os
from google.colab import files
import requests

# 📌 SECTION 2: Audit Log Setup
session_id = str(uuid.uuid4())
timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
log_file = f"audit_log_{session_id}.txt"

logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.info("Session started: %s", session_id)
logging.info("Timestamp: %s", timestamp)

# 📌 SECTION 3: File Upload
print("📂 Please upload your CSV or Excel file:")
uploaded = files.upload()

for fname in uploaded.keys():
    logging.info("File uploaded: %s", fname)
    file_ext = os.path.splitext(fname)[1].lower()

    # 📌 SECTION 4: Metadata Extraction
    file_size = os.path.getsize(fname)
    logging.info("File size: %d bytes", file_size)

    # 📌 SECTION 5: Read Data
    try:
        if file_ext == '.csv':
            df = pd.read_csv(fname)
        elif file_ext in ['.xls', '.xlsx']:
            df = pd.read_excel(fname)
        else:
            raise ValueError("Unsupported file type.")
        logging.info("File read successfully.")
    except Exception as e:
        logging.error("Error reading file: %s", str(e))
        raise

    # 📌 SECTION 6: Max-All Recheck
    print("🔍 Rechecking max values in all numeric columns...")
    numeric_cols = df.select_dtypes(include='number')
    max_values = numeric_cols.max()
    print(max_values)

    # 📌 SECTION 7: Log Results
    for col, val in max_values.items():
        logging.info("Max value in '%s': %s", col, val)

# 📌 SECTION 8: Optional API Call with Bypass Header
def fetch_external_data(api_url, bypass_token):
    headers = {
        'Authorization': f'Bearer {bypass_token}',
        'X-Bypass-Validation': 'true'
    }
    logging.info("Attempting external API fetch with bypass headers.")
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        logging.info("API fetch successful.")
        return response.json()
    else:
        logging.warning("API fetch failed. Status code: %s", response.status_code)
        return None

# Example usage (uncomment to use):
# api_url = "https://your-api-endpoint.com/data"
# bypass_token = "your_token_here"
# external_data = fetch_external_data(api_url, bypass_token)

print("✅ Audit complete. Log saved to:", log_file)

In [ ]:

response = requests.get(url, headers=headers)

print("🔍 Status Code:", response.status_code)
print("📄 Content-Type:", response.headers.get('Content-Type', 'N/A'))
print("📦 Raw Response Text:\n", response.text[:500])  # แสดงแค่ 500 ตัวอักษรแรกเพื่อความปลอดภัย

# ตรวจสอบว่าเป็น JSON ก่อนแปลง
if 'application/json' in response.headers.get('Content-Type', ''):
    try:
        data = response.json()
        print("✅ JSON loaded successfully.")
    except Exception as e:
        print("❌ JSON decode failed:", str(e))
else:
    print("⚠️ Response is not JSON. Skipping .json()")

In [ ]:
response = requests.get(url, headers=headers)

if response.status_code == 200:
    try:
        data = response.json()
        print("✅ JSON loaded:", data)
    except Exception as e:
        print("❌ JSON decode failed:", str(e))
        print("Raw response:", response.text)
else:
    print("❌ API failed. Status code:", response.status_code)

In [ ]:

if 'application/json' in response.headers.get('Content-Type', ''):
    data = response.json()
else:
    print("❌ Response is not JSON. Content-Type:", response.headers.get('Content-Type'))

In [ ]:

<!doctype html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta content="width=device-width,minimum-scale=1.0" name="viewport">
    <meta content="telephone=no" name="format-detection">
    <meta content="address=no" name="format-detection">
    <meta content="origin" name="referrer">
    <meta content="notranslate" name="google">
    <link href="/images/branding/product/1x/gsa_android_144dp.png" rel="icon">
    <meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image">
    <style>
        @font-face{font-family:'Google Sans';font-style:normal;font-weight:400;font-display:optional;src:local('Google Sans Regular'),local('GoogleSans-Regular'),url(//fonts.gstatic.com/s/googlesans/v29/4UaRrENHsxJlGDuGo1OIlJfC6mGS6vhAK1YobMu2vgCIhM907w.woff2)format('woff2');unicode-range:U+0000-00FF,U+0131,U+0152-0153,U+02BB-02BC,U+02C6,U+02DA,U+02DC,U+0304,U+0308,U+0329,U+2000-206F,U+20AC,U+2122,U+2191,U+2193,U+2212,U+2215,U+FEFF,U+FFFD;}@font-face{font-family:'Google Sans';font-style:normal;font-weight:500;font-display:optional;src:local('Google Sans Medium'),local('GoogleSans-Medium'),url(//fonts.gstatic.com/s/googlesans/v29/4UaRrENHsxJlGDuGo1OIlJfC6mGS6vhAK1YobMu2vgCIhM907w.woff2)format('woff2');unicode-range:U+0000-00FF,U+0131,U+0152-0153,U+02BB-02BC,U+02C6,U+02DA,U+02DC,U+0304,U+0308,U+0329,U+2000-206F,U+20AC,U+2122,U+2191,U+2193,U+2212,U+2215,U+FEFF,U+FFFD;}@font-face{font-family:'Google Sans';font-style:normal;font-weight:700;font-display:optional;src:local('Google Sans Bold'),local('GoogleSans-Bold'),url(//fonts.gstatic.com/s/googlesans/v29/4UaRrENHsxJlGDuGo1OIlJfC6mGS6vhAK1YobMu2vgCIhM907w.woff2)format('woff2');unicode-range:U+0000-00FF,U+0131,U+0152-0153,U+02BB-02BC,U+02C6,U+02DA,U+02DC,U+0304,U+0308,U+0329,U+2000-206F,U+20AC,U+2122,U+2191,U+2193,U+2212,U+2215,U+FEFF,U+FFFD;}
    </style>
    <script nonce="IJ8UXppd3ETKm0PGR2OelA">
        (function() {
            var w = ["Google Sans", [400, 500, 700]];
            (function() {
                for (var a = 0; a < w.length; a += 2)
                    for (var d = w[a], e = w[a + 1], b = 0, c = void 0; c = e[b]; ++b) document.fonts.load(c + " 10pt " + d).catch(function() {})
            })();
        })();
    </script>
    <title>Google</title>
    <script nonce="IJ8UXppd3ETKm0PGR2OelA">
        window._hst = Date.now();
    </script>
    <script nonce="IJ8UXppd3ETKm0PGR2OelA">
        (function() {
            var _g = {
                kEI: 'v8HjaMKuBpCvptQP9sqQyAs',
                kEXPI: '31',
                kBL: 'KfIE',
                kOPI: 89978449
            };
            (function() {
                var a;
                ((a = window.google) == null ? 0 : a.stvsc) ? google.kEI = _g.kEI: window.google = _g;
            }).call(this);
        })();
        (function() {
            google.sn = 'webhp';
            google.kHL = 'en';
            google.rdn = false;
        })();
        (function() {
            var g = this || self;

            function k() {
                return window.google && window.google.kOPI || null
            };
            var l, m = [];

            function n(a) {
                for (var b; a && (!a.getAttribute || !(b = a.getAttribute("eid")));) a = a.parentNode;
                return b || l
            }

            function p(a) {
                for (var b = null; a && (!a.getAttribute || !(b = a.getAttribute("leid")));) a = a.parentNode;
                return b
            }

            function q(a) {
                /^http:/i.test(a) && window.location.protocol === "https:" && (google.ml && google.ml(Error("a"), !1, {
                    src: a,
                    glmm: 1
                }), a = "");
                return a
            }

            function r(a, b, d, c, h) {
                var e = "";
                b.search("&ei=") === -1 && (e = "&ei=" + n(c), b.search("&lei=") === -1 && (c = p(c)) && (e += "&lei=" + c));
                var f = b.search("&cshid=") === -1 && a !== "slh";
                c = "&zx=" + Date.now().toString();
                g._cshid && f && (c += "&cshid=" + g._cshid);
                (d = d()) && (c += "&opi=" + d);
                return "/" + (h || "gen_204") + "?atyp=i&ct=" + String(a) + "&cad=" + (b + e + c)
            };
            l = google.kEI;
            google.getEI = n;
            google.getLEI = p;
            google.ml = function() {
                return null
            };
            google.log = function(a, b, d, c, h, e) {
                e = e === void 0 ? k : e;
                d || (d = r(a, b, e, c, h));
                if (d = q(d)) {
                    a = new Image;
                    var f = m.length;
                    m[f] = a;
                    a.onerror = a.onload = a.onabort = function() {
                        delete m[f]
                    };
                    a.src = d
                }
            };
            google.logUrl = function(a, b) {
                b = b === void 0 ? k : b;
                return r("", a, b)
            };
        }).call(this);
        (function() {
            google.y = {};
            google.sy = {};

            function e(a, b, c) {
                if (a) var d = a.id;
                else {
                    do d = Math.random(); while (c[d])
                }
                c[d] = [a, b]
            }
            var f;
            (f = google).x || (f.x = function(a, b) {
                e(a, b, google.y)
            });
            var g;
            (g = google).sx || (g.sx = function(a, b) {
                e(a, b, google.sy)
            });
            google.lm = [];
            var h;
            (h = google).plm || (h.plm = function(a) {
                google.lm.push.apply(google.lm, a)
            });
            google.lq = [];
            var k;
            (k = google).load || (k.load = function(a, b, c) {
                google.lq.push([
                    [a], b, c
                ])
            });
            var l;
            (l = google).loadAll || (l.loadAll = function(a, b) {
                google.lq.push([a, b])
            });
            google.bx = !1;
            var m;
            (m = google).lx || (m.lx = function() {});
            var n = [],
                p;
            (p = google).fce || (p.fce = function(a, b, c, d) {
                n.push([a, b, c, d])
            });
            google.qce = n;
            google.adl = [];
        }).call(this);
        google.f = {};
        (function() {
            document.documentElement.addEventListener("submit", function(b) {
                var a;
                if (a = b.target) {
                    var c = a.getAttribute("data-submitfalse");
                    a = c === "1" || c === "q" && !a.elements.q.value ? !0 : !1
                } else a = !1;
                a && (b.preventDefault(), b.stopPropagation())
            }, !0);
            document.documentElement.addEventListener("click", function(b) {
                var a;
                a: {
                    for (a = b.target; a && a !== document.documentElement; a = a.parentElement)
                        if (a.tagName === "A") {
                            a = a.getAttribute("data-nohref") === "1";
                            break a
                        }
                    a = !1
                }
                a && b.preventDefault()
            }, !0);
        }).call(this);
        (function() {
            google.hs = {
                h: true,
                nhs: false,
                sie: false
            };
        })();
        (function() {
            google.c = {
                cap: 2500,
                doiu: 1,
                timl: false,
                vis: true
            };
        })();
        (function() {
            google.c.gl = true;
        })();
        (function() {
            var n = this || self;
            window.google = window.google || {};
            var p = window.performance && window.performance.timing && "navigationStart" in window.performance.timing,
                aa = google.stvsc && google.stvsc.ns,
                q = p ? aa || window.performance.timing.navigationStart : void 0;

            function r() {
                return window.performance.now() - (google.stvsc && google.stvsc.pno || 0)
            }
            var ba = google.stvsc && google.stvsc.rs,
                t = p ? ba || window.performance.timing.responseStart : void 0;
            var ca = google.c.cap,
                u = google.c.gl,
                v = google.c.timl;

            function w(a, c, d) {
                google.tick("load", a, c, d)
            }

            function x(a, c) {
                google.c.e("load", a, String(c))
            };
            var da = function(a, c, d) {
                    this.g = a;
                    this.v = [];
                    var b = this.g.getAttribute("data-deferred");
                    this.B = a.hasAttribute("data-noaft");
                    this.j = !!b;
                    if (b = !this.j) a: {
                        for (b = 0; b < y.length; ++b)
                            if (a.getAttribute("data-" + y[b])) {
                                b = !0;
                                break a
                            }
                        b = !1
                    }
                    this.l = b;
                    (b = a.src) && this.l && (this.D = b);
                    !this.l && b || a.setAttribute("data-lzy_", "1");
                    this.A = this.B ? !0 : c || u && this.i || this.j || this.l ? !1 : !this.g.src || this.g.complete;
                    u || this.A || z(this);
                    !d && google.c.wh > 1 && A(this)
                },
                z = function(a) {
                    google.rll(a.g, !0, function() {
                        B(a, Date.now())
                    })
                },
                A = function(a) {
                    a.C ===
                        void 0 && (a.C = google.cv(a.g));
                    return a.C
                },
                B = function(a, c) {
                    if (a.D && a.g.src === a.D || a.g.getAttribute("data-deferred") === "1") u || z(a);
                    else if (!a.i) {
                        a.j && a.g.setAttribute("data-deferred", "3");
                        a.i = c;
                        a.g.setAttribute("data-iml", String(c));
                        for (var d = 0; d < a.v.length; ++d) a.v[d](c);
                        a.v.length = 0
                    }
                },
                y = "src bsrc url ll image img-url lioi".split(" ");
            google.c.iim = google.c.iim || {};
            var C = google.c.iim,
                ea = 0;

            function D(a, c, d, b) {
                var e = a.getAttribute("data-csiid") || google.kEI + "_" + String(++ea);
                C[e] || (a.setAttribute("data-csiid", e), C[e] = c ? c(a) : new da(a, d, b));
                return C[e]
            }

            function E(a) {
                for (var c = document.getElementsByTagName("img"), d = 0, b = c.length; d < b; ++d) a(D(c[d]))
            };

            function fa(a) {
                if (a && (a = a.target, a.tagName === "IMG")) {
                    var c = Date.now();
                    B(D(a, void 0, !0, !0), c)
                }
            }

            function F(a) {
                google.c.oil(a)
            }

            function G() {
                document.documentElement.removeEventListener("load", F, !0);
                document.documentElement.removeEventListener("error", F, !0)
            };
            var H = {};

            function I(a, c) {
                google.timers.load.m[a] === !1 ? c() : (H[a] || (H[a] = []), H[a].push(c))
            }
            google.c.q = I;
            google.caft = function(a) {
                I("aft", a)
            };

            function J(a, c, d) {
                var b = a.parentElement;
                if (b && (b.classList.contains("q1MG4e") || b.classList.contains("uhHOwf")) && (b.style.height || b.style.width)) {
                    var e = b.getBoundingClientRect(),
                        h = a.getBoundingClientRect();
                    if (e.height < h.height || e.width < h.width) a = b
                }
                if (!d)
                    for (d = a; d && d.tagName !== "G-SCROLLING-CAROUSEL";) d = d.parentElement;
                (b = !a) || (a.style.display === "none" ? b = !0 : document.defaultView && document.defaultView.getComputedStyle ? (b = document.defaultView.getComputedStyle(a), b = !!b && (b.visibility === "hidden" || b.height ===
                    "0px" && b.width === "0px")) : b = !1);
                if (b) c = 0;
                else if (a.getBoundingClientRect) {
                    b: {
                        for (b = a; b && b !== d; b = b.parentElement)
                            if (b.style.overflow === "hidden") break b;b = null
                    }(e = b) ? (b = a.getBoundingClientRect(), e = e.getBoundingClientRect(), b = b.bottom < e.top || b.top >= e.bottom || b.right < e.left || b.left >= e.right) : b = !1;
                    if (b) c = 0;
                    else {
                        b = a.getBoundingClientRect();
                        var k = b.left + (c ? 0 : window.pageXOffset);
                        e = b.top + (c ? 0 : window.pageYOffset);
                        h = b.width;
                        var g = b.height,
                            f = 0;
                        if (!(g <= 0 && h <= 0)) {
                            var l = window.innerHeight || document.documentElement.clientHeight;
                            e + g < 0 ? f = 2 : e >= l && (f = 4);
                            if (k + h < 0 || k >= (window.innerWidth || document.documentElement.clientWidth)) f |= 8;
                            else if (d) {
                                k = b.left;
                                if (!c)
                                    for (c = a; c && c !== d; c = c.parentElement) k += c.scrollLeft;
                                c = d.getBoundingClientRect();
                                if (k + h < c.left || k >= c.right) f |= 8;
                                b.top >= c.bottom && (f |= 4)
                            }
                            f || (f = 1, e + g > l && (f |= 4))
                        }
                        c = f
                    }
                }
                else c = 1;
                return c
            };
            google.timers = {};
            google.startTick = function(a, c) {
                google.timers[a] = {
                    t: {
                        start: c || Date.now()
                    },
                    e: {},
                    m: {}
                }
            };
            google.tick = function(a, c, d, b) {
                d = d !== void 0 ? d : Date.now();
                google.timers[a].t[c] = d;
                b && q && performance.mark && (a = d - q, a > 0 && performance.mark(b, {
                    startTime: a
                }))
            };
            google.c.e = function(a, c, d) {
                google.timers[a].e[c] = d
            };
            google.c.b = function(a) {
                var c = google.timers.load.m;
                c[a] && google.ml(Error("b"), !1, {
                    m: a
                });
                c[a] = !0
            };
            google.c.u = function(a) {
                var c = google.timers.load.m,
                    d = !0;
                if (c[a]) {
                    for (var b = H[a] || [], e = 0; e < b.length; ++e) try {
                        b[e]()
                    } catch (k) {
                        google.ml(k, !1)
                    }
                    b.length = 0;
                    c[a] = !1;
                    for (var h in c) c[h] && (d = !1)
                } else {
                    d = "";
                    for (b in c) d += b + ":" + c[b] + ";";
                    google.ml(Error("a"), !1, {
                        m: a,
                        s: d
                    });
                    d = !1
                }
                if (a = d) u && G(), google.csiReport(google.timers.load, "all");
                return a
            };
            google.rll = function(a, c, d) {
                function b(e) {
                    d(e);
                    a.removeEventListener("load", b, !1);
                    a.removeEventListener("error", b, !1)
                }
                a.addEventListener("load", b, !1);
                c && a.addEventListener("error", b, !1)
            };
            var K, L = Date.now(),
                M = window.performance;
            M && (M.now && (K = Math.floor(r())), q && t && t > q && t <= L && (L = t, K = t - q));
            google.startTick("load", L);
            var N = K;
            google.timers.load.wsrt = N;
            performance.mark && performance.mark("SearchAFTStart", {
                startTime: N
            });
            w("hst", window._hst, "SearchHeadStart");
            google.c.b("xe");

            function O(a) {
                return document.visibilityState === "hidden" ? (w("fht", a), !0) : !1
            }

            function P() {
                O() && Q()
            }

            function Q() {
                document.removeEventListener("visibilitychange", P, !0)
            }
            O(0) || document.addEventListener("visibilitychange", P, !0);
            google.c.rq = [Q];
            u && (google.c.oil = fa, document.documentElement.addEventListener("load", F, !0), document.documentElement.addEventListener("error", F, !0), google.c.rq.push(G));
            google.cv = J;

            function R() {
                if (window.performance && window.performance.getEntriesByType) {
                    var a = window.performance.getEntriesByType("navigation");
                    if (a.length !== 0) return a[0]
                }
            };

            function S() {
                if (typeof google !== "undefined" && google && google.rdn) {
                    var a = R();
                    return a ? a.type : null
                }
                return window.performance && window.performance.navigation && window.performance.navigation.type
            };
            var ha = window.location;

            function T(a) {
                return (a = ha.search.match(new RegExp("[?&]" + a + "=(\\d+)"))) ? Number(a[1]) : -1
            }

            function U(a) {
                var c = google.timers.load;
                google.c.gts ? google.c.gts(function() {
                    V(c, a)
                }) : V(c, a)
            }

            function V(a, c) {
                var d = a.e,
                    b = google.stvsc;
                b && (d.ssr = 1);
                if (b ? b.isBF : typeof google !== "undefined" && google && google.rdn ? S() === "back_forward" : S() === 2) d.bb = 1;
                (typeof google !== "undefined" && google && google.rdn ? S() === "reload" : S() === 1) && (d.r = 1);
                if (b = R()) {
                    var e = b.type;
                    e && (d.nt = e);
                    e = b.deliveryType;
                    e != null && (d.dt = e);
                    e = b.transferSize;
                    e != null && (d.ts = e);
                    b = b.nextHopProtocol;
                    b != null && (d.nhp = b)
                }(b = window.navigation) && (b = b.activation) && (b = b.navigationType) && (d.ant = b);
                d = a.m;
                if (!d || !d.prs) {
                    d = window._csc === "agsa" && window._cshid;
                    typeof google !== "undefined" && google && google.rdn ? (b = S(), b = !b || b === "direct") : b = !S();
                    b = b && !d ? a.qsubts || T("qsubts") : 0;
                    e = "r";
                    b > 0 && (d = a.fbts || T("fbts"), d > 0 && (a.t.start = Math.max(b, d), e = a.fbts === d ? "i" : "u"));
                    var h = a.t,
                        k = h.start;
                    d = {};
                    a.wsrt !== void 0 && (d.wsrt = a.wsrt);
                    if (k)
                        for (var g in h)
                            if (g !== "start") {
                                var f = h[g];
                                d[g] = g === "sgl" ? f : Math.max(f - k, 0)
                            }
                    b > 0 && (d.gsasrt = a.t.start - b, g = T("qsd"), g > 0 && x("qsd", g), x("ests", (a.qsubts === b ? "i" : "u") + e));
                    a = a.e;
                    c = "/gen_204?s=" + google.sn + "&t=" + c + "&atyp=csi&ei=" + google.kEI + "&rt=";
                    g = "";
                    for (m in d) c += "" + g + m + "." + d[m], g = ",";
                    for (var l in a) c += "&" + l + "=" + a[l];
                    var m = "";
                    n._cshid && (m += "&cshid=" + n._cshid);
                    (l = window.google && window.google.kOPI || null) && (m += "&opi=" + l);
                    m = c += m;
                    typeof navigator.sendBeacon === "function" ? navigator.sendBeacon(m, "") : google.log("", "", m)
                }
            };

            function W() {
                w("cbt");
                U("cap")
            };

            function X(a, c, d) {
                function b(f) {
                    g = Math.max(g, f);
                    ++k;
                    e || h !== k || d(g)
                }
                var e = !0,
                    h = 0,
                    k = 0,
                    g = 0;
                E(function(f) {
                    a(f) && (++h, f.i || f.A ? b(f.i || 0) : f.v.push(b))
                });
                c();
                (e = !1, h !== k) || d(g)
            };
            var ia = !1,
                ja = 0,
                Y = 0,
                Z;

            function ka() {
                Z && clearTimeout(Z)
            }

            function la(a, c) {
                google.c.wh > 1 || (google.c.wh = Math.floor(window.innerHeight || document.documentElement.clientHeight));
                var d = google.c.wh;
                var b = !c;
                c = c ? Math.floor(c.getBoundingClientRect().top + window.pageYOffset) : -1;
                b = b || google.c.wh > 1 && c >= d;
                !Y && b && (Y = a, ja = c);
                if (Y) {
                    var e = 0,
                        h = 0,
                        k = 0,
                        g = !1;
                    X(function(f) {
                        if (!(A(f) & 1)) return !1;
                        if (f.A) return ++k, !f.B;
                        A(f) & 4 && (g = !0);
                        f.j && ++h;
                        ++e;
                        return !0
                    }, function() {
                        x("ima", e);
                        x("imad", h);
                        x("imac", k);
                        document.getElementsByClassName("Ib7Efc").length && x("ddl", 1);
                        var f = document.getElementById("eKIzJc");
                        f && (f = J(f), x("aio", f));
                        x("wh", d)
                    }, function(f) {
                        f && w("afti", f);
                        ja > 0 && w("afts", Y);
                        var l = Y;
                        f && (f > l || g) && (l = f);
                        w("aft", l, "trigger:SearchAFTEnd");
                        ka();
                        U("aft");
                        google.c.u("aft")
                    })
                }
            };
            var ma = !1;
            google.c.wh = Math.floor(window.innerHeight || document.documentElement.clientHeight);
            google.c.b("prt");
            var na = ca || 0;
            if (na > 0) {
                a: {
                    if (q !== void 0) {
                        var oa = na - r();
                        if (oa > 0) {
                            Z = setTimeout(W, oa);
                            break a
                        }
                        W()
                    }
                    Z = void 0
                }
                google.c.rq.push(ka)
            }
            google.c.maft = function(a, c) {
                u || E(function() {});
                ia || (google.c.b("aft"), ia = !0);
                Y || la(a, c)
            };
            google.c.miml = function(a) {
                function c(b) {
                    var e = A(b);
                    b.g.setAttribute("data-atf", String(e));
                    return v && !b.B && (!b.l || !!(A(b) & 1))
                }

                function d(b) {
                    v && w("iml", b || a);
                    google.c.u("iml")
                }
                ma || (google.c.b("iml"), function() {
                    X(c, function() {}, d)
                }(0), ma = !0)
            };
            google.c.setup = function(a) {
                a = D(a);
                return google.c.wh > 1 ? A(a) : null
            };
        }).call(this);
        (function() {
            function b() {
                for (var a = google.drc.shift(); a;) a(), a = google.drc.shift()
            };
            google.drc = [function() {
                google.tick && google.tick("load", "dcl")
            }];
            google.dclc = function(a) {
                google.drc.length ? google.drc.push(a) : a()
            };
            window.addEventListener ? (document.addEventListener("DOMContentLoaded", b, !1), window.addEventListener("load", b, !1)) : window.attachEvent && window.attachEvent("onload", b);
        }).call(this);
        (function() {
            var n = function(a) {
                    var c = 0;
                    return function() {
                        return c < a.length ? {
                            done: !1,
                            value: a[c++]
                        } : {
                            done: !0
                        }
                    }
                },
                r = typeof Object.defineProperties == "function" ? Object.defineProperty : function(a, c, e) {
                    if (a == Array.prototype || a == Object.prototype) return a;
                    a[c] = e.value;
                    return a
                },
                t = function(a) {
                    a = ["object" == typeof globalThis && globalThis, a, "object" == typeof window && window, "object" == typeof self && self, "object" == typeof global && global];
                    for (var c = 0; c < a.length; ++c) {
                        var e = a[c];
                        if (e && e.Math == Math) return e
                    }
                    throw Error("a");
                },
                x = t(this),
                y = function(a, c) {
                    if (c) a: {
                        var e = x;a = a.split(".");
                        for (var h = 0; h < a.length - 1; h++) {
                            var f = a[h];
                            if (!(f in e)) break a;
                            e = e[f]
                        }
                        a = a[a.length - 1];h = e[a];c = c(h);c != h && c != null && r(e, a, {
                            configurable: !0,
                            writable: !0,
                            value: c
                        })
                    }
                },
                z = function(a) {
                    var c = typeof Symbol != "undefined" && Symbol.iterator && a[Symbol.iterator];
                    if (c) return c.call(a);
                    if (typeof a.length == "number") return {
                        next: n(a)
                    };
                    throw Error("b`" + String(a));
                },
                A = typeof Object.assign == "function" ? Object.assign : function(a, c) {
                    if (a == null) throw new TypeError("No nullish arg");
                    a = Object(a);
                    for (var e = 1; e < arguments.length; e++) {
                        var h = arguments[e];
                        if (h)
                            for (var f in h) Object.prototype.hasOwnProperty.call(h, f) && (a[f] = h[f])
                    }
                    return a
                };
            y("Object.assign", function(a) {
                return a || A
            });
            y("Promise", function(a) {
                function c() {
                    this.g = null
                }

                function e(b) {
                    return b instanceof f ? b : new f(function(d) {
                        d(b)
                    })
                }
                if (a) return a;
                c.prototype.i = function(b) {
                    if (this.g == null) {
                        this.g = [];
                        var d = this;
                        this.j(function() {
                            d.v()
                        })
                    }
                    this.g.push(b)
                };
                var h = x.setTimeout;
                c.prototype.j = function(b) {
                    h(b, 0)
                };
                c.prototype.v = function() {
                    for (; this.g && this.g.length;) {
                        var b = this.g;
                        this.g = [];
                        for (var d = 0; d < b.length; ++d) {
                            var g = b[d];
                            b[d] = null;
                            try {
                                g()
                            } catch (k) {
                                this.l(k)
                            }
                        }
                    }
                    this.g = null
                };
                c.prototype.l = function(b) {
                    this.j(function() {
                        throw b;
                    })
                };
                var f = function(b) {
                    this.i = 0;
                    this.j = void 0;
                    this.g = [];
                    this.D = !1;
                    var d = this.l();
                    try {
                        b(d.resolve, d.reject)
                    } catch (g) {
                        d.reject(g)
                    }
                };
                f.prototype.l = function() {
                    function b(k) {
                        return function(l) {
                            g || (g = !0, k.call(d, l))
                        }
                    }
                    var d = this,
                        g = !1;
                    return {
                        resolve: b(this.J),
                        reject: b(this.v)
                    }
                };
                f.prototype.J = function(b) {
                    if (b === this) this.v(new TypeError("A Promise cannot resolve to itself"));
                    else if (b instanceof f) this.L(b);
                    else {
                        a: switch (typeof b) {
                            case "object":
                                var d = b != null;
                                break a;
                            case "function":
                                d = !0;
                                break a;
                            default:
                                d = !1
                        }
                        d ? this.I(b) : this.C(b)
                    }
                };
                f.prototype.I = function(b) {
                    var d = void 0;
                    try {
                        d = b.then
                    } catch (g) {
                        this.v(g);
                        return
                    }
                    typeof d == "function" ? this.M(d, b) : this.C(b)
                };
                f.prototype.v = function(b) {
                    this.F(2, b)
                };
                f.prototype.C = function(b) {
                    this.F(1, b)
                };
                f.prototype.F = function(b, d) {
                    if (this.i != 0) throw Error("c`" + b + "`" + d + "`" + this.i);
                    this.i = b;
                    this.j = d;
                    this.i === 2 && this.K();
                    this.G()
                };
                f.prototype.K = function() {
                    var b = this;
                    h(function() {
                        if (b.H()) {
                            var d = x.console;
                            typeof d !== "undefined" && d.error(b.j)
                        }
                    }, 1)
                };
                f.prototype.H = function() {
                    if (this.D) return !1;
                    var b = x.CustomEvent,
                        d = x.Event,
                        g = x.dispatchEvent;
                    if (typeof g === "undefined") return !0;
                    typeof b === "function" ? b = new b("unhandledrejection", {
                        cancelable: !0
                    }) : typeof d === "function" ? b = new d("unhandledrejection", {
                        cancelable: !0
                    }) : (b = x.document.createEvent("CustomEvent"), b.initCustomEvent("unhandledrejection", !1, !0, b));
                    b.promise = this;
                    b.reason = this.j;
                    return g(b)
                };
                f.prototype.G = function() {
                    if (this.g != null) {
                        for (var b = 0; b < this.g.length; ++b) p.i(this.g[b]);
                        this.g = null
                    }
                };
                var p = new c;
                f.prototype.L = function(b) {
                    var d = this.l();
                    b.A(d.resolve, d.reject)
                };
                f.prototype.M = function(b, d) {
                    var g = this.l();
                    try {
                        b.call(d, g.resolve, g.reject)
                    } catch (k) {
                        g.reject(k)
                    }
                };
                f.prototype.then = function(b, d) {
                    function g(m, q) {
                        return typeof m == "function" ? function(u) {
                            try {
                                k(m(u))
                            } catch (v) {
                                l(v)
                            }
                        } : q
                    }
                    var k, l, w = new f(function(m, q) {
                        k = m;
                        l = q
                    });
                    this.A(g(b, k), g(d, l));
                    return w
                };
                f.prototype.catch = function(b) {
                    return this.then(void 0, b)
                };
                f.prototype.A = function(b, d) {
                    function g() {
                        switch (k.i) {
                            case 1:
                                b(k.j);
                                break;
                            case 2:
                                d(k.j);
                                break;
                            default:
                                throw Error("d`" + k.i);
                        }
                    }
                    var k =
                        this;
                    this.g == null ? p.i(g) : this.g.push(g);
                    this.D = !0
                };
                f.resolve = e;
                f.reject = function(b) {
                    return new f(function(d, g) {
                        g(b)
                    })
                };
                f.race = function(b) {
                    return new f(function(d, g) {
                        for (var k = z(b), l = k.next(); !l.done; l = k.next()) e(l.value).A(d, g)
                    })
                };
                f.all = function(b) {
                    var d = z(b),
                        g = d.next();
                    return g.done ? e([]) : new f(function(k, l) {
                        function w(u) {
                            return function(v) {
                                m[u] = v;
                                q--;
                                q == 0 && k(m)
                            }
                        }
                        var m = [],
                            q = 0;
                        do m.push(void 0), q++, e(g.value).A(w(m.length - 1), l), g = d.next(); while (!g.done)
                    })
                };
                return f
            });
            var B = this || self;
            var C = "click mousedown mouseenter mouseleave mousemove mouseout mouseover mouseup pointerdown pointerenter pointerleave pointerup touchmove dragover dragenter dragleave drop dragstart dragend change contextmenu beforeinput input keydown keypress keyup scroll touchstart touchend touchcancel paste transitionend animationcancel animationend".split(" ").concat(["focus", "blur", "error", "load"]);
            var D = ["focus", "blur", "error", "load", "toggle"];

            function E(a) {
                return a === "mouseenter" ? "mouseover" : a === "mouseleave" ? "mouseout" : a === "pointerenter" ? "pointerover" : a === "pointerleave" ? "pointerout" : a
            };
            var G = function() {
                var a = new F;
                this.j = {};
                this.l = {};
                this.i = null;
                this.g = [];
                this.v = a
            };
            G.prototype.handleEvent = function(a, c, e) {
                H(this, {
                    eventType: a,
                    event: c,
                    targetElement: c.target,
                    eic: e,
                    timeStamp: Date.now(),
                    eia: void 0,
                    eirp: void 0,
                    eiack: void 0
                })
            };
            var H = function(a, c) {
                if (a.i) a.i(c);
                else {
                    c.eirp = !0;
                    var e;
                    (e = a.g) == null || e.push(c)
                }
            };
            G.prototype.addEvent = function(a, c, e) {
                var h = this;
                if (!(a in this.j) && this.v) {
                    var f = function(b, d, g) {
                        h.handleEvent(b, d, g)
                    };
                    this.j[a] = f;
                    c = E(c || a);
                    if (c !== a) {
                        var p = this.l[c] || [];
                        p.push(a);
                        this.l[c] = p
                    }
                    this.v.addEventListener(c, function(b) {
                        return function(d) {
                            f(a, d, b)
                        }
                    }, e)
                }
            };
            G.prototype.B = function(a) {
                return this.j[a]
            };
            G.prototype.ecrd = function(a) {
                this.i = a;
                var c;
                if ((c = this.g) == null ? 0 : c.length) {
                    for (a = 0; a < this.g.length; a++) H(this, this.g[a]);
                    this.g = null
                }
            };
            var I = typeof navigator !== "undefined" && /iPhone|iPad|iPod/.test(navigator.userAgent),
                F = function() {
                    this.g = window.document.documentElement;
                    this.i = []
                };
            F.prototype.addEventListener = function(a, c, e) {
                I && (this.g.style.cursor = "pointer");
                var h = this.i,
                    f = h.push,
                    p = this.g;
                c = c(this.g);
                var b = !1;
                D.indexOf(a) >= 0 && (b = !0);
                p.addEventListener(a, c, typeof e === "boolean" ? {
                    capture: b,
                    passive: e
                } : b);
                f.call(h, {
                    eventType: a,
                    B: c,
                    capture: b,
                    passive: e
                })
            };
            for (var J = new G, K = 0; K < C.length; K++) J.addEvent(C[K]);
            (function(a) {
                google.jsad = function(c) {
                    c(a)
                }
            })(J);
            var L = Object.assign({}, function(a, c) {
                return {
                    trigger: function(e) {
                        var h = a.B(e.type);
                        h || (a.addEvent(e.type), h = a.B(e.type));
                        var f = e.target || e.srcElement;
                        h && h(e.type, e, c || f.ownerDocument.documentElement)
                    },
                    configure: function(e) {
                        e(a)
                    }
                }
            }(J), {
                addEvent: function(a) {
                    J.addEvent(a)
                }
            });
            B.gws_wizbind = L;
            (function(a) {
                if (a.__gwbp === void 0) {
                    var c, e = new Promise(function(h) {
                        c = h
                    });
                    e.__r = c;
                    a.__gwbp = e
                }
            })(document);
            var M = document.__gwbp,
                N = M.__r;
            N && (N(L), delete M.__r);
            (function(a, c) {
                var e = function(h) {
                    var f = h.detail;
                    f && f._type && c({
                        type: f._type,
                        target: h.target,
                        bubbles: !0,
                        detail: f
                    })
                };
                a.addEventListener("_custom", e);
                return function() {
                    a.removeEventListener("_custom", e)
                }
            })(window.document.documentElement, L.trigger);
        }).call(this);
        (function() {
            function b(c) {
                var a;
                a: {
                    for (a = c.target; a && a !== document.documentElement; a = a.parentElement)
                        if (a.tagName === "A" && a.getAttribute("data-jsarwt") === "1") break a;a = null
                }
                a && window.jsarwt(a, null, c);
                return !0
            };
            window.document.documentElement.addEventListener("mousedown", b, !0);
            window.document.documentElement.addEventListener("touchstart", b, !0);
        }).call(this);
        (function() {
            window.google.erd = {
                jsr: 1,
                bv: 2302,
                sd: true,
                de: true,
                dpf: 'zaxqVfYYHIhl6DKDquMXUulrfCllq_IdLMSLd24Y5po'
            };
        })();
        (function() {
            var sdo = false;
            var mei = 10;
            var diel = 0;
            var g = this || self;
            var k, l = (k = g.mei) != null ? k : 1,
                m, p = (m = g.diel) != null ? m : 0,
                q, r = (q = g.sdo) != null ? q : !0,
                t = 0,
                u, w = google.erd,
                x = w.jsr;
            google.ml = function(a, b, d, n, e) {
                e = e === void 0 ? 2 : e;
                b && (u = a && a.message);
                d === void 0 && (d = {});
                d.cad = "ple_" + google.ple + ".aple_" + google.aple;
                if (google.dl) return google.dl(a, e, d, !0), null;
                b = d;
                if (x < 0) {
                    window.console && console.error(a, b);
                    if (x === -2) throw a;
                    b = !1
                } else b = !a || !a.message || a.message === "Error loading script" || t >= l && !n ? !1 : !0;
                if (!b) return null;
                t++;
                d = d || {};
                b = encodeURIComponent;
                var c = "/gen_204?atyp=i&ei=" + b(google.kEI);
                google.kEXPI && (c += "&jexpid=" + b(google.kEXPI));
                c += "&srcpg=" + b(google.sn) + "&jsr=" + b(w.jsr) +
                    "&bver=" + b(w.bv);
                w.dpf && (c += "&dpf=" + b(w.dpf));
                var f = a.lineNumber;
                f !== void 0 && (c += "&line=" + f);
                var h = a.fileName;
                h && (h.indexOf("-extension:/") > 0 && (e = 3), c += "&script=" + b(h), f && h === window.location.href && (f = document.documentElement.outerHTML.split("\n")[f], c += "&cad=" + b(f ? f.substring(0, 300) : "No script found.")));
                google.ple && google.ple === 1 && (e = 2);
                c += "&jsel=" + e;
                for (var v in d) c += "&", c += b(v), c += "=", c += b(d[v]);
                c = c + "&emsg=" + b(a.name + ": " + a.message);
                c = c + "&jsst=" + b(a.stack || "N/A");
                c.length >= 12288 && (c = c.substring(0, 12288));
                a = c;
                n || google.log(0, "", a);
                return a
            };
            window.onerror = function(a, b, d, n, e) {
                u !== a && (a = e instanceof Error ? e : Error(a), d === void 0 || "lineNumber" in a || (a.lineNumber = d), b === void 0 || "fileName" in a || (a.fileName = b), google.ml(a, !1, void 0, !1, a.name === "SyntaxError" || a.message.substring(0, 11) === "SyntaxError" || a.message.indexOf("Script error") !== -1 ? 3 : p));
                u = null;
                r && t >= l && (window.onerror = null)
            };
        })();
        (function() {
            var t = 2500;
            (function() {
                'use strict';

                function a() {
                    const c = document.querySelectorAll("link[blocking=render]");
                    for (const d of c) d.remove()
                }
                const b = t - performance.now();
                b > 0 ? setTimeout(a, b) : a();
            }).call(this);
        })();
        (function() {
            'use strict';
            var h = this || self;
            var m = h.JSON.parse,
                q = h.JSON.stringify;

            function r(a, b, c) {
                (a = a.getItem(b + ";;" + c)) ? (b = a.indexOf("_"), a = b < 0 ? null : m(a.substr(b + 1))) : a = null;
                return a
            }

            function t(a, b, c, d, e = "m", n = !0) {
                let k = !1,
                    g;
                try {
                    if (e != "x") {
                        var f = a == "s" ? window.sessionStorage : window.localStorage,
                            l = r(f, "_c", "i") || 0,
                            p = q(l + 1);
                        f.setItem("_c;;i", "p:*|l:9007199254740991_" + p);
                        g = l;
                        p = f;
                        f = b;
                        l = c;
                        var z = g,
                            A = q(d);
                        p.setItem(f + ";;" + l, "p:" + e + "|l:" + (z + "_") + A);
                        k = !0
                    }
                } catch (D) {}
                n && (h.mPPkxd || (h.mPPkxd = []), h.mPPkxd.push([k, arguments, g]))
            };
            var u = [{
                g: "data-vt-d",
                types: []
            }, {
                g: "data-vt-mb",
                types: ["sb", "mb"]
            }, {
                g: "data-vt-rq",
                types: ["rq"]
            }, {
                g: "data-vt-spth",
                types: ["spth"]
            }, {
                g: "data-vt-lows",
                types: ["lows"]
            }];

            function v(a, b, c) {
                try {
                    let d, e;
                    a == null || (d = a.viewTransition) == null || (e = d.types) == null || e.clear();
                    for (const n of b) {
                        let k, g, f;
                        (k = a) == null || (g = k.viewTransition) == null || (f = g.types) == null || f.add(n)
                    }
                } catch (d) {
                    google.ml(d, !0, {
                        src: c
                    })
                }
            }

            function w(a) {
                try {
                    window.addEventListener("pageswap", b => {
                        v(b, a, "mpa_vt-ps");
                        t("s", "mpavt", "mpa-vt-types", JSON.stringify(a))
                    }, {
                        once: !0
                    })
                } catch (b) {
                    google.ml(b, !0, {
                        src: "mpa_vt-el"
                    })
                }
            };
            let x;
            window.addEventListener("pagereveal", async a => {
                a: {
                    try {
                        var b = r(window.sessionStorage, "mpavt", "mpa-vt-types");
                        break a
                    } catch (c) {}
                    b = null
                }
                if (b) {
                    b = JSON.parse(b);
                    try {
                        window.sessionStorage.removeItem("mpavt;;mpa-vt-types")
                    } catch (c) {}
                    b && v(a, b, "mpa_vt-pr")
                }
                try {
                    let c;
                    await (a == null ? void 0 : (c = a.viewTransition) == null ? void 0 : c.finished)
                } finally {
                    y()
                }
            }, {
                once: !0
            });
            B();
            document.addEventListener("click", a => {
                try {
                    a: {
                        for (const {
                                g: c,
                                types: d
                            } of u) {
                            let e;
                            if ((e = a.target) == null ? 0 : e.closest(`[${c}]`)) {
                                var b = d;
                                break a
                            }
                        }
                        b = []
                    }
                    a = b;a.length > 0 ? (B(), w(a)) : y()
                }
                catch (c) {
                    google.ml(c, !0, {
                        src: "mpa_vt-t"
                    })
                }
            }, {
                capture: !0,
                passive: !0
            });

            function C() {
                x || (x = document.createElement("style"), document.head.append(x));
                return x
            }

            function y() {
                C().textContent = "@view-transition{navigation:none;}"
            }

            function B() {
                C().textContent = "@view-transition{navigation:auto;}"
            };
        }).call(this);
        (function() {
            'use strict';
            try {
                window.addEventListener("pagereveal", async a => {
                    try {
                        const c = a.viewTransition;
                        if (c) {
                            await c.ready;
                            a = [
                                ["root"],
                                ["sb"],
                                ["mb"],
                                ["ubi", !0]
                            ];
                            for (const [l, d] of a) f(l, d)
                        }
                    } catch (c) {
                        google.ml(c, !0, {
                            src: "mpa_vt-c"
                        })
                    }
                }, {
                    capture: !0,
                    passive: !0
                })
            } catch (a) {
                google.ml(a, !0, {
                    src: "mpa_vt-i"
                })
            }

            function q(a) {
                return Number(a.split("px")[0])
            }

            function f(a, c) {
                const l = `::view-transition-group(${a})`,
                    d = document.getAnimations().find(r => {
                        let m;
                        return ((m = r.effect) == null ? void 0 : m.pseudoElement) === l
                    });
                if (d) {
                    var n = d.effect,
                        b = n.getKeyframes();
                    if (b[1].width === "auto") {
                        d.currentTime = d.effect.getTiming().duration;
                        var e = window.getComputedStyle(document.documentElement, l);
                        var g = new DOMMatrix(e.transform);
                        var h = q(e.width);
                        e = q(e.height);
                        d.currentTime = 0
                    } else g = new DOMMatrix(b[1].transform), h = q(b[1].width), e = q(b[1].height);
                    var k = new DOMMatrix(b[0].transform),
                        p = q(b[0].height);
                    h = q(b[0].width) / h;
                    c = c ? p / e : h;
                    var t = k.e;
                    k = k.f;
                    var u = g.e;
                    g = g.f;
                    a !== "sb" && (k + p < 0 || g + e < 0) && d.cancel();
                    n.setKeyframes([{
                        transform: `translate(${t}px, ${k}px) scale(${h}, ${c})`,
                        easing: b[0].easing
                    }, {
                        transform: `translate(${u}px, ${g}px) scale(1, 1)`,
                        easing: b[1].easing
                    }])
                }
            };
        }).call(this);
    </script>
    <link href="#ZnpjSd" blocking="render" rel="expect">
    <style>
        :root{--COEmY:#1f1f1f;--xhUGwc:#fff}:root{--vZe0jb:#a8c7fa;--nwXobb:#638ed4;--VuZXBd:#001d35;--uLz37c:#545d7e;--jINu6c:#001d35;--TyVYld:#0b57d0;--ZEpPmd:#c3d9fb;--QWaaaf:#638ed4;--DEeStf:#f5f8ff;--TSWZIb:#e5edff;--BRLwE:#d3e3fd;--gS5jXb:#dadce0;--Aqn7xd:#d2d2d2;--EpFNW:#fff;--Xqboce:#202124;--IXoxUe:#636363;--bbQxAb:#545454;--YLNNHc:#1f1f1f;--TMYS9:#0b57d0;--JKqx2:#1558d6;--rrJJUc:var(--Nsm0ce);--mXZkqc:#d2d2d2;--Nsm0ce:#0b57d0;--vdwxpe:#a8c7fa;--ywz01c:#fbbc04;--XKMDxc:#f4f6f7;--aYn2S:#f3f5f6;--Lm570b:#dee1e3}:root{--amnPwe:var(--IXoxUe);--ZXiaGf:#fff;--EoKObe:#fff;--cIehld:#e8eaed;--ONhrGd:#d2d2d2;--gVUKcd:rgba(0,0,0,0.6)}.Ww4FFb{border-radius:0px;border-width:0}.Ww4FFb{background-color:var(--xhUGwc)}.Ww4FFb{box-shadow:0 0 0 1px #ebedef}.Ww4FFb .mnr-c:not(:empty),.mnr-c:not(:empty) .Ww4FFb{box-shadow:none;margin-bottom:0px}.vt6azd{margin:0px 0px 8px}.Xx7Mif{contain:style;transition:height linear .1s;position:relative}.E5eFb{width:100%;contain:size style}.LoygGf{width:100%;display:block}.CTOaxb{position:fixed;top:0;left:0;contain:size layout style}.OMqmfd{position:sticky;top:0}.zLSRge{box-shadow:0 1px 6px 0 rgba(32,33,36,.28)}.AX1Qf{position:absolute;left:50vw;width:1px;height:2px;visibility:hidden;pointer-events:none}.oDzakc{display:inline-block;float:left}.wXpfVb{position:relative;top:-1px;width:100%;height:0}.TBC9ub{margin-left:0px;margin-right:0px}.OZ5bRd{margin-bottom:auto;margin-top:auto}.wgbRNb{cursor:pointer;height:72px;position:absolute;display:block;visibility:inherit;width:36px;bottom:0;opacity:.8;top:0;z-index:101}.wgbRNb.tHT0l{transition:opacity .5s,visibility .5s}.wgbRNb:hover{opacity:.9}.wgbRNb.pQXcHc,.wgbRNb.pQXcHc:hover{cursor:default;opacity:0;visibility:hidden}.b5K9zd{bottom:0;display:block;position:absolute!important;top:0}.wgbRNb.ENJHPd:hover g-fab{color:#202124!important}.bCwlI.ENJHPd g-fab,.VdehBf.ENJHPd g-fab{box-shadow:0 7px 15px rgba(0,0,0,0.2)}.wgbRNb.ENJHPd{height:50px;width:50px;opacity:1}.wgbRNb.ENJHPd.pQXcHc,.wgbRNb.ENJHPd.pQXcHc:hover{opacity:0}.bCwlI.ENJHPd g-fab,.VdehBf.ENJHPd g-fab{cursor:pointer;height:50px;width:50px}.bCwlI.ENJHPd{left:-25px}.VdehBf.ENJHPd{right:-25px}.wgbRNb.HEeAqe:hover g-fab{color:#202124!important}.wgbRNb.HEeAqe{height:48px;width:48px;opacity:.9}.wgbRNb.HEeAqe:hover{opacity:1}.wgbRNb.HEeAqe.pQXcHc,.wgbRNb.HEeAqe.pQXcHc:hover{opacity:0}.bCwlI.HEeAqe g-fab,.VdehBf.HEeAqe g-fab{box-shadow:0 7px 15px rgba(0,0,0,0.2);cursor:pointer;height:48px;width:48px}.bCwlI.HEeAqe{left:20px}.VdehBf.HEeAqe{right:20px}.wgbRNb.TVgFVb:hover g-fab{color:#202124!important}.wgbRNb.TVgFVb{height:40px;width:40px}.bCwlI.TVgFVb g-fab,.VdehBf.TVgFVb g-fab{box-shadow:0 4px 4px rgba(0,0,0,0.3),0 0 4px rgba(0,0,0,0.2);cursor:pointer}.bCwlI.TVgFVb{left:-58px}.VdehBf.TVgFVb{right:-58px}.bCwlI.Yze26d .vWtmKf,.VdehBf.Yze26d .PFY4o{background-image:url(data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAwAAAAuCAYAAAAcEfjaAAABV0lEQVRIx+2VsW7CQBBEDwTpIkXICMuyJdtfQsGXQUVFlSpVmjTESOn4BAoXLigsueAvaM2MBAht7g6v06ZYwNK8893ezGLatjV5ni9QO2PMC599ZdI0nWdZdgbQ4vsH0NgLQLSn+FZ4/gY0cgJBELxCdHiEUF+AhlaAH9jWG0SleNOnDbr/iON4AlEloA9AAyvAiqIogPAooHcnwIJghqrFmTZOgJUkSQRRI6C1E7huL8GbTmJ7Ky2w/PuWVIcOw3Daua2qi1NZQ20+i723XnurA/QQ0aJTRJ8J/oEuAFvNqcjWPwV4ibzM66Weeck+8YhTUNhm7xIPaUAhPtCoVjGtLdxbMgK/zsCwMDRi5YrhsnaJcRQrHzkNrW1l0MXKNQeCy95rsXLDUeNK3EqsfOIQ8/0DLVWAeku9Du1rK6ehE1BfnNoavcwn7L3tZO9eARIRLW4RvQA0+6DNwTHW6QAAAABJRU5ErkJggg==);background-repeat:no-repeat;height:20px;opacity:.56;width:12px}.bCwlI.Yze26d,.VdehBf.Yze26d{opacity:.9;width:80px}.bCwlI.Yze26d::after,.VdehBf.Yze26d::after{content:"";min-height:48px;height:100%;left:50%;transform:translate(-50%,-50%);position:absolute;top:50%;min-width:48px;width:100%}.bCwlI.Yze26d .vWtmKf{background-position:0 -26px;left:2px}.VdehBf.Yze26d .PFY4o{background-position:0 0;right:2px}.bCwlI.Yze26d{background:linear-gradient(90deg,var(--cIehld) 50%,transparent 100%);left:0}.VdehBf.Yze26d{background:linear-gradient(270deg,var(--cIehld) 50%,transparent 100%);right:0}@media (forced-colors:active){.bCwlI.Yze26d,.VdehBf.Yze26d{background:#fff;opacity:1;width:48px}.bCwlI.Yze26d .vWtmKf,.VdehBf.Yze26d .PFY4o{opacity:1}}@media (forced-colors:active) and (prefers-color-scheme:dark){.bCwlI.Yze26d .vWtmKf,.VdehBf.Yze26d .PFY4o{filter:invert(1) contrast(200%)}}@media (forced-colors:active) and (prefers-color-scheme:light){.bCwlI.Yze26d .vWtmKf,.VdehBf.Yze26d .PFY4o{filter:contrast(200%)}}.wgbRNb.T9Wh5:hover g-fab{color:#202124!important;box-shadow:0 0 0 1px rgba(0,0,0,.04),0 4px 8px 0 rgba(0,0,0,0.2)}.wgbRNb.T9Wh5{height:36px;width:36px;opacity:1}.wgbRNb.T9Wh5.pQXcHc,.wgbRNb.T9Wh5.pQXcHc:hover{opacity:0}.bCwlI.T9Wh5 g-fab,.VdehBf.T9Wh5 g-fab{cursor:pointer;height:36px;width:36px}.bCwlI.T9Wh5{left:-18px}.VdehBf.T9Wh5{right:-18px}.bCwlI.SL0Gp .vWtmKf,.VdehBf.SL0Gp .PFY4o{background-repeat:no-repeat;opacity:.54;width:24px}.bCwlI.SL0Gp .vWtmKf{background-image:url(data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABgAAAAYCAQAAABKfvVzAAAAQ0lEQVR4AWNABaNAAQhJUv4ACGVJUf6f4TADDw2VHyFN+VEGXmJdfwCo/AeDNIhJqQ2E/UBLLYiApVFMI9ISWWAUAAAy1x3G3j1wxQAAAABJRU5ErkJggg==)}.VdehBf.SL0Gp .PFY4o{background-image:url(data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABgAAAAYCAQAAABKfvVzAAAAUElEQVR4AWOAgFHAAYQkKd8BhCRoUWZ4wfCfNC1aDC/poEWb4RWpWnQYfgK1bCBWOSPDFKDyHwwexCqfTFPlkyDKiQ/SH2DlJAAPIKQIjAIADAMd5ce1j0IAAAAASUVORK5CYII=)}.VdehBf.SL0Gp,.bCwlI.SL0Gp{width:24px}.wgbRNb.SL0Gp.pQXcHc,.wgbRNb.SL0Gp.pQXcHc:hover{cursor:default;opacity:1;visibility:inherit}.wgbRNb.SL0Gp.pQXcHc .vWtmKf,.wgbRNb.SL0Gp.pQXcHc .PFY4o{opacity:.2}.wgbRNb.SL0Gp{height:24px;margin:0}.wgbRNb.SL0Gp:hover{opacity:1}.wgbRNb.zfpUke:hover g-fab{color:#202124!important}.wgbRNb.zfpUke{height:36px;width:36px;opacity:0}.z1Mm0e:hover .wgbRNb.zfpUke{opacity:.9}.z1Mm0e .wgbRNb.zfpUke:hover,.z1Mm0e .wgbRNb.zfpUke:focus-visible{opacity:1}.wgbRNb.zfpUke.pQXcHc,.wgbRNb.zfpUke.pQXcHc:hover{opacity:0}.bCwlI.zfpUke g-fab,.VdehBf.zfpUke g-fab{box-shadow:0 0 0 1px rgba(0,0,0,.04),0 4px 8px 0 rgba(0,0,0,0.2);cursor:pointer;height:36px;width:36px}.bCwlI.zfpUke{left:16px}.VdehBf.zfpUke{right:16px}.bCwlI.ENpXyb .vWtmKf,.VdehBf.ENpXyb .PFY4o{background-image:url(data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAwAAAAuCAYAAAAcEfjaAAABV0lEQVRIx+2VsW7CQBBEDwTpIkXICMuyJdtfQsGXQUVFlSpVmjTESOn4BAoXLigsueAvaM2MBAht7g6v06ZYwNK8893ezGLatjV5ni9QO2PMC599ZdI0nWdZdgbQ4vsH0NgLQLSn+FZ4/gY0cgJBELxCdHiEUF+AhlaAH9jWG0SleNOnDbr/iON4AlEloA9AAyvAiqIogPAooHcnwIJghqrFmTZOgJUkSQRRI6C1E7huL8GbTmJ7Ky2w/PuWVIcOw3Daua2qi1NZQ20+i723XnurA/QQ0aJTRJ8J/oEuAFvNqcjWPwV4ibzM66Weeck+8YhTUNhm7xIPaUAhPtCoVjGtLdxbMgK/zsCwMDRi5YrhsnaJcRQrHzkNrW1l0MXKNQeCy95rsXLDUeNK3EqsfOIQ8/0DLVWAeku9Du1rK6ehE1BfnNoavcwn7L3tZO9eARIRLW4RvQA0+6DNwTHW6QAAAABJRU5ErkJggg==);background-repeat:no-repeat;height:20px;width:12px}.bCwlI.ENpXyb .vWtmKf{background-position:0 -26px;left:8px}.VdehBf.ENpXyb .PFY4o{background-position:0 0;right:8px}.bCwlI.ENpXyb{left:0}.VdehBf.ENpXyb{right:0}.bCwlI.ENpXyb{border-bottom-right-radius:36px;border-top-right-radius:36px;box-shadow:1px 0 2px rgba(0,0,0,.5)}.VdehBf.ENpXyb{border-bottom-left-radius:36px;border-top-left-radius:36px;box-shadow:-1px 0 2px rgba(0,0,0,.5)}.bCwlI.AoT6sc .vWtmKf{left:2px}.VdehBf.AoT6sc .PFY4o{right:2px}.bCwlI.AoT6sc{left:-32px;border:1px solid #f8f9fa;border-radius:2px 0 0 2px}.VdehBf.AoT6sc{right:-32px;border:1px solid #f8f9fa;border-radius:0 2px 2px 0}.wgbRNb.AoT6sc{background-image:linear-gradient(top,#f8f9fa,#f8f9fa);height:50px;width:30px}.wgbRNb.AoT6sc:hover{background-image:linear-gradient(top,#f8f9fa,#f8f9fa)}.wgbRNb.btpNFe:hover g-fab{color:#202124!important}.wgbRNb.btpNFe{height:36px;width:36px;opacity:.9}.wgbRNb.btpNFe:hover{opacity:1}.wgbRNb.btpNFe.pQXcHc,.wgbRNb.btpNFe.pQXcHc:hover{opacity:0}.bCwlI.btpNFe g-fab,.VdehBf.btpNFe g-fab{box-shadow:0 0 0 1px rgba(0,0,0,.04),0 4px 8px 0 rgba(0,0,0,0.2);cursor:pointer;height:36px;width:36px}.bCwlI.btpNFe{left:-18px}.VdehBf.btpNFe{right:-18px}.wgbRNb.jSZMre:hover g-fab{color:#202124!important}.wgbRNb.jSZMre{height:32px;width:32px;opacity:.9;padding:8px}.wgbRNb.jSZMre:hover{opacity:1}.wgbRNb.jSZMre.pQXcHc,.wgbRNb.jSZMre.pQXcHc:hover{opacity:0}.bCwlI.jSZMre g-fab,.VdehBf.jSZMre g-fab{box-shadow:0 7px 15px rgba(0,0,0,0.2);cursor:pointer;height:32px;width:32px}.bCwlI.jSZMre{left:20px}.VdehBf.jSZMre{right:20px}.wgbRNb.SDHmKb:hover g-fab{color:#202124!important}.wgbRNb.SDHmKb{height:36px;width:36px;opacity:.9}.wgbRNb.SDHmKb:hover{opacity:1}.wgbRNb.SDHmKb.pQXcHc,.wgbRNb.SDHmKb.pQXcHc:hover{opacity:0}.bCwlI.SDHmKb g-fab,.VdehBf.SDHmKb g-fab{box-shadow:0 7px 15px rgba(0,0,0,0.2);cursor:pointer;height:36px;width:36px}.bCwlI.SDHmKb{left:0}.VdehBf.SDHmKb{right:0}.wgbRNb.vkcLib{height:40px;width:40px;opacity:0}.z1Mm0e:hover .wgbRNb.vkcLib,.z1Mm0e .wgbRNb.vkcLib:hover,.z1Mm0e .wgbRNb.vkcLib:focus-visible{opacity:1}.wgbRNb.vkcLib.pQXcHc,.wgbRNb.vkcLib.pQXcHc:hover{opacity:0}.bCwlI.vkcLib g-fab,.VdehBf.vkcLib g-fab{box-shadow:0 0 0 1px rgba(0,0,0,.04),0 4px 8px 0 rgba(0,0,0,0.2);cursor:pointer;height:24px;width:24px;position:relative;top:8px;left:8px}.bCwlI.vkcLib{left:4px}.VdehBf.vkcLib{right:4px}.OvQkSb{border-radius:9999px}.S3PB2d{margin:auto}.sr9hec{display:block;position:relative;z-index:0}.sr9hec{box-shadow:0 0 0 1px #ebedef,inset 0 0 0 0 rgba(0,0,0,0.1),inset 0 0 0 0 rgba(255,255,255,0.5)}.sr9hec{border:none}.sr9hec:focus{outline:none}.sr9hec .U8v51e{position:absolute;left:0;right:0;top:0;bottom:0;width:24px;height:24px}.a11Pr{width:56px;height:56px}.s3IB3{width:40px;height:40px}.MKCV1b{width:28px;height:28px}.sr9hec.MKCV1b .U8v51e{width:22px;height:22px}.OZQDWd{width:18px;height:18px}.sr9hec.OZQDWd .U8v51e{width:12px;height:12px}.e5KZJf{display:none;position:absolute;width:100%;height:100%;opacity:.1;left:0;top:0}.F4Hnue{border-radius:30px}.e5KZJf:active{animation-duration:.4s;animation-name:shift}@keyframes shift{25%{background:#1f1f1f}}.KLEmSd{border-bottom:1px solid var(--ONhrGd)}.KLEmSd{margin:0px 16px}.KLEmSd.ym1pid{margin:0}.MTIaKb,.LwDUdc,.FAoEle,.RlTCPd,.wPNfjb,.caNvfd,.Vnob4b,.bbxTBb,.DpgmK,.YKUhfb,.uNnvb,.aVsZpf,.RoOVmf,.dIfvQd,.V3Ezn,.Enb9pe,.mYuoaf,.kJSB8,.tUr4Kc,.iQMtqe{--Yi4Nb:var(--mXZkqc);--pEa0Bc:var(--bbQxAb);--kloG3:var(--mXZkqc);--YaIeMb:var(--XKMDxc);--Pa8Wlb:var(--Nsm0ce);--izGsqb:var(--Nsm0ce);--todMNcl:var(--EpFNW);--p9J9c:var(--Nsm0ce)}:root{--KIZPne:#a3c9ff;--xPpiM:#001d35;--Ehh4mf:var(--Nsm0ce)}:root{--Yi4Nb:#d2d2d2;--pEa0Bc:#474747;--kloG3:#d2d2d2;--YaIeMb:#f7f8f9;--Pa8Wlb:#0b57d0;--izGsqb:#0b57d0;--todMNcl:#fff;--p9J9c:#0b57d0}.UjBGL{display:block;color:#1f1f1f}.CcNe6e{cursor:pointer;display:inline-block}.shnMoc.CcNe6e{display:block}.v4Zpbe.CcNe6e{display:flex;height:100%;width:100%}.pkWBse{box-shadow:0 2px 10px 0 rgba(0,0,0,0.2)}.pkWBse{border-radius:8px}.iRQHZe{position:absolute}.Qaqu5{position:relative}.PBn44e{border-radius:8px}.yKUyj .EpPYLd:first-of-type{border-top-left-radius:8px;border-top-right-radius:8px}.yKUyj .EpPYLd:last-of-type{border-bottom-left-radius:8px;border-bottom-right-radius:8px}.yTik0{border:none;display:block;outline:none}.wplJBd{white-space:nowrap}.iQXTJe{padding:5px 0}.JM22S::-webkit-scrollbar{width:8px}.JM22S::-webkit-scrollbar-thumb{background-color:#bababa;border-right:4px solid #fff}.Zt0a5e.LGiluc{border-top-color:var(--gS5jXb)}.Zt0a5e.LGiluc,.Zt0a5e.EpPYLd[disabled]{color:rgba(0,0,0,.26)!important}.CjiZvb,.GZnQqe.EpPYLd:active{background-color:rgba(0,0,0,.1)}.EpPYLd{display:block;position:relative}.YpcDnf{padding:0 16px;vertical-align:middle}.WtV5nd .YpcDnf{padding-left:28px}.LGiluc,.EpPYLd[disabled]{pointer-events:none;cursor:default}@media (forced-colors:active){.EpPYLd[disabled]{color:GrayText}}.LGiluc{border-top:1px solid;height:0;margin:5px 0}.Zt0a5e .YpcDnf{line-height:48px}.Zt0a5e.CB8nDe{background:no-repeat left 8px center}.Zt0a5e.CB8nDe{background-image:url(https://ssl.gstatic.com/images/icons/material/system/1x/done_black_16dp.png)}@media (forced-colors:active){.Zt0a5e.CB8nDe{background-image:url(https://ssl.gstatic.com/images/icons/material/system/1x/done_white_16dp.png)}}.GZnQqe .YpcDnf{line-height:23px}.GZnQqe.CB8nDe{background:no-repeat left center}.GZnQqe.CB8nDe{background-image:url(https://ssl.gstatic.com/ui/v1/menu/checkmark2.png)}@media (forced-colors:active){.GZnQqe.CB8nDe{background-image:url(https://ssl.gstatic.com/ui/v1/menu/checkmark2-light.png)}}.GZnQqe.LGiluc,.GZnQqe.EpPYLd[disabled]{opacity:61%}.GZnQqe.LGiluc{border-top-color:var(--gS5jXb)}.YpcDnf.HG1dvd{padding:0}.HG1dvd>*{padding:0 16px}.eCcUA,.cDnxO{pointer-events:none;position:absolute;top:0;left:0;width:100%;height:100%;border-radius:50%}.rLdWfe{position:absolute;border-radius:inherit;top:0;left:0;width:100%;height:100%}.MDfoTd,.hdqIFe{pointer-events:none;position:absolute;top:0;left:0;width:100%;height:100%}.MDfoTd,.cDnxO{opacity:0}.hdqIFe,.cDnxO{overflow:hidden}.bKvogb .MDfoTd,.bKvogb .hdqIFe{border-radius:50%}.bKvogb .eCcUA{overflow:hidden}.RiJqbb{font-family:Google Sans,Roboto-Medium,sans-serif;font-weight:500}.b7GjXe{flex:0 auto;display:flex;align-items:center;width:fit-content;padding:2px 8px;overflow:hidden;font-size:12px;line-height:16px;border-radius:9999px;background:var(--gVUKcd);color:var(--EoKObe);letter-spacing:0.1px;pointer-events:none}@media (forced-colors:active){.b7GjXe{background:Canvas;outline:1px solid;outline-offset:-1px}}.b7GjXe span{overflow:hidden;white-space:nowrap;text-overflow:ellipsis}.b7GjXe.vtxgNc{padding:4px;flex-shrink:0}.b7GjXe div{flex:0 0 auto;display:flex;justify-content:center;align-items:center;width:12px;height:12px}.b7GjXe:not(.vtxgNc) div{-webkit-margin-end:4px;margin-inline-end:4px}.b7GjXe.CuEKaf{background:#d93025;color:#fff}.TUOsUe{text-align:left}.KUf18.ivkdbf{background-color:var(--gVUKcd);opacity:1;visibility:inherit}.VfsLpf.ivkdbf{background-color:#000;opacity:.4;visibility:inherit}.J3Hnlf.ivkdbf{background-color:#1f1f1f;opacity:.7;visibility:inherit}.X46m8.ivkdbf{background-color:#000;opacity:.8;visibility:inherit}.cBoDed.ivkdbf{background-color:#f8f9fa;opacity:.85;visibility:inherit}.kyk7qb.ivkdbf{background-color:#1f1f1f;opacity:.6;visibility:inherit}.qk7LXc.ivkdbf{opacity:1}.mcPPZ.ivkdbf{opacity:1;visibility:inherit}.mcPPZ.nP0TDe{cursor:pointer}.mcPPZ.nP0TDe .qk7LXc{cursor:default}.kJFf0c{position:fixed;z-index:9997;right:0;bottom:-200px;top:0;left:0;transition:opacity .25s;opacity:0;visibility:hidden}.qk7LXc{display:inline-block;z-index:9997;background-color:var(--xhUGwc);opacity:0;white-space:normal;overflow:hidden}.qk7LXc{border-radius:8px}.qk7LXc{box-shadow:0px 5px 26px 0px rgba(0,0,0,0.22),0px 20px 28px 0px rgba(0,0,0,0.3)}.qk7LXc.TSURG{border-bottom-left-radius:0;border-bottom-right-radius:0}.qk7LXc.DJEOfc{background-color:transparent}.qk7LXc.DJEOfc{box-shadow:none}.qk7LXc.Fb1AKc{position:relative;vertical-align:middle}.qk7LXc.By9mMc{position:fixed}.qk7LXc.W6Z5of{top:50%;left:50%;transform:translate(-50%,-50%)}.qk7LXc.ulWzbd{position:absolute}.qk7LXc.P1WYLb{border:1px solid var(--mXZkqc);box-shadow:#dadce0}.mcPPZ{position:fixed;right:0;bottom:0;top:0;left:0;z-index:9997;vertical-align:middle;visibility:hidden;white-space:nowrap;max-height:100%;max-width:100%;overflow-x:hidden;overflow-y:auto}.mcPPZ.xg7rAe{text-align:center}.mcPPZ::after{content:"";display:inline-block;height:100%;vertical-align:middle}.LjfRsf{height:0;opacity:0;position:absolute;width:0}.VH47ed{visibility:hidden}.TaoyYc{overflow:hidden}.TaoyYc{position:fixed;width:100%}.TaoyYc{position:fixed;width:100%}.yMNJR .qk7LXc,.qk7LXc.yMNJR{max-width:100%}.cJFqsd .qk7LXc,.qk7LXc.cJFqsd{height:100%}.rfx2Y .qk7LXc,.qk7LXc.rfx2Y{width:100%}@media (min-height:576px){.uSolm .qk7LXc{height:100%}.mcPPZ.uSolm{padding:64px 0px}.qk7LXc.uSolm{height:calc(100% - 128px)}}@media (max-height:575px){.uSolm .qk7LXc,.qk7LXc.uSolm{height:100%;max-height:448px}}.BhUHze .qk7LXc,.qk7LXc.BhUHze{width:75%}@media (min-height:496px){.GeOznc .qk7LXc{height:100%}.mcPPZ.GeOznc{padding:24px 0px}.qk7LXc.GeOznc{height:calc(100% - 48px)}}@media (max-height:495px){.GeOznc .qk7LXc,.qk7LXc.GeOznc{height:100%;max-height:448px}}.dgVGnc .qk7LXc,.qk7LXc.dgVGnc{width:90%}.vAJJzd{position:absolute;z-index:9997;opacity:.999}.qk7LXc.aJPx6e{overflow:visible}.hObAcc{margin-left:4px;margin-right:4px}.r2fjmd{margin-bottom:0px;margin-top:0px}.gTewb{padding-left:8px;padding-right:8px}::view-transition-old(sb){height:100%}::view-transition-new(sb){height:100%}@keyframes sb-slide-down-fade-in{0%{transform:scale(.5) translateY(-50px);opacity:0}20%{opacity:0}100%{transform:scale(1) translateY(0);opacity:1}}@keyframes sb-slide-up-fade-out{0%{transform:scale(1) translateY(0);opacity:1}20%{opacity:0}100%{transform:scale(.5) translateY(-50px);opacity:0}}@keyframes ubi-fade-in{0%{transform:translateY(-20px);opacity:0}20%{opacity:0}100%{transform:translateY(0);opacity:1}}@keyframes ubi-fade-out{0%{transform:translateY(0);opacity:1}20%{opacity:0}100%{transform:translateY(-20px);opacity:0}}html:active-view-transition-type(sb)::view-transition-old(sb),html:active-view-transition-type(sb)::view-transition-new(sb){height:100%}html:active-view-transition-type(sb)::view-transition-old(sb):only-child{animation:sb-slide-up-fade-out .5s cubic-bezier(.38,.72,0,1)}html:active-view-transition-type(sb)::view-transition-new(sb):only-child{animation:sb-slide-down-fade-in .5s cubic-bezier(.38,.72,0,1)}html:active-view-transition-type(mb)::view-transition-old(mb),html:active-view-transition-type(mb)::view-transition-new(mb){background-color:var(--EpFNW);width:auto;animation-duration:.5s;animation-timing-function:cubic-bezier(.38,.72,0,1)}html:active-view-transition-type(mb)::view-transition-group(*){transform-origin:top left}html:active-view-transition-type(mb)::view-transition-old(ubi),html:active-view-transition-type(mb)::view-transition-new(ubi){width:100%;height:100%;animation-duration:.5s;animation-timing-function:cubic-bezier(.38,.72,0,1)}html:active-view-transition-type(mb)::view-transition-old(ubi):only-child{animation:ubi-fade-out .5s cubic-bezier(.38,.72,0,1)}html:active-view-transition-type(mb)::view-transition-new(ubi):only-child{animation:ubi-fade-in .5s cubic-bezier(.38,.72,0,1)}.v0rrvd{padding-bottom:16px}.GUHazd{padding-bottom:12px}.bvSTKc{padding:8px}.cB4NFc{padding-top:16px}.OdBhM{padding-top:8px}.wok5vf{padding:24px}.Wm2B8c{color:var(--YLNNHc);font-size:20px;font-weight:600;line-height:24px}.Wm2B8c.pe7eXb{font-family:Google Sans,Roboto,sans-serif;font-size:28px;font-weight:400}.Wm2B8c.UrmYj{padding-bottom:0}.xjR8dd.xjR8dd{cursor:pointer;height:24px;position:absolute;right:16px;top:8px;width:24px}.ji9Nfb{border-radius:2px;box-shadow:0px 5px 26px 0px rgba(0,0,0,0.22),0px 20px 28px 0px rgba(0,0,0,0.3);background-color:var(--xhUGwc);cursor:default;width:220px;z-index:1002}.ji9Nfb.mdTXvc{border-radius:25px;max-width:80%;width:600px}.Ixjegd{max-height:400px;overflow-y:scroll;padding-right:10px}.gLeVcb{font-family:Google Sans,Roboto,sans-serif;font-size:24px;font-weight:500;line-height:32px}.xlan3d{font-family:Google Sans,Roboto,sans-serif;font-size:18px;font-weight:400;white-space:pre-wrap}.Xcntib{border:2px solid var(--mXZkqc);border-radius:20px;margin-bottom:14px;margin-top:10px;padding:10px}@media (forced-colors:active){.ji9Nfb,.xjR8dd{border:1px solid var(--xhUGwc)}}.GmoL0c{margin-right:12px}.xTWltf{margin-right:24px}.Wt5Tfe{padding-left:0px;padding-right:0px}.eJtrMc{padding-bottom:8px;padding-top:8px}.fudVZb{display:inline-block;vertical-align:bottom}.ALMXvf{display:flex}.ALMXvf.yjaPCc{flex-direction:column}.ALMXvf.heW0Cf{flex-direction:row;gap:10px;margin-bottom:10px}.H3ZkHf.H3ZkHf{line-height:24px}.H3ZkHf{color:var(--YLNNHc);cursor:pointer;font-weight:normal}.QSaZCc{fill:var(--TMYS9);padding-right:8px}.LlXeJ{height:24px;width:24px;vertical-align:middle}.btku5b{display:inline-block;vertical-align:middle;cursor:pointer}.btku5b,.brKmxb:focus-visible{outline:0;-webkit-tap-highlight-color:transparent}.niO4u::before,.niO4u::after{position:absolute;top:0;left:0;width:100%;height:100%;border-radius:inherit}.btku5b:hover .niO4u::after,.btku5b:active .niO4u::after{content:""}.niO4u::after{z-index:-1}.btku5b:focus-visible .niO4u::after,.brKmxb:focus-visible .niO4u::after{content:"";outline:2px solid var(--Pa8Wlb);outline-offset:2px}.niO4u{display:flex;justify-content:center;position:relative;align-items:center;width:100%;z-index:0}.niO4u::before{content:"";height:48px;margin-top:-24px;top:50%}.d3o3Ad.d3o3Ad,.clOx1e{display:flex;align-items:center}.niO4u{border-radius:9999px}@media (forced-colors:active){.niO4u{outline:1px solid ButtonBorder;outline-offset:-1px}}.LwdV0e .clOx1e,.LwdV0e .d3o3Ad{margin:8px 16px}.btku5b.btku5b[disabled]{color:var(--pEa0Bc);opacity:.38}.btku5b.btku5b.btku5b.btku5b.btku5b.btku5b[disabled] .niO4u{outline-color:var(--kloG3);cursor:auto}.btku5b.btku5b.btku5b.btku5b.btku5b.btku5b[disabled] .niO4u::after{background:transparent}.qVhvac[selected]{color:var(--xPpiM)}.qVhvac[selected]:hover .niO4u::after{background:rgba(11,87,208,.0784313725)}.qVhvac[selected] .niO4u{background:var(--KIZPne);outline:none}.qVhvac[selected]:active .niO4u::after{background:rgba(11,87,208,.2392156863)}.VDgVie{text-align:center}.SlP8xc{text-transform:none}.ogBxF{color:var(--JKqx2)}.ogBxF .niO4u{outline-offset:-1px;background:transparent;outline:1px solid var(--mXZkqc)}.ogBxF:active .niO4u::after{background:#0b57d039}.brKmxb:visited .ogBxF{color:#681da8}.gUAcff .clOx1e,.gUAcff .d3o3Ad{margin:8px 16px}.gUAcff .d3o3Ad{height:16px}.k1U36b{font-size:12px;font-family:Roboto,sans-serif;font-weight:500;line-height:16px}.sjVJQd{font-family:Google Sans,Roboto-Medium,sans-serif;font-size:14px;font-weight:500;line-height:20px}.pdJ1Ff{height:20px}.mh3zcf.mh3zcf.mh3zcf,.btku5b[selected] .KAe3ie{margin-left:0}.pdJ1Ff.pdJ1Ff.pdJ1Ff,.R04TOd.R04TOd.R04TOd{margin-left:12px;margin-right:4px}.QuU3Wb{margin-top:6px}.LwdV0e.OJeuxf .niO4u{width:36px;height:36px}.gUAcff.OJeuxf .niO4u{width:32px;height:32px}.NQYJvc.OJeuxf .niO4u{width:44px;height:44px}.OJeuxf .niO4u{margin:0 auto;height:100%}.gJdC8e.gJdC8e.gJdC8e{margin:0}.OJeuxf .niO4u::before{width:48px;margin-left:-24px;left:50%}.pAn7ne{white-space:nowrap;overflow:hidden;text-overflow:ellipsis}.NQYJvc .clOx1e,.NQYJvc .d3o3Ad{margin:12px 16px}.NQYJvc .d3o3Ad{height:20px}.FR7ZSc{color:var(--rrJJUc)}.FR7ZSc .niO4u{background:transparent;outline:1px solid var(--Yi4Nb);outline-offset:-1px}.FR7ZSc:active .niO4u::after{background:rgba(11,87,208,.2392156863)}.expUPd{color:var(--todMNcl)}.expUPd .niO4u{background:var(--izGsqb)}.expUPd[disabled] .niO4u{background:var(--YaIeMb)}.expUPd:active .niO4u::after{background:rgba(11,87,208,.2392156863)}.zqrO0{color:var(--bbQxAb)}.zqrO0 .niO4u{background:var(--xhUGwc);outline:1px solid var(--mXZkqc);outline-offset:-1px}.zqrO0:active .niO4u::after{background:rgba(0,29,53,.2392156863)}.xab99e .niO4u{z-index:2;background:var(--xhUGwc);outline:1px solid var(--mXZkqc)}.xab99e.xab99e.OJeuxf .niO4u{width:40px;height:40px}.NtaMpb{color:var(--bbQxAb)}.NtaMpb .niO4u{background:var(--xhUGwc)}.brKmxb:focus-visible .NtaMpb .niO4u::after,.NtaMpb:focus-visible .niO4u::after{background:#0b57d015}.NtaMpb:active .niO4u::after{background:#0b57d039}.Bmegof{color:#001d35}.Bmegof .niO4u{background:var(--BRLwE)}.Bmegof:active .niO4u::after{background:rgba(31,31,31,.2392156863)}.r69FZ{color:#001d35}.r69FZ .niO4u{background:#ebf1ff}.r69FZ:active .niO4u::after{background:rgba(8,75,177,.2392156863)}.rXoM2c{color:var(--amnPwe)}.rXoM2c .niO4u{background:#f1f3f4}.jdOrZc{color:var(--rrJJUc)}.jdOrZc .niO4u{background:transparent}.fXVarf{color:var(--EoKObe)}.fXVarf .niO4u{background:rgba(0,0,0,.6);backdrop-filter:blur(8px)}.fXVarf:active .niO4u::after{background:#1f1f1f66}@media (forced-colors:active){.btku5b.btku5b.btku5b[selected],.btku5b.btku5b.btku5b[selected] .d3o3Ad{color:SelectedItemText}.btku5b[selected] .clOx1e{forced-color-adjust:preserve-parent-color}.btku5b.btku5b[selected] .niO4u{background:SelectedItem}}.d2D5h[selected] .niO4u{background:var(--xhUGwc)}.brKmxb:focus-visible .d2D5h[selected],.d2D5h[selected]:focus-visible{color:#1967d2}.btku5b:not([selected]) .R04TOd,.btku5b[selected] .BGVjpf{display:none}.clOx1e.clOx1e.aqBay{margin-right:4px}.WoA9Zd.WoA9Zd.WoA9Zd{margin-right:12px;margin-left:4px}.btku5b:not([aria-expanded=true]) .RwPSab,.btku5b[aria-haspopup=true][aria-expanded=true] .rDI98b{display:none}.fqiILb[selected]:not(:focus-visible) .niO4u::after,.brKmxb:not(:focus-visible) .fqiILb[selected] .niO4u::after{content:"";outline:1px solid var(--rrJJUc);outline-offset:0}.gja3Ve{color:var(--amnPwe)}.gja3Ve .QuU3Wb{color:var(--YLNNHc)}.gja3Ve .niO4u{background:#f1f3f4}.gja3Ve:active{color:var(--YLNNHc)}.gja3Ve:active .niO4u::after{background:rgba(31,31,31,0.23921569)}.x2GJWb,.x2GJWb .QuU3Wb{color:var(--YLNNHc)}.x2GJWb .niO4u{background:#f1f3f4}.x2GJWb:active .niO4u::after{background:rgba(32,33,36,0.24)}.brKmxb:focus-visible .x2GJWb .niO4u::after,.x2GJWb:focus-visible .niO4u::after{background:#0b57d015}.x2GJWb:active .niO4u::after{background:#0b57d039}.zJUuqf{margin-bottom:4px}.AB4Wff{margin-left:16px}.wHYlTd{font-family:Roboto,sans-serif;font-size:14px;line-height:20px}.yUTMj{font-family:Roboto,sans-serif;font-weight:400}@keyframes g-snackbar-show{from{pointer-events:none;transform:translateY(0)}to{transform:translateY(-100%)}}@keyframes g-snackbar-hide{from{transform:translateY(-100%)}to{transform:translateY(0)}}@keyframes g-snackbar-show-content{from{opacity:0}}@keyframes g-snackbar-hide-content{to{opacity:0}}.LH3wG{bottom:0;height:0;position:fixed;z-index:999}.yK6jqe{box-sizing:border-box;visibility:hidden}.rTYTNb{animation:g-snackbar-hide .4s cubic-bezier(0.4,0,0.2,1) both;visibility:inherit}.UewPMd{animation:g-snackbar-show .5s cubic-bezier(0.4,0,0.2,1) both;visibility:inherit}.b77HKf{background-color:#323232;padding:0 24px}.rIxsve{align-items:center;display:flex}.rTYTNb .rIxsve{animation:g-snackbar-hide-content .35s cubic-bezier(0.4,0,0.2,1) both}.UewPMd .rIxsve{animation:g-snackbar-show-content .35s cubic-bezier(0.4,0,0.2,1) .15s both}.Txngnb.Txngnb{line-height:20px}.Txngnb{color:#fff;flex:1 1 auto;margin:14px 0;word-break:break-word}@media (min-width:569px) and (min-height:569px){.LH3wG{text-align:center}.yK6jqe{display:inline-block;max-width:568px;min-width:288px;text-align:left}.b77HKf{border-radius:8px}}.LH3wG{left:0;right:0}.yK6jqe{position:relative}.Ox8Cyd{height:0;position:fixed;z-index:999;left:0;right:0}.E7Hdgb{box-sizing:border-box;visibility:hidden;display:inline-block;position:relative}.jhZvod{bottom:0;height:0;position:fixed;z-index:999;left:16px;right:auto}.Wu0v9b{box-sizing:border-box;visibility:hidden;position:relative}@media (min-width:569px) and (min-height:569px){.jhZvod{text-align:center}.Wu0v9b{display:inline-block;max-width:568px;min-width:288px;text-align:left}}.sHFNYd{margin-right:-8px}@media (min-width:569px) and (min-height:569px){.sHFNYd{margin-left:40px}}.G9jore{position:absolute;top:-24px;bottom:-24px;left:-24px;right:-24px}.V9O1Yd .rIxsve{display:block;padding:8px 0}.V9O1Yd .sHFNYd{margin-left:0}.V9O1Yd .sHFNYd g-flat-button{padding-left:0}.OhScic{margin:0px}.zUdppc{padding-bottom:4px}.n5Hqzc{cursor:pointer}.K4PHmc,.U3T04b input.U3T04b share-url-input{color:var(--YLNNHc);font-size:16px;line-height:18px;width:100%;cursor:pointer}.zs5EIc{border-bottom:1px solid #ddd;white-space:nowrap}.hEcKn,.U3T04b a.U3T04b share-url-anchor,.U3T04b a.U3T04b share-url-anchor:visited{color:var(--YLNNHc);font-size:16px;line-height:26px}.KZMqi{opacity:0;position:absolute}.KVCvbb .D5kRdd{top:4px}.mMB4wd .D5kRdd{top:0}.zu4nZc.KVCvbb .D5kRdd{top:10px}.tHaYSc .D5kRdd{top:8px}.zu4nZc.tHaYSc .D5kRdd{top:14px}.TkZZsf{padding-bottom:4px;padding-top:4px}.CdKv9{display:block;position:relative;margin:0 3px;vertical-align:middle}.baeIxf{border:none;box-shadow:none;color:var(--YLNNHc);font-size:16px;width:100%}.oWf3dc .baeIxf,.oWf3dc .D5kRdd{font-size:14px}.baeIxf:focus{outline:none}.oPHL4:not(.KIChyf) .WO1lOd,.CaMEAc:not(.KIChyf) .WO1lOd{border-color:#d93025}.jFKGGf.KIChyf .WO1lOd{border-bottom-style:dashed}.WO1lOd{display:table;width:100%}.icEKy .Wt5Tfe{padding-left:0px;padding-right:0px}.icEKy .TkZZsf{padding-bottom:4px;padding-top:4px}.icEKy .eJtrMc{padding-bottom:8px;padding-top:8px}.icEKy .OhScic{margin:0px}.s9cjxc{color:#b3261e;display:none;margin-top:4px}.oPHL4:not(.KIChyf) .s9cjxc,.CaMEAc:not(.KIChyf) .s9cjxc{display:block}.MdFDgb.HUQNIe{color:rgba(0,0,0,.26)}.CdKv9.oJaOuf{margin-top:14px}.CdKv9.b7fuzd{margin-top:0}.D5kRdd{left:0;right:0;font-size:16px;pointer-events:none;position:absolute;overflow:hidden;white-space:nowrap;transition-duration:.2s;transition-timing-function:cubic-bezier(.4,0,.2,1)}.MdFDgb.d53QNd{color:var(--IXoxUe)}.ZCGr5c .MdFDgb,.ZCGr5c .MdFDgb.HUQNIe,.ZCGr5c .MdFDgb.d53QNd{color:var(--Pa8Wlb)}.CdKv9.ZCGr5c .D5kRdd,.CdKv9.TJgfJ .D5kRdd{display:none}.CdKv9.ZCGr5c .D5kRdd.zLEPgb,.CdKv9.TJgfJ .D5kRdd.zLEPgb{font-size:12px;top:-16px;display:flex}.czUY2e{background-color:var(--Pa8Wlb)}.czUY2e{bottom:0;height:2px;left:45%;margin-top:-2px;pointer-events:none;position:relative;transition-duration:.2s;transition-timing-function:cubic-bezier(.4,0,.2,1);visibility:hidden;width:10px}.ZCGr5c .czUY2e{left:0;visibility:inherit;width:100%}.oPHL4 .czUY2e,.CaMEAc .czUY2e{background-color:#d93025}.jFKGGf .WO1lOd{border-bottom:1px solid var(--mXZkqc);border-radius:0}.zu4nZc .D5kRdd{left:8px;right:8px}.CdKv9.zu4nZc.ZCGr5c .zLEPgb,.CdKv9.zu4nZc.TJgfJ .zLEPgb{top:-8px}.zu4nZc.oPHL4:not(.ZCGr5c) .MdFDgb,.zu4nZc.CaMEAc:not(.ZCGr5c) .MdFDgb{color:#b3261e}.zu4nZc .WO1lOd{border-radius:6px;border:1px solid var(--mXZkqc);padding:5px 8px;box-sizing:border-box}.zu4nZc.ZCGr5c .WO1lOd{padding:4px 7px;border-width:2px}.zu4nZc.ZCGr5c:not(.kXKNac) .WO1lOd{border-color:var(--Pa8Wlb)}.hUhkLe,.UgQR0c{color:var(--IXoxUe);font-size:16px;white-space:nowrap;display:table-cell}.hUhkLe{padding-right:5px}.UgQR0c{padding-left:5px}.ek8SXd{vertical-align:middle}.WO1lOd .FFTibe{display:table-cell;width:100%}.oWf3dc .hUhkLe,.oWf3dc .UgQR0c{font-size:14px}.hUhkLe.AT2KAb,.UgQR0c.AT2KAb{transition-duration:.2s;transition-timing-function:cubic-bezier(.4,0,.2,1);display:inline-block;max-width:100%;visibility:hidden}.tv3VAe{display:flex}.MdFDgb{width:100%;white-space:nowrap}.CdKv9.ZCGr5c .AT2KAb,.CdKv9.TJgfJ .AT2KAb{max-width:0;padding:0}.kXKNac .WO1lOd{border-color:rgba(255,255,255,0.54)}.kXKNac.CaMEAc:not(.KIChyf) .WO1lOd,.kXKNac.oPHL4 .WO1lOd:not(.KIChyf){border-color:#fdd663}.kXKNac.ZCGr5c .MdFDgb,.kXKNac.ZCGr5c .MdFDgb.HUQNIe,.kXKNac.ZCGr5c .MdFDgb.d53QNd{color:var(--xhUGwc)}.kXKNac .MdFDgb.HUQNIe,.kXKNac .MdFDgb.d53QNd{color:rgba(255,255,255,0.54)}.kXKNac.zu4nZc.oPHL4:not(.ZCGr5c) .zLEPgb,.kXKNac.zu4nZc.CaMEAc:not(.ZCGr5c) .zLEPgb{color:#fdd663}.kXKNac .czUY2e{background:var(--xhUGwc)}.kXKNac.oPHL4 .czUY2e,.kXKNac.CaMEAc .czUY2e{background-color:#ffdf92}.kXKNac .s9cjxc{color:#fdd663}.kXKNac .hUhkLe,.kXKNac .UgQR0c{color:rgba(255,255,255,0.54)}.zsYMMe{padding:0px}.jbBItf{display:block;position:relative}.DU0NJ{bottom:0;left:0;position:absolute;right:0;top:0}.lP3Jof{display:inline-block;position:relative}.nNMuOd{animation:qli-container-rotate 1568.2352941176ms linear infinite}@keyframes qli-container-rotate{from{transform:rotate(0)}to{transform:rotate(1turn)}}.RoKmhb{height:100%;opacity:0;position:absolute;width:100%}.nNMuOd .VQdeab{animation:qli-fill-unfill-rotate 5332ms cubic-bezier(0.4,0,0.2,1) infinite both,qli-blue-fade-in-out 5332ms cubic-bezier(0.4,0,0.2,1) infinite both}.nNMuOd .IEqiAf{animation:qli-fill-unfill-rotate 5332ms cubic-bezier(0.4,0,0.2,1) infinite both,qli-red-fade-in-out 5332ms cubic-bezier(0.4,0,0.2,1) infinite both}.nNMuOd .smocse{animation:qli-fill-unfill-rotate 5332ms cubic-bezier(0.4,0,0.2,1) infinite both,qli-yellow-fade-in-out 5332ms cubic-bezier(0.4,0,0.2,1) infinite both}.nNMuOd .FlKbCe{animation:qli-fill-unfill-rotate 5332ms cubic-bezier(0.4,0,0.2,1) infinite both,qli-green-fade-in-out 5332ms cubic-bezier(0.4,0,0.2,1) infinite both}.BSnLb .nNMuOd .RoKmhb{animation:qli-fill-unfill-rotate 5332ms cubic-bezier(0.4,0,0.2,1) infinite both;opacity:0.99}@keyframes qli-fill-unfill-rotate{0%{transform:rotate(0)}12.5%{transform:rotate(135deg)}25%{transform:rotate(270deg)}37.5%{transform:rotate(405deg)}50%{transform:rotate(540deg)}62.5%{transform:rotate(675deg)}75%{transform:rotate(810deg)}87.5%{transform:rotate(945deg)}100%{transform:rotate(3turn)}}@keyframes qli-blue-fade-in-out{0%{opacity:0.99}25%{opacity:0.99}26%{opacity:0}89%{opacity:0}90%{opacity:0.99}100%{opacity:0.99}}@keyframes qli-red-fade-in-out{0%{opacity:0}15%{opacity:0}25%{opacity:0.99}50%{opacity:0.99}51%{opacity:0}}@keyframes qli-yellow-fade-in-out{0%{opacity:0}40%{opacity:0}50%{opacity:0.99}75%{opacity:0.99}76%{opacity:0}}@keyframes qli-green-fade-in-out{0%{opacity:0}65%{opacity:0}75%{opacity:0.99}90%{opacity:0.99}100%{opacity:0}}.beDQP{display:inline-block;height:100%;overflow:hidden;position:relative;width:50%}.FcXfi{box-sizing:border-box;height:100%;left:45%;overflow:hidden;position:absolute;top:0;width:10%}.SPKFmc{border-radius:50%;border:3px solid transparent;box-sizing:border-box}@media (forced-colors:active),(prefers-contrast:more){.beDQP:last-child .SPKFmc{border:none}}.x3SdXd{width:200%}.J7uuUe{transform:rotate(129deg)}.sDPIC{left:-100%;transform:rotate(-129deg)}.tS3P5{left:-450%;width:1000%}.VQdeab .SPKFmc{border-color:#4285f4}.IEqiAf .SPKFmc{border-color:#ea4335}.smocse .SPKFmc{border-color:#fbbc04}.FlKbCe .SPKFmc{border-color:#34a853}.RoKmhb .J7uuUe{border-bottom-color:transparent;border-right-color:transparent}.RoKmhb .sDPIC{border-bottom-color:transparent;border-left-color:transparent}.RoKmhb .tS3P5{border-bottom-color:transparent}.GgTJWe .nNMuOd .J7uuUe{animation:qli-left-spin 1333ms cubic-bezier(0.4,0,0.2,1) infinite both}.GgTJWe .nNMuOd .sDPIC{animation:qli-right-spin 1333ms cubic-bezier(0.4,0,0.2,1) infinite both}.BSnLb .nNMuOd .J7uuUe{animation:qli-left-spin 1333ms cubic-bezier(0.4,0,0.2,1) infinite both;border-left-color:#fff;border-top-color:#fff}.BSnLb .nNMuOd .sDPIC{animation:qli-right-spin 1333ms cubic-bezier(0.4,0,0.2,1) infinite both;border-right-color:#fff;border-top-color:#fff}.BSnLb .nNMuOd .tS3P5{border-color:#fff;border-bottom-color:transparent}@keyframes qli-left-spin{0%{transform:rotate(130deg)}50%{transform:rotate(-5deg)}100%{transform:rotate(130deg)}}@keyframes qli-right-spin{0%{transform:rotate(-130deg)}50%{transform:rotate(5deg)}100%{transform:rotate(-130deg)}}.thsZXc{background:#1a73e8}.hiQRQc{background:#e8eaed}.YQ4gaf{display:block;border:0}.u9wH7d .YQ4gaf{object-fit:fill}.mNsIhb .YQ4gaf{object-fit:cover}.tb08Pd .YQ4gaf{object-fit:contain}.wA1Bge{position:relative}.ZGomKf{overflow:hidden}.LLO8yd{background-color:rgba(0,0,0,0.03);position:absolute;top:0;bottom:0;pointer-events:none;left:0;right:0}.LqkKtf{align-items:center;display:none;height:100%;justify-content:center;left:0;opacity:.999;overflow:hidden;position:absolute;top:0;width:100%}.Iz740d.LqkKtf{border-radius:50%}.LqkKtf.DngrPc{left:50%;margin-left:-50vw;margin-right:-50vw;right:50%;width:100vw}.amp_re{position:relative}.CQC9Oe{--skw-color-primary:var(--aYn2S);--skw-color-on-primary:var(--bbQxAb);--skw-color-primary-container:var(--mXZkqc);--skw-color-on-primary-container:#a9acaa;--skw-color-secondary:var(--JKqx2);--skw-color-secondary-container:var(--BRLwE);--skw-color-on-secondary-container:var(--jINu6c);--skw-color-error-container:#f9dedc;--skw-color-on-error-container:#410e0b;--skw-color-surface:var(--xhUGwc);--skw-color-on-surface:var(--YLNNHc);--skw-color-on-surface-variant:var(--IXoxUe);--skw-color-outline:var(--mXZkqc);--skw-color-shadow:transparent;--skw-color-scrim:var(--gVUKcd);--skw-color-surface-container-low:var(--xhUGwc);--skw-color-surface-container:var(--XKMDxc);--skw-color-surface-container-high:var(--aYn2S);--skw-color-surface-container-highest:var(--Lm570b);--skw-color-outline-variant:#a9acaa;--skw-font:Google Sans,Roboto,sans-serif}.wb5ZZc{display:none}g-dropdown-menu{display:inline-block;position:relative}.Jb0Zif g-dropdown-menu{vertical-align:middle}.WNN1b{background-color:var(--xhUGwc)}.adGN6[disabled]{pointer-events:none;cursor:default;background-color:var(--YaIeMb)}.W4XqN{cursor:pointer;background-color:var(--xhUGwc)}.GKXWV{border-top:var(--mXZkqc);height:0;margin-left:5px;margin-right:5px}.eNRwyf{height:100%;width:100%}.QyJI3d{background-color:var(--xhUGwc);color:#666;box-shadow:0 4px 16px rgba(0,0,0,0.2)}.oQcPt{background-color:var(--xhUGwc)}.QyJI3d{border:1px solid rgba(0,0,0,.2);position:absolute;z-index:9120}.nFdFHf{animation:g-bubble-show .2s forwards}.bE3Kif{animation:g-bubble-hide .2s forwards}@keyframes g-bubble-show{from{opacity:0}to{opacity:1}}@keyframes g-bubble-hide{from{opacity:1}to{opacity:0}}.QyJI3d.QJnoze{border-radius:12px}.QyJI3d.SiOjJb{border-left-width:0;border-right-width:0;width:100%}.QyJI3d.PnQMie{background-color:#1f1f1f;border:1px solid rgba(0,0,0,.5);color:#dadce0}.QyJI3d.LWen5c{background-color:#1a73e8;border:none;color:#fff;z-index:9100}.tYmfxe{transform:translate(2.5px,1.8px) rotateZ(45deg);position:absolute;z-index:9121}[dir=rtl] .tYmfxe{transform:translate(-2.5px,1.8px) rotateZ(45deg)}.IBPZu.tYmfxe{transform:translate(2.5px,-5.7px) rotateZ(45deg)}[dir=rtl] .IBPZu.tYmfxe{transform:translate(-2.5px,-5.7px) rotateZ(45deg)}.oQcPt{border-bottom:none;border-left:1px solid rgba(0,0,0,.2);border-right:none;border-top:1px solid rgba(0,0,0,.2);box-sizing:border-box;height:13.435px;width:13.435px}.IBPZu .oQcPt{border-bottom:1px solid rgba(0,0,0,.2);border-left:none;border-right:1px solid rgba(0,0,0,.2);border-top:none}.PnQMie .oQcPt{background-color:#1f1f1f;border-color:rgba(0,0,0,.5)}.LWen5c .oQcPt{background-color:#1a73e8;border:none}.nnFGuf{display:none}.c5aZPb{cursor:pointer}.Dcltre{pointer-events:none}.TYQ8Af{clip:rect(1px,1px,1px,1px);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px;z-index:-1000;top:3px;right:3px}.ZMZ3he{color:var(--COEmY)}.JgzqYd{font-family:Google Sans,Roboto,sans-serif;font-size:24px;line-height:32px}.Pqkn2e{font-family:Roboto,sans-serif;font-size:16px;line-height:24px}.ZwRhJd{font-family:Roboto,sans-serif;font-size:14px;line-height:18px}.MBeuO{font-family:Google Sans,Roboto,sans-serif;font-size:20px;font-weight:400}.MBeuO{line-height:26px}.RES9jf{color:var(--YLNNHc)}.ZYHQ7e{color:var(--IXoxUe)}.GS5rRd{color:var(--JKqx2)}.GS5rRd:visited{color:#681da8}.LwVyHd{transition:max-height .3s;overflow:hidden}.Yh3tf{pointer-events:none;position:absolute!important;right:0;color:var(--amnPwe);top:50%;margin-top:-12px;display:flex;align-items:center}.FXMOpb .Yh3tf{margin-top:-14px}.FXMOpb .MLgx0e{transform:rotateX(180deg)}.ydSoC{padding-left:8px}.kGfs{position:relative}.Yh3tf.chvYFc{background:var(--XKMDxc);border-radius:25px;margin-top:-18px;padding:6px}.V1sL5c{overflow:visible}.S8PBwe{max-height:0;display:none}.SpKsUc{margin:0 0px;padding:0 0px}.HbX59e{margin-left:0px}.iRPzcb{border-bottom:1px solid var(--gS5jXb)}.yunLzb{--preferred-text-scale:env(preferred-text-scale,-1)}.Bb1JKe{padding-bottom:8px}.ouy7Mc{padding-left:16px;padding-right:16px}.M8CEed{padding-top:12px}@-webkit-keyframes qs-timer {0%{}}
    </style>
</head>

<body jsmodel="hspDDf ">
    <style>
        .z1asCe{display:inline-block;fill:currentColor;height:24px;line-height:24px;position:relative;width:24px}.z1asCe svg{display:block;height:100%;width:100%}body,html{height:100%;margin:0;padding:0}body.srp,body{background:#fff}body{font-family:Roboto,sans-serif;font-size:10pt;}#belowsb{margin-top:24px;}.yKiQp{min-height:25px}#belowsb a{color:var(--JKqx2)}#swml,#belowsb{text-align:center}#swml{height:25px;margin:5px 0}#swml>div{margin:1px 0}#gb-main{display:-webkit-box;display:flex;display:-ms-flexbox;display:flex;-webkit-box-orient:vertical;flex-direction:column;flex-direction:column;min-height:100%}#n0tgWb{-webkit-box-flex:1;flex:1;flex:1}@media only screen and (orientation:portrait){#n0tgWb{padding-top:20px}}#mlogo{display:none}.gs_ml #lst-ib{white-space:normal;height:auto}header{display:-webkit-box;display:flex;display:-ms-flexbox;display:flex;min-height:48px;width:100%}#SBmmZd{display:-webkit-box;display:flex;display:-ms-flexbox;display:flex;flex-wrap:wrap;padding-left:16px}#SBmmZd a{color:var(--IXoxUe);display:block;-webkit-box-flex:0;flex:0 0 auto;flex:0 0 auto;font-size:12px;font-weight:600;line-height:calc(48px - 0*12px);margin-right:8px;padding:0 8px;text-align:center;text-transform:uppercase;text-decoration:none}#SBmmZd a.active{color:var(--rrJJUc);border-bottom:2px solid var(--rrJJUc)}#vLkmZd{-webkit-box-flex:0;flex:0 0 auto;flex:0 0 auto;position:relative;z-index:1;padding-top:calc(0*12px)}.igu8Pb{-webkit-box-flex:1;flex:1 0 auto;flex:1 0 auto}
    </style>
    <script nonce="IJ8UXppd3ETKm0PGR2OelA">
        ;
        this.gbar_ = {
            CONFIG: [
                [
                    [0, "www.gstatic.com", "og.qtm.en_US.InInzjbyX2s.2019.O", "com", "en", "1", 0, [4, 2, "", "", "", "809810803", "0"], null, "v8HjaN2mB_3l5OMPormAkAo", null, 0, "og.qtm.pfog22aEnAs.L.W.O", "AA2YrTthnASDYd0N4gbIL91FDhrpfXlj5w", "AA2YrTskOaSug7MVZwlus97OpUaPcMM3bw", "", 2, 1, 200, "USA", null, null, "1", "1", 1, null, null, 89978449, null, 3000, 0], null, [1, 0.1000000014901161, 2, 1], null, [0, 0, 0, null, "", "", "", "", 0, 0, null, ""],
                    [0, 0, "", 1, 0, 0, 0, 0, 0, 0, null, 0, 0, null, 0, 0, null, null, 0, 0, 0, "", "", "", "", "", "", null, 0, 0, 0, 0, 0, null, null, null, "rgba(32,33,36,1)", "rgba(255,255,255,1)", 0, 0, 1, null, null, null, 0], null, null, ["1", "gci_91f30755d6a6b787dcc2a4062e6e9824.js", "googleapis.client:gapi.iframes", "", "en"], null, null, null, null, ["m;/_/scs/abc-static/_/js/k=gapi.gapi.en.wGQ1JnvbZsA.O/d=1/rs=AHpOoo_CQByDq4M_GLyNYLaxwBcABC_4uA/m=__features__", "https://apis.google.com", "", "", "", "", null, 1, "es_plusone_gc_20250903.0_p1", "en", null, 0],
                    [0.009999999776482582, "com", "1", [null, "n", "", null, 1, 2592000, null, null, "https://www.google.com/url?q=https://accounts.google.com/signin/v2/identifier%3Fec%3Dfutura_hpp_co_si_001_p%26continue%3Dhttps%253A%252F%252Fwww.google.com%252F%253Fptid%253D19027681%2526ptt%253D8%2526fpts%253D0\u0026source=hpp\u0026id=19046229\u0026ct=7\u0026usg=AOvVaw33MBGJMT3TA0n4WMEDSPEO", null, null, null, null, null, 0, null, 0, null, 1, 0, 1, 0, null, null, 0, 0, null, 0, 0, 0, 0, 0], null, null, null, 0],
                    [1, null, null, 31151, 1, "USA", "en", "809810803.0", 8, null, 0, 0, null, null, null, null, "3700949,3701384,102772546", null, null, null, "v8HjaN2mB_3l5OMPormAkAo", 0, 0, 0, null, 2, 5, "gh", 96, 0, 0, null, null, 1, 89978449, 1, 0],
                    [
                        [null, null, null, "https://www.gstatic.com/og/_/js/k=og.qtm.en_US.InInzjbyX2s.2019.O/rt=j/m=qabr,q_d,qcwid,qapid,qads,q_dg/exm=qaaw,qadd,qaid,qein,qhaw,qhba,qhbr,qhch,qhga,qhid,qhin/d=1/ed=1/rs=AA2YrTthnASDYd0N4gbIL91FDhrpfXlj5w"],
                        [null, null, null, "https://www.gstatic.com/og/_/ss/k=og.qtm.pfog22aEnAs.L.W.O/m=qcwid,d_b_gm3,d_wi_gm3,d_lo_gm3/excm=qaaw,qadd,qaid,qein,qhaw,qhba,qhbr,qhch,qhga,qhid,qhin/d=1/ed=1/ct=zgms/rs=AA2YrTskOaSug7MVZwlus97OpUaPcMM3bw"]
                    ], null, null, null, [
                        [
                            [null, null, [null, null, null, "https://ogs.google.com/widget/app/so?eom=1\u0026awwd=1\u0026dpi=89978449"], 0, 470, 370, 50, 4, 1, 0, 0, 63, 64, 8000, "https://www.google.com/intl/en/about/products?tab=wh", 67, 1, 69, null, 1, 70, "Can't seem to load the app launcher right now. Try again or go to the %1$sGoogle Products%2$s page.", 3, 0, 0, 74, 0, null, null, null, null, null, null, null, "/widget/app/so", null, null, null, null, null, null, null, 0, null, null, null, null, null, null, null, null, null, null, 1, null, 144, null, null, 3, 0, 0, 0, 0, "(opens a new tab)"],
                            [null, null, [null, null, null, "https://ogs.google.com/widget/account/so?eom=1\u0026baea=1\u0026amb=1"], 0, 414, 436, 50, 4, 1, 0, 0, 65, 66, 8000, "https://accounts.google.com/SignOutOptions?hl=en\u0026continue=https://www.google.com/\u0026ec=GBRAAQ", 68, 2, null, null, 1, 113, "Something went wrong.%1$s Refresh to try again or %2$schoose another account%3$s.", 3, null, null, 75, 0, null, null, null, null, null, null, null, "/widget/account/so", null, 0, 0, 1, ["Critical security alert", "Important account alert", "Storage usage alert", null, 0, 0, 0], 0, 1, null, 1, 1, null, null, null, null, 0, 0, 0, null, 1, 1, null, null, null, null, null, null, null, null, null, 0],
                            [null, null, [null, null, null, "https://ogs.google.com/widget/callout?eom=1\u0026dc=1"], null, 280, 420, 70, 25, 0, null, 0, null, null, 8000, null, 71, 4, null, null, null, null, null, null, null, null, 76, null, null, null, 107, 108, 109, "", null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 1]
                        ], null, null, "1", "1", 1, 0, null, "en", 0, ["", "", "https://accounts.google.com/Logout?hl=en\u0026continue=https://www.google.com/\u0026ec=GAdAAQ", "", 1, 0, "", 0, 0, ["Google Search", [
                            [
                                [
                                    [
                                        ["og-am-ss", "SafeSearch", ["safesearch_logo", 0, 0], 1, "/safesearch?prev=https://www.google.com/", 1, "", "Blurring on", null, 0]
                                    ],
                                    [
                                        ["og-am-lg", "Language", ["language", 0, 0], 1, "/preferences?lang=1\u0026hl=en\u0026prev=https://www.google.com/#languages", 1, "", "English", null, 0]
                                    ],
                                    [
                                        ["og-am-dm", "Dark theme", ["dark_mode", 0, 0], 1, "/setprefs?hl=en\u0026prev=https://www.google.com/?pccc%3D1\u0026sig=0_7QCvYJ2iJw40tBfcNgG7LYMrLU0%3D\u0026cs=2", 1, "", "Device default", null, 0]
                                    ],
                                    [null, [
                                        ["og-am-ms", "More settings", ["settings", 0, 0], 1, "/preferences?hl=en\u0026prev=https://www.google.com/", 1, "", "", null, 0],
                                        ["og-am-hf", "Help", ["help_outline", 0, 0], 1, "https://support.google.com/websearch/?p=dsrp_search_hc\u0026hl=en", 1, "", "", null, 0]
                                    ]]
                                ]
                            ]
                        ], null, null, "og-te", ""], 0, 0, "https://accounts.google.com/ServiceLogin?hl=en\u0026passive=true\u0026continue=https://www.google.com/\u0026ec=GAZAAQ", null, null, 0, null, null, null, 0], 0, 0, 0, [null, "", null, null, null, 0, null, 0, 0, "", "", "", "https://ogads-pa.clients6.google.com", 0, 0, 0, "", "", 0, 0, null, 86400, null, 0, null, null, 0, null, 0, 0, "8559284470", 3, 0, 1], 0, null, null, null, 0, 0, "", 0
                    ]
                ]
            ],
        };
        this.gbar_ = this.gbar_ || {};
        (function(_) {
            var window = this;
            try {
                _._F_toggles_initialize = function(a) {
                    (typeof globalThis !== "undefined" ? globalThis : typeof self !== "undefined" ? self : this)._F_toggles_gbar_ = a || []
                };
                (0, _._F_toggles_initialize)([]);
                /*

                 Copyright The Closure Library Authors.
                 SPDX-License-Identifier: Apache-2.0
                */
                var ja, pa, qa, ua, wa, xa, Fa, Ga, $a, cb, eb, jb, fb, lb, rb, Db, Eb, Fb, Gb;
                _.aa = function(a, b) {
                    if (Error.captureStackTrace) Error.captureStackTrace(this, _.aa);
                    else {
                        const c = Error().stack;
                        c && (this.stack = c)
                    }
                    a && (this.message = String(a));
                    b !== void 0 && (this.cause = b)
                };
                _.ba = function(a) {
                    a.yk = !0;
                    return a
                };
                _.ia = function(a) {
                    var b = a;
                    if (da(b)) {
                        if (!/^\s*(?:-?[1-9]\d*|0)?\s*$/.test(b)) throw Error(String(b));
                    } else if (ea(b) && !Number.isSafeInteger(b)) throw Error(String(b));
                    return fa ? BigInt(a) : a = ha(a) ? a ? "1" : "0" : da(a) ? a.trim() || "0" : String(a)
                };
                ja = function(a, b) {
                    if (a.length > b.length) return !1;
                    if (a.length < b.length || a === b) return !0;
                    for (let c = 0; c < a.length; c++) {
                        const d = a[c],
                            e = b[c];
                        if (d > e) return !1;
                        if (d < e) return !0
                    }
                };
                _.ka = function(a) {
                    _.t.setTimeout(() => {
                        throw a;
                    }, 0)
                };
                _.ma = function() {
                    return _.la().toLowerCase().indexOf("webkit") != -1
                };
                _.la = function() {
                    var a = _.t.navigator;
                    return a && (a = a.userAgent) ? a : ""
                };
                pa = function(a) {
                    if (!na || !oa) return !1;
                    for (let b = 0; b < oa.brands.length; b++) {
                        const {
                            brand: c
                        } = oa.brands[b];
                        if (c && c.indexOf(a) != -1) return !0
                    }
                    return !1
                };
                _.u = function(a) {
                    return _.la().indexOf(a) != -1
                };
                qa = function() {
                    return na ? !!oa && oa.brands.length > 0 : !1
                };
                _.ra = function() {
                    return qa() ? !1 : _.u("Opera")
                };
                _.sa = function() {
                    return qa() ? !1 : _.u("Trident") || _.u("MSIE")
                };
                _.ta = function() {
                    return _.u("Firefox") || _.u("FxiOS")
                };
                _.va = function() {
                    return _.u("Safari") && !(ua() || (qa() ? 0 : _.u("Coast")) || _.ra() || (qa() ? 0 : _.u("Edge")) || (qa() ? pa("Microsoft Edge") : _.u("Edg/")) || (qa() ? pa("Opera") : _.u("OPR")) || _.ta() || _.u("Silk") || _.u("Android"))
                };
                ua = function() {
                    return qa() ? pa("Chromium") : (_.u("Chrome") || _.u("CriOS")) && !(qa() ? 0 : _.u("Edge")) || _.u("Silk")
                };
                wa = function() {
                    return na ? !!oa && !!oa.platform : !1
                };
                xa = function() {
                    return _.u("iPhone") && !_.u("iPod") && !_.u("iPad")
                };
                _.ya = function() {
                    return xa() || _.u("iPad") || _.u("iPod")
                };
                _.za = function() {
                    return wa() ? oa.platform === "macOS" : _.u("Macintosh")
                };
                _.Ba = function(a, b) {
                    return _.Aa(a, b) >= 0
                };
                _.Ca = function(a, b = !1) {
                    return b && Symbol.for && a ? Symbol.for(a) : a != null ? Symbol(a) : Symbol()
                };
                _.Ea = function(a, b) {
                    return b === void 0 ? a.j !== Da && !!(2 & (a.fa[_.v] | 0)) : !!(2 & b) && a.j !== Da
                };
                Fa = function(a) {
                    return a
                };
                Ga = function(a, b) {
                    a.__closure__error__context__984382 || (a.__closure__error__context__984382 = {});
                    a.__closure__error__context__984382.severity = b
                };
                _.Ha = function(a) {
                    a = Error(a);
                    Ga(a, "warning");
                    return a
                };
                _.Ja = function(a, b) {
                    if (a != null) {
                        var c;
                        var d = (c = Ia) != null ? c : Ia = {};
                        c = d[a] || 0;
                        c >= b || (d[a] = c + 1, a = Error(), Ga(a, "incident"), _.ka(a))
                    }
                };
                _.La = function(a) {
                    if (typeof a !== "boolean") throw Error("k`" + _.Ka(a) + "`" + a);
                    return a
                };
                _.Ma = function(a) {
                    if (a == null || typeof a === "boolean") return a;
                    if (typeof a === "number") return !!a
                };
                _.Oa = function(a) {
                    if (!(0, _.Na)(a)) throw _.Ha("enum");
                    return a | 0
                };
                _.Pa = function(a) {
                    return a == null ? a : (0, _.Na)(a) ? a | 0 : void 0
                };
                _.Qa = function(a) {
                    if (typeof a !== "number") throw _.Ha("int32");
                    if (!(0, _.Na)(a)) throw _.Ha("int32");
                    return a | 0
                };
                _.Ra = function(a) {
                    if (a != null && typeof a !== "string") throw Error();
                    return a
                };
                _.Sa = function(a) {
                    return a == null || typeof a === "string" ? a : void 0
                };
                _.Va = function(a, b, c) {
                    if (a != null && a[_.Ta] === _.Ua) return a;
                    if (Array.isArray(a)) {
                        var d = a[_.v] | 0;
                        c = d | c & 32 | c & 2;
                        c !== d && (a[_.v] = c);
                        return new b(a)
                    }
                };
                _.Ya = function(a) {
                    const b = _.Wa(_.Xa);
                    return b ? a[b] : void 0
                };
                $a = function(a, b) {
                    b < 100 || _.Ja(Za, 1)
                };
                cb = function(a, b, c, d) {
                    const e = d !== void 0;
                    d = !!d;
                    var f = _.Wa(_.Xa),
                        g;
                    !e && f && (g = a[f]) && g.xd($a);
                    f = [];
                    var h = a.length;
                    let k;
                    g = 4294967295;
                    let l = !1;
                    const m = !!(b & 64),
                        p = m ? b & 128 ? 0 : -1 : void 0;
                    if (!(b & 1 || (k = h && a[h - 1], k != null && typeof k === "object" && k.constructor === Object ? (h--, g = h) : k = void 0, !m || b & 128 || e))) {
                        l = !0;
                        var r;
                        g = ((r = ab) != null ? r : Fa)(g - p, p, a, k, void 0) + p
                    }
                    b = void 0;
                    for (r = 0; r < h; r++) {
                        let w = a[r];
                        if (w != null && (w = c(w, d)) != null)
                            if (m && r >= g) {
                                const D = r - p;
                                var q = void 0;
                                ((q = b) != null ? q : b = {})[D] = w
                            } else f[r] = w
                    }
                    if (k)
                        for (let w in k) {
                            q =
                                k[w];
                            if (q == null || (q = c(q, d)) == null) continue;
                            h = +w;
                            let D;
                            if (m && !Number.isNaN(h) && (D = h + p) < g) f[D] = q;
                            else {
                                let Q;
                                ((Q = b) != null ? Q : b = {})[w] = q
                            }
                        }
                    b && (l ? f.push(b) : f[g] = b);
                    e && _.Wa(_.Xa) && (a = _.Ya(a)) && "function" == typeof _.bb && a instanceof _.bb && (f[_.Xa] = a.i());
                    return f
                };
                eb = function(a) {
                    switch (typeof a) {
                        case "number":
                            return Number.isFinite(a) ? a : "" + a;
                        case "bigint":
                            return (0, _.db)(a) ? Number(a) : "" + a;
                        case "boolean":
                            return a ? 1 : 0;
                        case "object":
                            if (Array.isArray(a)) {
                                const b = a[_.v] | 0;
                                return a.length === 0 && b & 1 ? void 0 : cb(a, b, eb)
                            }
                            if (a != null && a[_.Ta] === _.Ua) return fb(a);
                            if ("function" == typeof _.gb && a instanceof _.gb) return a.j();
                            return
                    }
                    return a
                };
                jb = function(a, b) {
                    if (b) {
                        ab = b == null || b === Fa || b[hb] !== ib ? Fa : b;
                        try {
                            return fb(a)
                        } finally {
                            ab = void 0
                        }
                    }
                    return fb(a)
                };
                fb = function(a) {
                    a = a.fa;
                    return cb(a, a[_.v] | 0, eb)
                };
                _.mb = function(a, b, c, d = 0) {
                    if (a == null) {
                        var e = 32;
                        c ? (a = [c], e |= 128) : a = [];
                        b && (e = e & -8380417 | (b & 1023) << 13)
                    } else {
                        if (!Array.isArray(a)) throw Error("l");
                        e = a[_.v] | 0;
                        if (kb && 1 & e) throw Error("m");
                        2048 & e && !(2 & e) && lb();
                        if (e & 256) throw Error("n");
                        if (e & 64) return d !== 0 || e & 2048 || (a[_.v] = e | 2048), a;
                        if (c && (e |= 128, c !== a[0])) throw Error("o");
                        a: {
                            c = a;e |= 64;
                            var f = c.length;
                            if (f) {
                                var g = f - 1;
                                const k = c[g];
                                if (k != null && typeof k === "object" && k.constructor === Object) {
                                    b = e & 128 ? 0 : -1;
                                    g -= b;
                                    if (g >= 1024) throw Error("q");
                                    for (var h in k)
                                        if (f = +h, f <
                                            g) c[f + b] = k[h], delete k[h];
                                        else break;
                                    e = e & -8380417 | (g & 1023) << 13;
                                    break a
                                }
                            }
                            if (b) {
                                h = Math.max(b, f - (e & 128 ? 0 : -1));
                                if (h > 1024) throw Error("r");
                                e = e & -8380417 | (h & 1023) << 13
                            }
                        }
                    }
                    e |= 64;
                    d === 0 && (e |= 2048);
                    a[_.v] = e;
                    return a
                };
                lb = function() {
                    if (kb) throw Error("p");
                    _.Ja(nb, 5)
                };
                rb = function(a, b) {
                    if (typeof a !== "object") return a;
                    if (Array.isArray(a)) {
                        var c = a[_.v] | 0;
                        a.length === 0 && c & 1 ? a = void 0 : c & 2 || (!b || 4096 & c || 16 & c ? a = _.ob(a, c, !1, b && !(c & 16)) : (a[_.v] |= 34, c & 4 && Object.freeze(a)));
                        return a
                    }
                    if (a != null && a[_.Ta] === _.Ua) return b = a.fa, c = b[_.v] | 0, _.Ea(a, c) ? a : _.pb(a, b, c) ? _.qb(a, b) : _.ob(b, c);
                    if ("function" == typeof _.gb && a instanceof _.gb) return a
                };
                _.qb = function(a, b, c) {
                    a = new a.constructor(b);
                    c && (a.j = Da);
                    a.o = Da;
                    return a
                };
                _.ob = function(a, b, c, d) {
                    d != null || (d = !!(34 & b));
                    a = cb(a, b, rb, d);
                    d = 32;
                    c && (d |= 2);
                    b = b & 8380609 | d;
                    a[_.v] = b;
                    return a
                };
                _.tb = function(a) {
                    const b = a.fa,
                        c = b[_.v] | 0;
                    return _.Ea(a, c) ? _.pb(a, b, c) ? _.qb(a, b, !0) : new a.constructor(_.ob(b, c, !1)) : a
                };
                _.ub = function(a) {
                    if (a.j !== Da) return !1;
                    var b = a.fa;
                    b = _.ob(b, b[_.v] | 0);
                    b[_.v] |= 2048;
                    a.fa = b;
                    a.j = void 0;
                    a.o = void 0;
                    return !0
                };
                _.vb = function(a) {
                    if (!_.ub(a) && _.Ea(a, a.fa[_.v] | 0)) throw Error();
                };
                _.wb = function(a, b) {
                    b === void 0 && (b = a[_.v] | 0);
                    b & 32 && !(b & 4096) && (a[_.v] = b | 4096)
                };
                _.pb = function(a, b, c) {
                    return c & 2 ? !0 : c & 32 && !(c & 4096) ? (b[_.v] = c | 2, a.j = Da, !0) : !1
                };
                _.xb = function(a, b, c, d, e) {
                    const f = c + (e ? 0 : -1);
                    var g = a.length - 1;
                    if (g >= 1 + (e ? 0 : -1) && f >= g) {
                        const h = a[g];
                        if (h != null && typeof h === "object" && h.constructor === Object) return h[c] = d, b
                    }
                    if (f <= g) return a[f] = d, b;
                    if (d !== void 0) {
                        let h;
                        g = ((h = b) != null ? h : b = a[_.v] | 0) >> 13 & 1023 || 536870912;
                        c >= g ? d != null && (a[g + (e ? 0 : -1)] = {
                            [c]: d
                        }) : a[f] = d
                    }
                    return b
                };
                _.zb = function(a, b, c, d, e) {
                    let f = !1;
                    d = _.yb(a, d, e, g => {
                        const h = _.Va(g, c, b);
                        f = h !== g && h != null;
                        return h
                    });
                    if (d != null) return f && !_.Ea(d) && _.wb(a, b), d
                };
                _.Ab = function() {
                    const a = class {
                        constructor() {
                            throw Error();
                        }
                    };
                    Object.setPrototypeOf(a, a.prototype);
                    return a
                };
                _.x = function(a, b) {
                    return a != null ? !!a : !!b
                };
                _.y = function(a, b) {
                    b == void 0 && (b = "");
                    return a != null ? a : b
                };
                _.Bb = function(a, b, c) {
                    for (const d in a) b.call(c, a[d], d, a)
                };
                _.Cb = function(a) {
                    for (const b in a) return !1;
                    return !0
                };
                Db = Object.defineProperty;
                Eb = function(a) {
                    a = ["object" == typeof globalThis && globalThis, a, "object" == typeof window && window, "object" == typeof self && self, "object" == typeof global && global];
                    for (var b = 0; b < a.length; ++b) {
                        var c = a[b];
                        if (c && c.Math == Math) return c
                    }
                    throw Error("a");
                };
                Fb = Eb(this);
                Gb = function(a, b) {
                    if (b) a: {
                        var c = Fb;a = a.split(".");
                        for (var d = 0; d < a.length - 1; d++) {
                            var e = a[d];
                            if (!(e in c)) break a;
                            c = c[e]
                        }
                        a = a[a.length - 1];d = c[a];b = b(d);b != d && b != null && Db(c, a, {
                            configurable: !0,
                            writable: !0,
                            value: b
                        })
                    }
                };
                Gb("globalThis", function(a) {
                    return a || Fb
                });
                Gb("Symbol.dispose", function(a) {
                    return a ? a : Symbol("b")
                });
                Gb("Promise.prototype.finally", function(a) {
                    return a ? a : function(b) {
                        return this.then(function(c) {
                            return Promise.resolve(b()).then(function() {
                                return c
                            })
                        }, function(c) {
                            return Promise.resolve(b()).then(function() {
                                throw c;
                            })
                        })
                    }
                });
                Gb("Array.prototype.flat", function(a) {
                    return a ? a : function(b) {
                        b = b === void 0 ? 1 : b;
                        var c = [];
                        Array.prototype.forEach.call(this, function(d) {
                            Array.isArray(d) && b > 0 ? (d = Array.prototype.flat.call(d, b - 1), c.push.apply(c, d)) : c.push(d)
                        });
                        return c
                    }
                });
                var Jb, Kb, Nb;
                _.Hb = _.Hb || {};
                _.t = this || self;
                Jb = function(a, b) {
                    var c = _.Ib("WIZ_global_data.oxN3nb");
                    a = c && c[a];
                    return a != null ? a : b
                };
                Kb = _.t._F_toggles_gbar_ || [];
                _.Ib = function(a, b) {
                    a = a.split(".");
                    b = b || _.t;
                    for (var c = 0; c < a.length; c++)
                        if (b = b[a[c]], b == null) return null;
                    return b
                };
                _.Ka = function(a) {
                    var b = typeof a;
                    return b != "object" ? b : a ? Array.isArray(a) ? "array" : b : "null"
                };
                _.Lb = function(a) {
                    var b = typeof a;
                    return b == "object" && a != null || b == "function"
                };
                _.Mb = "closure_uid_" + (Math.random() * 1E9 >>> 0);
                Nb = function(a, b, c) {
                    return a.call.apply(a.bind, arguments)
                };
                _.z = function(a, b, c) {
                    _.z = Nb;
                    return _.z.apply(null, arguments)
                };
                _.Ob = function(a, b) {
                    var c = Array.prototype.slice.call(arguments, 1);
                    return function() {
                        var d = c.slice();
                        d.push.apply(d, arguments);
                        return a.apply(this, d)
                    }
                };
                _.A = function(a, b) {
                    a = a.split(".");
                    for (var c = _.t, d; a.length && (d = a.shift());) a.length || b === void 0 ? c[d] && c[d] !== Object.prototype[d] ? c = c[d] : c = c[d] = {} : c[d] = b
                };
                _.Wa = function(a) {
                    return a
                };
                _.B = function(a, b) {
                    function c() {}
                    c.prototype = b.prototype;
                    a.X = b.prototype;
                    a.prototype = new c;
                    a.prototype.constructor = a;
                    a.mk = function(d, e, f) {
                        for (var g = Array(arguments.length - 2), h = 2; h < arguments.length; h++) g[h - 2] = arguments[h];
                        return b.prototype[e].apply(d, g)
                    }
                };
                _.B(_.aa, Error);
                _.aa.prototype.name = "CustomError";
                var Pb = !!(Kb[0] >> 15 & 1),
                    Qb = !!(Kb[0] & 1024),
                    Rb = !!(Kb[0] >> 16 & 1),
                    Sb = !!(Kb[0] & 128);
                var Tb = Jb(1, !0),
                    na = Pb ? Rb : Jb(610401301, !1),
                    kb = Pb ? Qb || !Sb : Jb(748402147, Tb);
                _.Ub = _.ba(a => a !== null && a !== void 0);
                var ea = _.ba(a => typeof a === "number"),
                    da = _.ba(a => typeof a === "string"),
                    ha = _.ba(a => typeof a === "boolean");
                var fa = typeof _.t.BigInt === "function" && typeof _.t.BigInt(0) === "bigint";
                var Xb, Vb, Yb, Wb;
                _.db = _.ba(a => fa ? a >= Vb && a <= Wb : a[0] === "-" ? ja(a, Xb) : ja(a, Yb));
                Xb = Number.MIN_SAFE_INTEGER.toString();
                Vb = fa ? BigInt(Number.MIN_SAFE_INTEGER) : void 0;
                Yb = Number.MAX_SAFE_INTEGER.toString();
                Wb = fa ? BigInt(Number.MAX_SAFE_INTEGER) : void 0;
                _.Zb = typeof TextDecoder !== "undefined";
                _.$b = typeof TextEncoder !== "undefined";
                var oa, ac = _.t.navigator;
                oa = ac ? ac.userAgentData || null : null;
                _.Aa = function(a, b) {
                    return Array.prototype.indexOf.call(a, b, void 0)
                };
                _.bc = function(a, b, c) {
                    Array.prototype.forEach.call(a, b, c)
                };
                _.cc = function(a, b) {
                    return Array.prototype.some.call(a, b, void 0)
                };
                _.dc = function(a) {
                    _.dc[" "](a);
                    return a
                };
                _.dc[" "] = function() {};
                var rc;
                _.ec = _.ra();
                _.fc = _.sa();
                _.hc = _.u("Edge");
                _.ic = _.u("Gecko") && !(_.ma() && !_.u("Edge")) && !(_.u("Trident") || _.u("MSIE")) && !_.u("Edge");
                _.jc = _.ma() && !_.u("Edge");
                _.kc = _.za();
                _.lc = wa() ? oa.platform === "Windows" : _.u("Windows");
                _.mc = wa() ? oa.platform === "Android" : _.u("Android");
                _.nc = xa();
                _.oc = _.u("iPad");
                _.pc = _.u("iPod");
                _.qc = _.ya();
                a: {
                    let a = "";
                    const b = function() {
                        const c = _.la();
                        if (_.ic) return /rv:([^\);]+)(\)|;)/.exec(c);
                        if (_.hc) return /Edge\/([\d\.]+)/.exec(c);
                        if (_.fc) return /\b(?:MSIE|rv)[: ]([^\);]+)(\)|;)/.exec(c);
                        if (_.jc) return /WebKit\/(\S+)/.exec(c);
                        if (_.ec) return /(?:Version)[ \/]?(\S+)/.exec(c)
                    }();b && (a = b ? b[1] : "");
                    if (_.fc) {
                        var sc;
                        const c = _.t.document;
                        sc = c ? c.documentMode : void 0;
                        if (sc != null && sc > parseFloat(a)) {
                            rc = String(sc);
                            break a
                        }
                    }
                    rc = a
                }
                _.tc = rc;
                _.uc = _.ta();
                _.vc = xa() || _.u("iPod");
                _.wc = _.u("iPad");
                _.xc = _.u("Android") && !(ua() || _.ta() || _.ra() || _.u("Silk"));
                _.yc = ua();
                _.zc = _.va() && !_.ya();
                var Za, nb, hb;
                _.Xa = _.Ca();
                _.Ac = _.Ca();
                Za = _.Ca();
                _.Bc = _.Ca();
                nb = _.Ca();
                _.Ta = _.Ca("m_m", !0);
                hb = _.Ca();
                _.Cc = _.Ca();
                var Ec;
                _.v = _.Ca("jas", !0);
                Ec = [];
                Ec[_.v] = 7;
                _.Dc = Object.freeze(Ec);
                var Da;
                _.Ua = {};
                Da = {};
                _.Fc = Object.freeze({});
                var ib = {};
                var Ia = void 0;
                _.Gc = typeof BigInt === "function" ? BigInt.asIntN : void 0;
                _.Hc = Number.isSafeInteger;
                _.Na = Number.isFinite;
                _.Ic = Math.trunc;
                var ab;
                _.Jc = _.ia(0);
                _.Kc = {};
                _.Lc = function(a, b, c, d, e) {
                    b = _.yb(a.fa, b, c, e);
                    if (b !== null || d && a.o !== Da) return b
                };
                _.yb = function(a, b, c, d) {
                    if (b === -1) return null;
                    const e = b + (c ? 0 : -1),
                        f = a.length - 1;
                    let g, h;
                    if (!(f < 1 + (c ? 0 : -1))) {
                        if (e >= f)
                            if (g = a[f], g != null && typeof g === "object" && g.constructor === Object) c = g[b], h = !0;
                            else if (e === f) c = g;
                        else return;
                        else c = a[e];
                        if (d && c != null) {
                            d = d(c);
                            if (d == null) return d;
                            if (!Object.is(d, c)) return h ? g[b] = d : a[e] = d, d
                        }
                        return c
                    }
                };
                _.Mc = function(a, b, c, d) {
                    _.vb(a);
                    const e = a.fa;
                    _.xb(e, e[_.v] | 0, b, c, d);
                    return a
                };
                _.C = function(a, b, c, d) {
                    let e = a.fa,
                        f = e[_.v] | 0;
                    b = _.zb(e, f, b, c, d);
                    if (b == null) return b;
                    f = e[_.v] | 0;
                    if (!_.Ea(a, f)) {
                        const g = _.tb(b);
                        g !== b && (_.ub(a) && (e = a.fa, f = e[_.v] | 0), b = g, f = _.xb(e, f, c, b, d), _.wb(e, f))
                    }
                    return b
                };
                _.E = function(a, b, c) {
                    c == null && (c = void 0);
                    _.Mc(a, b, c);
                    c && !_.Ea(c) && _.wb(a.fa);
                    return a
                };
                _.Nc = function(a, b, c, d) {
                    return _.Pa(_.Lc(a, b, c, d))
                };
                _.F = function(a, b, c = !1, d) {
                    let e;
                    return (e = _.Ma(_.Lc(a, b, d))) != null ? e : c
                };
                _.G = function(a, b, c = "", d) {
                    let e;
                    return (e = _.Sa(_.Lc(a, b, d))) != null ? e : c
                };
                _.I = function(a, b, c) {
                    return _.Sa(_.Lc(a, b, c, _.Kc))
                };
                _.J = function(a, b, c, d) {
                    return _.Mc(a, b, c == null ? c : _.La(c), d)
                };
                _.K = function(a, b, c) {
                    return _.Mc(a, b, c == null ? c : _.Qa(c))
                };
                _.L = function(a, b, c, d) {
                    return _.Mc(a, b, _.Ra(c), d)
                };
                _.N = function(a, b, c, d) {
                    return _.Mc(a, b, c == null ? c : _.Oa(c), d)
                };
                _.O = class {
                    constructor(a, b, c) {
                        this.fa = _.mb(a, b, c)
                    }
                    toJSON() {
                        return jb(this)
                    }
                    wa(a) {
                        return JSON.stringify(jb(this, a))
                    }
                };
                _.O.prototype[_.Ta] = _.Ua;
                _.O.prototype.toString = function() {
                    return this.fa.toString()
                };
                _.Oc = _.Ab();
                _.Qc = _.Ab();
                _.Rc = _.Ab();
                _.Sc = Symbol();
                var Tc = class extends _.O {
                    constructor(a) {
                        super(a)
                    }
                };
                _.Uc = class extends _.O {
                    constructor(a) {
                        super(a)
                    }
                    D(a) {
                        return _.K(this, 3, a)
                    }
                };
                _.Vc = class extends _.O {
                    constructor(a) {
                        super(a)
                    }
                };
                _.P = function() {
                    this.qa = this.qa;
                    this.Y = this.Y
                };
                _.P.prototype.qa = !1;
                _.P.prototype.isDisposed = function() {
                    return this.qa
                };
                _.P.prototype.dispose = function() {
                    this.qa || (this.qa = !0, this.R())
                };
                _.P.prototype[Symbol.dispose] = function() {
                    this.dispose()
                };
                _.P.prototype.R = function() {
                    if (this.Y)
                        for (; this.Y.length;) this.Y.shift()()
                };
                var Wc = class extends _.P {
                    constructor() {
                        var a = window;
                        super();
                        this.o = a;
                        this.i = [];
                        this.j = {}
                    }
                    resolve(a) {
                        let b = this.o;
                        a = a.split(".");
                        const c = a.length;
                        for (let d = 0; d < c; ++d)
                            if (b[a[d]]) b = b[a[d]];
                            else return null;
                        return b instanceof Function ? b : null
                    }
                    ub() {
                        const a = this.i.length,
                            b = this.i,
                            c = [];
                        for (let d = 0; d < a; ++d) {
                            const e = b[d].i(),
                                f = this.resolve(e);
                            if (f && f != this.j[e]) try {
                                b[d].ub(f)
                            } catch (g) {} else c.push(b[d])
                        }
                        this.i = c.concat(b.slice(a))
                    }
                };
                var Yc = class extends _.P {
                    constructor() {
                        var a = _.Xc;
                        super();
                        this.o = a;
                        this.A = this.i = null;
                        this.v = 0;
                        this.B = {};
                        this.j = !1;
                        a = window.navigator.userAgent;
                        a.indexOf("MSIE") >= 0 && a.indexOf("Trident") >= 0 && (a = /\b(?:MSIE|rv)[: ]([^\);]+)(\)|;)/.exec(a)) && a[1] && parseFloat(a[1]) < 9 && (this.j = !0)
                    }
                    C(a, b) {
                        this.i = b;
                        this.A = a;
                        b.preventDefault ? b.preventDefault() : b.returnValue = !1
                    }
                };
                _.Zc = class extends _.O {
                    constructor(a) {
                        super(a)
                    }
                };
                var $c = class extends _.O {
                    constructor(a) {
                        super(a)
                    }
                };
                var cd;
                _.ad = function(a, b, c = 98, d = new _.Uc) {
                    if (a.i) {
                        const e = new Tc;
                        _.L(e, 1, b.message);
                        _.L(e, 2, b.stack);
                        _.K(e, 3, b.lineNumber);
                        _.N(e, 5, 1);
                        _.E(d, 40, e);
                        a.i.log(c, d)
                    }
                };
                cd = class {
                    constructor() {
                        var a = bd;
                        this.i = null;
                        _.F(a, 4, !0)
                    }
                    log(a, b, c = new _.Uc) {
                        _.ad(this, a, 98, c)
                    }
                };
                var dd, ed;
                dd = function(a) {
                    if (a.o.length > 0) {
                        var b = a.i !== void 0,
                            c = a.j !== void 0;
                        if (b || c) {
                            b = b ? a.v : a.A;
                            c = a.o;
                            a.o = [];
                            try {
                                _.bc(c, b, a)
                            } catch (d) {
                                console.error(d)
                            }
                        }
                    }
                };
                _.fd = class {
                    constructor(a) {
                        this.i = a;
                        this.j = void 0;
                        this.o = []
                    }
                    then(a, b, c) {
                        this.o.push(new ed(a, b, c));
                        dd(this)
                    }
                    resolve(a) {
                        if (this.i !== void 0 || this.j !== void 0) throw Error("v");
                        this.i = a;
                        dd(this)
                    }
                    reject(a) {
                        if (this.i !== void 0 || this.j !== void 0) throw Error("v");
                        this.j = a;
                        dd(this)
                    }
                    v(a) {
                        a.j && a.j.call(a.i, this.i)
                    }
                    A(a) {
                        a.o && a.o.call(a.i, this.j)
                    }
                };
                ed = class {
                    constructor(a, b, c) {
                        this.j = a;
                        this.o = b;
                        this.i = c
                    }
                };
                _.gd = a => {
                    var b = "rc";
                    if (a.rc && a.hasOwnProperty(b)) return a.rc;
                    b = new a;
                    return a.rc = b
                };
                _.R = class {
                    constructor() {
                        this.v = new _.fd;
                        this.i = new _.fd;
                        this.D = new _.fd;
                        this.B = new _.fd;
                        this.C = new _.fd;
                        this.A = new _.fd;
                        this.o = new _.fd;
                        this.j = new _.fd;
                        this.F = new _.fd;
                        this.G = new _.fd
                    }
                    K() {
                        return this.v
                    }
                    qa() {
                        return this.i
                    }
                    O() {
                        return this.D
                    }
                    M() {
                        return this.B
                    }
                    P() {
                        return this.C
                    }
                    L() {
                        return this.A
                    }
                    Y() {
                        return this.o
                    }
                    J() {
                        return this.j
                    }
                    N() {
                        return this.F
                    }
                    static i() {
                        return _.gd(_.R)
                    }
                };
                var jd;
                _.id = function() {
                    return _.C(_.hd, _.Vc, 5)
                };
                jd = class extends _.O {
                    constructor(a) {
                        super(a)
                    }
                };
                var kd;
                window.gbar_ && window.gbar_.CONFIG ? kd = window.gbar_.CONFIG[0] || {} : kd = [];
                _.hd = new jd(kd);
                var bd;
                bd = _.C(_.hd, $c, 3) || new $c;
                _.Xc = new cd;
                _.A("gbar_._DumpException", function(a) {
                    _.Xc ? _.Xc.log(a) : console.error(a)
                });
                _.ld = new Yc;
                var nd;
                _.od = function(a, b) {
                    var c = _.md.i();
                    if (a in c.i) {
                        if (c.i[a] != b) throw new nd;
                    } else {
                        c.i[a] = b;
                        const h = c.j[a];
                        if (h)
                            for (let k = 0, l = h.length; k < l; k++) {
                                b = h[k];
                                var d = c.i;
                                delete b.i[a];
                                if (_.Cb(b.i)) {
                                    for (var e = b.j.length, f = Array(e), g = 0; g < e; g++) f[g] = d[b.j[g]];
                                    b.o.apply(b.v, f)
                                }
                            }
                        delete c.j[a]
                    }
                };
                _.md = class {
                    constructor() {
                        this.i = {};
                        this.j = {}
                    }
                    static i() {
                        return _.gd(_.md)
                    }
                };
                _.pd = class extends _.aa {
                    constructor() {
                        super()
                    }
                };
                nd = class extends _.pd {};
                _.A("gbar.A", _.fd);
                _.fd.prototype.aa = _.fd.prototype.then;
                _.A("gbar.B", _.R);
                _.R.prototype.ba = _.R.prototype.qa;
                _.R.prototype.bb = _.R.prototype.O;
                _.R.prototype.bd = _.R.prototype.P;
                _.R.prototype.bf = _.R.prototype.K;
                _.R.prototype.bg = _.R.prototype.M;
                _.R.prototype.bh = _.R.prototype.L;
                _.R.prototype.bj = _.R.prototype.Y;
                _.R.prototype.bk = _.R.prototype.J;
                _.R.prototype.bl = _.R.prototype.N;
                _.A("gbar.a", _.R.i());
                window.gbar && window.gbar.ap && window.gbar.ap(window.gbar.a);
                var qd = new Wc;
                _.od("api", qd);
                var rd = _.id() || new _.Vc,
                    sd = window,
                    td = _.y(_.I(rd, 8));
                sd.__PVT = td;
                _.od("eq", _.ld);
            } catch (e) {
                _._DumpException(e)
            }
            try {
                _.ud = class extends _.O {
                    constructor(a) {
                        super(a)
                    }
                };
            } catch (e) {
                _._DumpException(e)
            }
            try {
                var vd = class extends _.O {
                    constructor(a) {
                        super(a)
                    }
                };
                var wd = class extends _.P {
                    constructor() {
                        super();
                        this.j = [];
                        this.i = []
                    }
                    o(a, b) {
                        this.j.push({
                            features: a,
                            options: b != null ? b : null
                        })
                    }
                    init(a, b, c) {
                        window.gapi = {};
                        const d = window.___jsl = {};
                        d.h = _.y(_.I(a, 1));
                        _.Ma(_.Lc(a, 12)) != null && (d.dpo = _.x(_.F(a, 12)));
                        d.ms = _.y(_.I(a, 2));
                        d.m = _.y(_.I(a, 3));
                        d.l = [];
                        _.G(b, 1) && (a = _.I(b, 3)) && this.i.push(a);
                        _.G(c, 1) && (c = _.I(c, 2)) && this.i.push(c);
                        _.A("gapi.load", (0, _.z)(this.o, this));
                        return this
                    }
                };
                var xd = _.C(_.hd, _.Zc, 14);
                if (xd) {
                    var yd = _.C(_.hd, _.ud, 9) || new _.ud,
                        zd = new vd,
                        Bd = new wd;
                    Bd.init(xd, yd, zd);
                    _.od("gs", Bd)
                };
            } catch (e) {
                _._DumpException(e)
            }
        })(this.gbar_);
        // Google Inc.
    </script>
    <style>
        .gb_ub{font:13px/27px Roboto,Arial,sans-serif;z-index:986}.gb_R{display:none}.gb_Q{-webkit-background-size:32px 32px;background-size:32px 32px;border:0;-webkit-border-radius:50%;border-radius:50%;display:block;margin:0px;position:relative;height:32px;width:32px;z-index:0}.gb_gb{background-color:#e8f0fe;border:1px solid rgba(32,33,36,.08);position:relative}.gb_gb.gb_Q{height:30px;width:30px}.gb_gb.gb_Q:hover,.gb_gb.gb_Q:active{-webkit-box-shadow:none;box-shadow:none}.gb_hb{background:#fff;border:none;-webkit-border-radius:50%;border-radius:50%;bottom:2px;-webkit-box-shadow:0px 1px 2px 0px rgba(60,64,67,.30),0px 1px 3px 1px rgba(60,64,67,.15);box-shadow:0px 1px 2px 0px rgba(60,64,67,.30),0px 1px 3px 1px rgba(60,64,67,.15);height:14px;margin:2px;position:absolute;right:0;width:14px;line-height:normal;z-index:1}.gb_ib{color:#1f71e7;font:400 22px/32px Google Sans,Roboto,Helvetica,Arial,sans-serif;text-align:center;text-transform:uppercase}@media (-webkit-min-device-pixel-ratio:1.25),(min-resolution:1.25dppx),(min-device-pixel-ratio:1.25){.gb_Q::before,.gb_jb::before{display:inline-block;-webkit-transform:scale(0.5);-webkit-transform:scale(0.5);transform:scale(0.5);-webkit-transform-origin:left 0;-webkit-transform-origin:left 0;transform-origin:left 0}.gb_4 .gb_jb::before{-webkit-transform:scale(scale(0.416666667));-webkit-transform:scale(scale(0.416666667));transform:scale(scale(0.416666667))}}.gb_Q:hover,.gb_Q:focus{-webkit-box-shadow:0 1px 0 rgba(0,0,0,.15);box-shadow:0 1px 0 rgba(0,0,0,.15)}.gb_Q:active{-webkit-box-shadow:inset 0 2px 0 rgba(0,0,0,.15);box-shadow:inset 0 2px 0 rgba(0,0,0,.15)}.gb_Q:active::after{background:rgba(0,0,0,.1);-webkit-border-radius:50%;border-radius:50%;content:"";display:block;height:100%}.gb_kb{cursor:pointer;line-height:40px;min-width:30px;opacity:.75;overflow:hidden;vertical-align:middle;text-overflow:ellipsis}.gb_B.gb_kb{width:auto}.gb_kb:hover,.gb_kb:focus{opacity:.85}.gb_lb .gb_kb,.gb_lb .gb_mb{line-height:26px}#gb#gb.gb_lb a.gb_kb,.gb_lb .gb_mb{font-size:11px;height:auto}.gb_nb{border-top:4px solid #000;border-left:4px dashed transparent;border-right:4px dashed transparent;display:inline-block;margin-left:6px;opacity:.75;vertical-align:middle}.gb_0a:hover .gb_nb{opacity:.85}.gb_Xa>.gb_z{padding:3px 3px 3px 4px}.gb_ob.gb_fb{color:#fff}.gb_2 .gb_kb,.gb_2 .gb_nb{opacity:1}#gb#gb.gb_2.gb_2 a.gb_kb,#gb#gb .gb_2.gb_2 a.gb_kb{color:#fff}.gb_2.gb_2 .gb_nb{border-top-color:#fff;opacity:1}.gb_la .gb_Q:hover,.gb_2 .gb_Q:hover,.gb_la .gb_Q:focus,.gb_2 .gb_Q:focus{-webkit-box-shadow:0 1px 0 rgba(0,0,0,.15),0 1px 2px rgba(0,0,0,.2);box-shadow:0 1px 0 rgba(0,0,0,.15),0 1px 2px rgba(0,0,0,.2)}.gb_pb .gb_z,.gb_qb .gb_z{position:absolute;right:1px}.gb_z.gb_1,.gb_rb.gb_1,.gb_0a.gb_1{-webkit-flex:0 1 auto;-webkit-box-flex:0;flex:0 1 auto}.gb_sb.gb_tb .gb_kb{width:30px!important}.gb_P{height:40px;position:absolute;right:-5px;top:-5px;width:40px}.gb_ub .gb_P,.gb_vb .gb_P{right:0;top:0}a.gb_Va{border-radius:100px;background:#0b57d0;background:var(--gm3-sys-color-primary,#0b57d0);box-sizing:border-box;color:#fff;color:var(--gm3-sys-color-on-primary,#fff);display:inline-block;font-size:14px;font-weight:500;min-height:40px;outline:none;padding:10px 24px;text-align:center;text-decoration:none;white-space:normal;line-height:18px;position:relative}a.gb_Wa{border-radius:100px;border:1px solid;border-color:#747775;border-color:var(--gm3-sys-color-outline,#747775);background:none;box-sizing:border-box;color:#0b57d0;color:var(--gm3-sys-color-primary,#0b57d0);display:inline-block;font-size:14px;font-weight:500;min-height:40px;outline:none;padding:10px 24px;text-align:center;text-decoration:none;white-space:normal;line-height:18px;position:relative}.gb_1a.gb_H a.gb_Va,.gb_2a.gb_H a.gb_Va,.gb_3a.gb_H a.gb_Va{background:#c2e7ff;background:var(--gm3-sys-color-secondary-fixed,#c2e7ff);color:#001d35;color:var(--gm3-sys-color-on-secondary-fixed,#001d35)}.gb_Ha.gb_H a.gb_Wa{color:#a8c7fa;color:var(--gm3-sys-color-primary,#a8c7fa)}a.gb_Md{padding:10px 12px;margin:12px 16px 12px 10px;min-width:85px}@media (max-width:640px){a.gb_Md{min-width:75px}}.gb_Ha,.gb_Bd{font-family:"Google Sans Text",Roboto,Helvetica,Arial,sans-serif;font-style:normal}.gb_Ha.gb_1a{color:#1f1f1f;color:var(--og-bar-color,var(--gm3-sys-color-on-surface,#1f1f1f))}.gb_Ha.gb_1a.gb_Nd{background:#fff;background:var(--og-bar-background,var(--gm3-sys-color-background,#fff))}.gb_Ha.gb_1a .gb_nd.gb_od,.gb_Ha.gb_1a a.gb_Z,.gb_Ha.gb_1a span.gb_Z{color:#1f1f1f;color:var(--og-link-color,var(--gm3-sys-color-on-surface,#1f1f1f))}.gb_Ha.gb_1a .gb_pd .gb_Od,.gb_Ha.gb_1a .gb_gd .gb_Od{color:#1f1f1f;color:var(--og-logo-color,var(--gm3-sys-color-on-surface,#1f1f1f))}.gb_Ha.gb_1a svg{color:#444746;color:var(--og-svg-color,var(--gm3-sys-color-on-surface-variant,#444746))}@media (forced-colors:active) and (prefers-color-scheme:dark){.gb_Ha svg,.gb_Ha.gb_1a svg,.gb_Ha.gb_H svg{color:white}}.gb_Ha.gb_H.gb_1a{color:#e3e3e3;color:var(--og-bar-color,var(--gm3-sys-color-on-surface,#e3e3e3))}.gb_Ha.gb_H.gb_1a.gb_Nd{background:transparent}.gb_Ha.gb_H.gb_1a .gb_nd.gb_od,.gb_Ha.gb_H.gb_1a a.gb_Z,.gb_Ha.gb_H.gb_1a span.gb_Z{color:#e3e3e3;color:var(--og-link-color,var(--gm3-sys-color-on-surface,#e3e3e3))}.gb_Ha.gb_H.gb_1a .gb_pd .gb_Od,.gb_Ha.gb_H.gb_1a .gb_gd .gb_Od{color:#e3e3e3;color:var(--og-logo-color,var(--gm3-sys-color-on-surface,#e3e3e3))}.gb_Ha.gb_H.gb_1a svg{color:#c4c7c5;color:var(--og-svg-color,var(--gm3-sys-color-on-surface-variant,#c4c7c5))}.gb_Ha.gb_H.gb_1a.gb_Nd{background:#1f1f1f;background:var(--og-bar-background,var(--gm3-sys-color-background,#131314))}.gb_Ha.gb_2a{color:#1f1f1f;color:var(--og-bar-color,var(--gm3-sys-color-on-surface,#1f1f1f))}.gb_Ha.gb_2a.gb_Nd{background:#e9eef6;background:var(--og-bar-background,var(--gm3-sys-color-surface-container-high,#e9eef6))}.gb_Ha.gb_2a .gb_nd.gb_od,.gb_Ha.gb_2a a.gb_Z,.gb_Ha.gb_2a span.gb_Z{color:#1f1f1f;color:var(--og-link-color,var(--gm3-sys-color-on-surface,#1f1f1f))}.gb_Ha.gb_2a .gb_pd .gb_Od,.gb_Ha.gb_2a .gb_gd .gb_Od{color:#1f1f1f;color:var(--og-logo-color,var(--gm3-sys-color-on-surface,#1f1f1f))}.gb_Ha.gb_2a svg{color:#444746;color:var(--og-svg-color,var(--gm3-sys-color-on-surface-variant,#444746))}.gb_Ha.gb_H.gb_2a{color:#e3e3e3;color:var(--og-bar-color,var(--gm3-sys-color-on-surface,#e3e3e3))}.gb_Ha.gb_H.gb_2a.gb_Nd{background:#282a2c;background:var(--og-bar-background,var(--gm3-sys-color-surface-container-high,#282a2c))}.gb_Ha.gb_H.gb_2a .gb_nd.gb_od,.gb_Ha.gb_H.gb_2a a.gb_Z,.gb_Ha.gb_H.gb_2a span.gb_Z{color:#e3e3e3;color:var(--og-link-color,var(--gm3-sys-color-on-surface,#e3e3e3))}.gb_Ha.gb_H.gb_2a .gb_pd .gb_Od,.gb_Ha.gb_H.gb_2a .gb_gd .gb_Od{color:#e3e3e3;color:var(--og-logo-color,var(--gm3-sys-color-on-surface,#e3e3e3))}.gb_Ha.gb_H.gb_2a svg{color:#c4c7c5;color:var(--og-svg-color,var(--gm3-sys-color-on-surface-variant,#c4c7c5))}.gb_Ha.gb_3a{color:#1f1f1f;color:var(--og-bar-color,var(--gm3-sys-color-on-surface,#1f1f1f))}.gb_Ha.gb_3a.gb_Nd{background:transparent}.gb_Ha.gb_3a .gb_nd.gb_od,.gb_Ha.gb_3a a.gb_Z,.gb_Ha.gb_3a span.gb_Z{color:#1f1f1f;color:var(--og-link-color,var(--gm3-sys-color-on-surface,#1f1f1f))}.gb_Ha.gb_3a .gb_pd .gb_Od,.gb_Ha.gb_3a .gb_gd .gb_Od{color:#1f1f1f;color:var(--og-logo-color,var(--gm3-sys-color-on-surface,#1f1f1f))}.gb_Ha.gb_3a svg{color:#444746;color:var(--og-svg-color,var(--gm3-sys-color-on-surface-variant,#444746))}.gb_Ha.gb_3a.gb_H.gb_Nd{background:transparent}.gb_Ha.gb_3a.gb_H .gb_nd.gb_od,.gb_Ha.gb_3a.gb_H a.gb_Z,.gb_Ha.gb_3a.gb_H span.gb_Z{color:white;color:var(--og-theme-color,white)}.gb_Ha.gb_3a.gb_H .gb_pd .gb_Od,.gb_Ha.gb_3a.gb_H .gb_gd .gb_Od{color:white;color:var(--og-theme-color,white)}.gb_Ha.gb_3a.gb_H svg{color:white;color:var(--og-theme-color,white)}.gb_Ha a.gb_Z,.gb_Ha span.gb_Z{text-decoration:none}.gb_nd{font-family:Google Sans,Roboto,Helvetica,Arial,sans-serif;font-size:20px;font-weight:400;letter-spacing:.25px;line-height:48px;margin-bottom:2px;opacity:1;overflow:hidden;padding-left:16px;position:relative;text-overflow:ellipsis;vertical-align:middle;top:2px;white-space:nowrap;-webkit-box-flex:1;-webkit-flex:1 1 auto;flex:1 1 auto}.gb_sd{display:none}.gb_Ha.gb_oc .gb_nd{margin-bottom:0}.gb_pd.gb_qd .gb_nd{padding-left:4px}.gb_Ha.gb_oc .gb_rd{position:relative;top:-2px}.gb_Ha{min-width:160px;position:relative}.gb_Ha.gb_7c{min-width:120px}.gb_Ha.gb_Pd .gb_Qd{display:none}.gb_Ha.gb_Pd .gb_Id{height:56px}header.gb_Ha{display:block}.gb_Ha svg{fill:currentColor}.gb_Rd{position:fixed;top:0;width:100%}.gb_Sd{box-shadow:0 4px 5px 0 rgba(0,0,0,.14),0 1px 10px 0 rgba(0,0,0,.12),0 2px 4px -1px rgba(0,0,0,.2)}.gb_Td{height:64px}.gb_Id{box-sizing:border-box;position:relative;width:100%;display:-webkit-box;display:-webkit-flex;display:flex;-webkit-box-pack:justify;-webkit-justify-content:space-between;justify-content:space-between;min-width:-webkit-min-content;min-width:min-content}.gb_Ha:not(.gb_oc) .gb_Id{padding:8px}.gb_Ha:not(.gb_oc) .gb_Id a.gb_Ud{margin:12px 8px 12px 10px}.gb_Ha.gb_Vd .gb_Id{-webkit-box-flex:1;-webkit-flex:1 0 auto;flex:1 0 auto}.gb_Ha .gb_Id.gb_Jd.gb_Wd{min-width:0}.gb_Ha.gb_oc .gb_Id{padding:4px;padding-left:8px;min-width:0}.gb_Ha.gb_oc .gb_Id a.gb_Ud{margin:12px 8px 12px 10px}.gb_Qd{height:48px;vertical-align:middle;white-space:nowrap;-webkit-box-align:center;-webkit-align-items:center;align-items:center;display:-webkit-box;display:-webkit-flex;display:flex;-webkit-user-select:none;user-select:none}.gb_Xd>.gb_Qd{display:table-cell;width:100%}.gb_pd{padding-right:25px;box-sizing:border-box;-webkit-box-flex:1;-webkit-flex:1 0 auto;flex:1 0 auto}.gb_Ha.gb_oc .gb_pd{padding-right:14px}.gb_Zd{-webkit-box-flex:1;-webkit-flex:1 1 100%;flex:1 1 100%}.gb_Zd>:only-child{display:inline-block}.gb_0d.gb_hd{padding-left:4px}.gb_0d.gb_1d,.gb_Ha.gb_Vd .gb_0d,.gb_Ha.gb_oc:not(.gb_Bd) .gb_0d{padding-left:0}.gb_Ha.gb_oc .gb_0d.gb_1d{padding-right:0}.gb_Ha.gb_oc .gb_0d.gb_1d .gb_Xa{margin-left:10px}.gb_hd{display:inline}.gb_Ha.gb_bd .gb_0d.gb_2d,.gb_Ha.gb_Bd .gb_0d.gb_2d{padding-left:2px}.gb_nd{display:inline-block}.gb_0d{box-sizing:border-box;height:48px;padding:0 4px;padding-left:5px;-webkit-box-flex:0;-webkit-flex:0 0 auto;flex:0 0 auto;-webkit-box-pack:end;-webkit-justify-content:flex-end;justify-content:flex-end}.gb_Bd{height:48px}.gb_Ha.gb_Bd{min-width:auto}.gb_Bd .gb_0d{float:right;padding-left:32px;padding-left:var(--og-bar-parts-side-padding,32px)}.gb_Bd .gb_0d.gb_3d{padding-left:0}.gb_4d{font-size:14px;max-width:200px;overflow:hidden;padding:0 12px;text-overflow:ellipsis;white-space:nowrap;-webkit-user-select:text;user-select:text}.gb_a a,.gb_4c a{color:inherit}.gb_od{text-rendering:optimizeLegibility;-webkit-font-smoothing:antialiased;-moz-osx-font-smoothing:grayscale}.gb_od{opacity:1}.gb_5d{position:relative}.gb_M{font-family:arial,sans-serif;line-height:normal;padding-right:15px}.gb_0{display:inline-block;padding-left:15px}.gb_0 .gb_Z{display:inline-block;line-height:24px;vertical-align:middle}.gb_6d{text-align:left}.gb_K{display:none}@media screen and (max-width:319px){.gb_Id .gb_J{display:none;visibility:hidden}}.gb_J .gb_B,.gb_J .gb_B:hover,.gb_J .gb_B:focus{opacity:1}.gb_L{display:none}.gb_S{display:none!important}.gb_fb{visibility:hidden}@media screen and (max-width:319px){.gb_Id:not(.gb_Jd) .gb_J{display:none;visibility:hidden}}.gb_td{display:inline-block;vertical-align:middle}.gb_ud .gb_R{bottom:-3px;right:-5px}@if (RTL_LANG){.gb_ud .gb_R{left:-5px}}.gb_td:first-child{padding-left:0}.gb_D{position:relative}.gb_B{display:inline-block;outline:none;vertical-align:middle;border-radius:50%;box-sizing:border-box;height:40px;width:40px}.gb_B,#gb#gb a.gb_B{cursor:pointer;text-decoration:none}.gb_B,a.gb_B{color:#000}x:-o-prefocus{border-bottom-color:#ccc}.gb_ma{background:#fff;border:1px solid #ccc;border-color:rgba(0,0,0,.2);color:#000;box-shadow:0 2px 10px rgba(0,0,0,.2);display:none;outline:none;overflow:hidden;position:absolute;right:0;top:54px;-webkit-animation:gb__a .2s;animation:gb__a .2s;border-radius:2px;-moz-user-select:text;-ms-user-select:text;-webkit-user-select:text}.gb_td.gb_8c .gb_ma,.gb_8c.gb_ma{display:block}.gb_yd{position:absolute;right:0;top:54px;z-index:-1}.gb_lb .gb_ma{margin-top:-10px}.gb_td:first-child{padding-left:4px}.gb_Ha.gb_zd .gb_td:first-child{padding-left:0}.gb_Ad{position:relative}.gb_gd .gb_Ad,.gb_Bd .gb_Ad{float:right}.gb_B{padding:8px;cursor:pointer}.gb_Dd button svg,.gb_B{border-radius:50%}.gb_td{padding:4px}.gb_Ha.gb_zd .gb_td{padding:4px 2px}.gb_Ha.gb_zd .gb_z.gb_td{padding-left:6px}.gb_ma{z-index:991;line-height:normal}.gb_ma.gb_Fd{left:0;right:auto}@media (max-width:350px){.gb_ma.gb_Fd{left:0}}.gb_Hd .gb_ma{top:56px}.gb_z .gb_B{padding:4px}.gb_T{display:none}.gb_0a:not(.gb_Ud){position:relative}.gb_ae::after{content:"";border:1px solid #202124;opacity:.13;position:absolute;top:4px;left:4px;border-radius:50%;width:30px;height:30px}.gb_Xa{-webkit-box-sizing:border-box;box-sizing:border-box;cursor:pointer;display:inline-block;height:48px;overflow:hidden;outline:none;padding:7px 0 0 16px;vertical-align:middle;width:142px;-webkit-border-radius:28px;border-radius:28px;background-color:transparent;border:1px solid;position:relative}.gb_Xa .gb_0a{width:32px;height:32px;padding:0}.gb_Xa .gb_0a .gb_Xa{bottom:-2px}.gb_1a .gb_Xa,.gb_2a .gb_Xa{border-color:#747775;border-color:var(--og-dasher-chip-outline,var(--gm3-sys-color-outline,#747775))}.gb_1a.gb_H .gb_Xa,.gb_2a.gb_H .gb_Xa{border-color:#8e918f;border-color:var(--og-dasher-chip-outline,var(--gm3-sys-color-outline,#8e918f))}.gb_3a .gb_Xa{border-color:#747775;border-color:var(--og-dasher-chip-outline,var(--gm3-sys-color-outline,#747775))}.gb_3a.gb_H .gb_Xa{border-color:#e3e3e3;border-color:var(--og-dasher-chip-outline,var(--gm3-sys-color-on-surface,#e3e3e3))}.gb_4a{display:inherit}.gb_Xa .gb_4a{background:#fff;-webkit-border-radius:6px;border-radius:6px;display:inline-block;left:15px;position:initial;padding:2px;top:-1px;height:32px;-webkit-box-sizing:border-box;box-sizing:border-box;width:78px}.gb_5a{text-align:center}.gb_5a.gb_6a{background-color:#f1f3f4}.gb_5a .gb_Vc{vertical-align:middle;max-height:28px;max-width:74px}.gb_Ha .gb_Xa .gb_z.gb_td{padding:0;margin-right:9px;float:right}.gb_Ha:not(.gb_oc) .gb_Xa{margin-left:10px;margin-right:4px}.gb_Xa .gb_ae::after{left:0;top:0}@media screen and (max-width:480px){.gb_Xa .gb_4a{display:none}.gb_Xa{border:none;-webkit-border-radius:50%;border-radius:50%;height:40px;margin:4px;outline:1px solid transparent;padding:0;width:40px}.gb_Ha .gb_Xa .gb_z.gb_td{padding:4px;margin-right:0}}sentinel{}
    </style>
    <div id="ZnpjSd"></div>
    <div>
        <div jscontroller="ms4mZb" jsname="FAeNCb" data-aipm="1" data-eaip="true" data-ecom="true" data-endpoint="1" data-oaipm="0" id="_v8HjaMKuBpCvptQP9sqQyAs_1" data-jiis="up" data-async-type="hpba" class="yp" jsaction="rcuQ6b:npT2md" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQj-0KCAE"></div>
    </div>
    <div id="gb-main">
        <div id="mpd"></div>
        <header>
            <div jscontroller="eT8Xz" id="og-te" aria-label="Search settings" jsaction="rcuQ6b:npT2md" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQmpcCCAI">
                <style>
                    #navd{position:absolute;top:0}.KojFAc:focus{outline:none}.xuwOJb{display:none;overflow:hidden;position:fixed;inset:0;z-index:199}.SSH64d{background-color:var(--gVUKcd);opacity:0.001;position:absolute;inset:0;z-index:-1;transition:250ms linear}.KojFAc{padding:12px 16px}.emdozc a{text-decoration:none}
                </style>
                <div jsname="hyP9Qc" class="KojFAc" aria-label="Main menu" aria-expanded="false" aria-haspopup="true" role="button" tabindex="0" jsaction="iQ2XYe"><svg style="fill:var(--IXoxUe);width:24px;height:24px" aria-hidden="true" viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg"><path d="M0 0h24v24H0z" fill="none"></path><path d="M3 18h18v-2H3v2zm0-5h18v-2H3v2zm0-7v2h18V6H3z"></path></svg></div>
            </div>
            <div id="SBmmZd"><a class="active" aria-current="true">All</a><a href="https://www.google.com/webhp?sca_esv=bbe3debe957613b7&amp;output=search&amp;tbm=isch&amp;tbo=u&amp;sa=X&amp;ved=0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQ0tQDCAUoAQ">Images</a></div>
            <div class="igu8Pb"></div>
            <div id="vLkmZd">
                <style>
                    #gb{}
                </style>
                <div jscontroller="SJpD2c" jsname="uZkjhb" data-dmt="" data-so="" jsaction="rcuQ6b:npT2md;YQNWc:MMAZpe;zWO23b:FPeyVe" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQnJQPCAY">
                    <div class="gb_Ha gb_Bd gb_ub gb_oc gb_e gb_3a" id="gb">
                        <div class="gb_0d gb_sb gb_Qd gb_3d" data-ogsr-up="">
                            <div class="gb_Ad">
                                <div class="gb_hd">
                                    <div class="gb_J gb_td gb_1" data-ogsr-fb="true" data-ogsr-alt="" id="gbwa">
                                        <div class="gb_D"><a class="gb_B" aria-label="Google apps" href="https://www.google.com/intl/en/about/products?tab=wh" aria-expanded="false" role="button" tabindex="0"><svg class="gb_F" focusable="false" viewbox="0 0 24 24"><path d="M6,8c1.1,0 2,-0.9 2,-2s-0.9,-2 -2,-2 -2,0.9 -2,2 0.9,2 2,2zM12,20c1.1,0 2,-0.9 2,-2s-0.9,-2 -2,-2 -2,0.9 -2,2 0.9,2 2,2zM6,20c1.1,0 2,-0.9 2,-2s-0.9,-2 -2,-2 -2,0.9 -2,2 0.9,2 2,2zM6,14c1.1,0 2,-0.9 2,-2s-0.9,-2 -2,-2 -2,0.9 -2,2 0.9,2 2,2zM12,14c1.1,0 2,-0.9 2,-2s-0.9,-2 -2,-2 -2,0.9 -2,2 0.9,2 2,2zM16,6c0,1.1 0.9,2 2,2s2,-0.9 2,-2 -0.9,-2 -2,-2 -2,0.9 -2,2zM12,8c1.1,0 2,-0.9 2,-2s-0.9,-2 -2,-2 -2,0.9 -2,2 0.9,2 2,2zM18,14c1.1,0 2,-0.9 2,-2s-0.9,-2 -2,-2 -2,0.9 -2,2 0.9,2 2,2zM18,20c1.1,0 2,-0.9 2,-2s-0.9,-2 -2,-2 -2,0.9 -2,2 0.9,2 2,2z"></path><image src="https://ssl.gstatic.com/gb/images/bar/al-icon.png" alt="" height="24" width="24" style="border:none;display:none \9"></image></svg></a></div>
                                    </div>
                                </div><a class="gb_Va gb_Ud gb_Md gb_Cd" aria-label="Sign in" href="https://accounts.google.com/ServiceLogin?hl=en&amp;passive=true&amp;continue=https://www.google.com/&amp;ec=GAZAAQ" target="_top"><span class="gb_be">Sign in</span></a></div>
                        </div>
                    </div>
                    <div jsname="pxGDvf" style="display:none" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQqsEHCAc"></div>
                    <div jsname="nSxrjf" style="display:none" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQtsANCAg"></div>
                    <div jsname="NG1V7d" style="display:none" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQ8JsICAk"></div>
                    <div jsname="IZKNDc" style="display:none" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQ8ZsICAo"></div>
                    <div jsname="tP7ayb" style="display:none" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQ65sICAs"></div>
                    <div jsname="uO6mde" style="display:none" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQ8psICAw"></div>
                </div>
            </div>
        </header>
        <div id="n0tgWb">
            <div style="text-align:center;margin:36px auto 18px;width:160px;line-height:0" id="hplogoo"> <svg aria-label="Google" height="56" role="img" viewBox="0 0 160 56" width="160" xmlns="http://www.w3.org/2000/svg">  <path d="M66.72 28.09c0 7.3-5.71 12.67-12.72 12.67s-12.72-5.38-12.72-12.67c0-7.34 5.71-12.67 12.72-12.67s12.72 5.33 12.72 12.67zm-5.57 0c0-4.56-3.31-7.68-7.15-7.68-3.84 0-7.15 3.12-7.15 7.68 0 4.51 3.31 7.68 7.15 7.68 3.84.01 7.15-3.16 7.15-7.68z" fill="#EA4335"></path> <path d="M94.72 28.09c0 7.3-5.71 12.67-12.72 12.67s-12.72-5.38-12.72-12.67c0-7.34 5.71-12.67 12.72-12.67s12.72 5.33 12.72 12.67zm-5.57 0c0-4.56-3.31-7.68-7.15-7.68-3.84 0-7.15 3.12-7.15 7.68 0 4.51 3.31 7.68 7.15 7.68 3.84.01 7.15-3.16 7.15-7.68z" fill="#FBBC05"></path> <path d="M121.55 16.19v22.75c0 9.36-5.52 13.2-12.05 13.2-6.14 0-9.84-4.13-11.23-7.49l4.85-2.02c.86 2.06 2.98 4.51 6.38 4.51 4.18 0 6.77-2.59 6.77-7.44v-1.82h-.19c-1.25 1.54-3.65 2.88-6.67 2.88-6.34 0-12.15-5.52-12.15-12.62 0-7.15 5.81-12.72 12.15-12.72 3.02 0 5.42 1.34 6.67 2.83h.19v-2.06h5.28zm-4.9 11.95c0-4.46-2.98-7.73-6.77-7.73-3.84 0-7.06 3.26-7.06 7.73 0 4.42 3.22 7.63 7.06 7.63 3.8.01 6.77-3.21 6.77-7.63z" fill="#4285F4"></path> <path d="M131.5 3v37H126V3h5.5z" fill="#34A853"></path> <path d="M153.22 32.27l4.32 2.88c-1.39 2.06-4.75 5.62-10.56 5.62-7.2 0-12.58-5.57-12.58-12.67 0-7.54 5.42-12.67 11.95-12.67 6.58 0 9.79 5.23 10.85 8.06l.58 1.44-16.95 7.01c1.3 2.54 3.31 3.84 6.14 3.84s4.81-1.4 6.25-3.51zm-13.29-4.56l11.33-4.7c-.62-1.58-2.5-2.69-4.7-2.69-2.84 0-6.78 2.49-6.63 7.39z" fill="#EA4335"></path> <path d="M38.47 19.24c.19 1.18.27 2.34.27 3.43 0 4.03-1.1 9.02-4.66 12.58-3.46 3.6-7.87 5.52-13.73 5.52C9.5 40.77.38 31.94.38 21.09S9.51 1.4 20.35 1.4c6 0 10.27 2.35 13.49 5.42l-3.79 3.79c-2.3-2.16-5.42-3.84-9.7-3.84-7.92 0-14.11 6.38-14.11 14.31 0 7.92 6.19 14.31 14.11 14.31 5.14 0 8.06-2.06 9.94-3.94 1.54-1.54 2.57-3.81 2.96-6.84h-12.9v-5.38l18.12.01z" fill="#4285F4"></path>   </svg>                </div>
            <div id="main">
                <style>
                    .fAc1Ee{position:absolute;left:-1000px}
                </style>
                <div>
                    <div>
                        <dialog class="spch-dlg" id="spch-dlg">
                            <div class="spch" id="spch" style="display:none"></div>
                        </dialog>
                    </div>
                </div>
                <style>
                    .Gwkg9c{margin:8px}
                </style>
                <div class="Gwkg9c">
                    <style>
                        .tsf{margin-left:auto;margin-right:auto;max-width:736px;overflow:hidden;width:auto;}.smCvtd{font-size:16px;color:#c5221f;background-color:var(--xhUGwc)}
                    </style>
                    <form class="tsf" action="/search" id="tsf" data-submitfalse="q" method="GET" name="gs" role="search">
                        <div jsmodel="b5W85 vNzKHd" jsdata="MuIEvd;_;v8HjaMKuBpCvptQP9sqQyAs1">
                            <div jscontroller="v6cJEc" jsmodel="VYkzu EPRt9d Qlyryb kjkykd LM7wx EtCx8b Ip3Erc Of1rIc L97mud        " class="A7Yvie" jsdata="LVplcb;_;" jsaction="DkpM0b:d3sQLd;IQOavd:dFyQEf;Aghsf:AVsnlb;R3Yrj:DURTdb;iHd9U:Q7Cnrc;f5hEHe:G0jgYd;vmxUb:j3bJnb;NOg9L:HLgh3;uGoIkd:epUokb;aIxJGc:TXyPN;acb7qb:H8sTz;kvwJje:s0yXd;lX6RWd:w3Wsmc;ocDSvd:Q83nl;XmGRxb:mVw6nb;rcuQ6b:npT2md"><span jsname="wn2Sie" role="none" tabindex="-1" jsaction="focus:O01aAd"></span>
                                <style>
                                    .A7Yvie{background:none;}.zGVn2e{display:flex;border-radius:25px;height:48px;margin-left:8px;margin-right:8px;border:1px solid #dadce0;background:var(--xhUGwc);}.Sl6fgd.A7Yvie{position:fixed;width:100%;height:100%;top:0;left:0;background:#fff;z-index:989;border-radius:0;overflow:auto;}.Sl6fgd .zGVn2e{border:none;margin-left:0;margin-right:0;height:auto}.Sl6fgd .A7Yvie{border:none}.noscroll{overflow:hidden}.qbOKL{position:fixed;overscroll-behavior:contain}.emcav .zGVn2e{border-bottom:none;border-bottom-right-radius:0;border-bottom-left-radius:0}
                                </style>
                                <div class="zGVn2e"><button jscontroller="jbu21b" jsname="Cdl0yb" class="Cdl0yb" style="display:none" aria-label="Back" tabindex="0" type="button" jsaction="a4fUwd"><style>.Cdl0yb{outline:none;background:none;border:none;margin-left:6px;padding:0 12px;flex:0 0 auto;height:48px}.Cdl0yb:focus,.Cdl0yb:active{outline:revert;outline-offset:-1px;z-index:1}.Z7Bonf{color:#1a73e8;height:24px}</style><div jsname="Z7Bonf" class="Z7Bonf"><span class="z1asCe IW3j9"><svg focusable="false" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M20 11H7.83l5.59-5.59L12 4l-8 8 8 8 1.41-1.41L7.83 13H20v-2z"></path></svg></span></div></button>
                                    <button
                                        class="Tg7LZd search_button_suggest" aria-label="Google Search" type="submit">
                                        <style>
                                            .Tg7LZd{background:transparent;border:none;padding:0 12px}.Sl6fgd .search_button_suggest{display:none}html:not(.zAoYTe) .Tg7LZd:focus{outline:0}.gBCQ5d{background:none;color:#9aa0a6;height:24px;width:24px;margin:auto}
                                        </style>
                                        <div class="gBCQ5d"><span class="z1asCe MZy1Rb"><svg focusable="false" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M15.5 14h-.79l-.28-.27A6.471 6.471 0 0 0 16 9.5 6.5 6.5 0 1 0 9.5 16c1.61 0 3.09-.59 4.23-1.57l.27.28v.79l5 4.99L20.49 19l-4.99-5zm-6 0C7.01 14 5 11.99 5 9.5S7.01 5 9.5 5 14 7.01 14 9.5 11.99 14 9.5 14z"></path></svg></span></div>
                                        </button>
                                        <div jscontroller="V6oqv" jsname="gLFyf" class="a4bIc" aria-expanded="false" aria-haspopup="listbox" role="combobox" jsaction="focus:Jt1EX;input:Lg5SV;paste:V64iSc;touchstart:ADZYmd">
                                            <style>
                                                .a4bIc{display:grid;flex:1;}.gLFyf,.HeEoJc,.u4frDf{line-height:24px;background:none;margin:0;padding:0;border-color:transparent;border-style:solid;border-width:12px 0;font-size:16px;font-family:Google Sans,Roboto,sans-serif;color:#202124;word-wrap:break-word;outline:none;display:flex;flex:1;-webkit-tap-highlight-color:transparent;width:100%}.Sl6fgd .gLFyf,.Sl6fgd .HeEoJc{line-height:24px;overflow-y:hidden}.Sl6fgd textarea.gLFyf,.Sl6fgd .HeEoJc{overflow-y:auto;white-space:normal}.Sl6fgd .HeEoJc{overflow-wrap:anywhere;white-space:pre-wrap}.Sl6fgd .gLFyf,.Sl6fgd .HeEoJc,.Sl6fgd .u4frDf{border-width:12px 0;border-style:solid;border-color:transparent}textarea.gLFyf,.HeEoJc,.u4frDf{max-height:144px;white-space:nowrap;resize:none;overflow:hidden;grid-area:1/1/2/2;box-sizing:border-box}.HeEoJc,.u4frDf{visibility:hidden}.gLFyf::placeholder{color:var(--IXoxUe)}
                                            </style>
                                            <div jsname="nzCU8b" class="HeEoJc"></div><textarea jsname="yZiJbe" class="gLFyf" aria-describedby="sb_input_box_aria_description_v8HjaMKuBpCvptQP9sqQyAs_1" title="" value="" placeholder="" aria-label="Google Search" aria-autocomplete="list" autocapitalize="off"
                                                autocomplete="off" autocorrect="off" enterkeyhint="search" maxlength="2048" name="q" role="textbox" rows="1" spellcheck="false" tabindex="0" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQ39UDCA8"></textarea>
                                            <div
                                                jsname="BMJpAe" class="u4frDf"></div>
                                        <div id="sb_input_box_aria_description_v8HjaMKuBpCvptQP9sqQyAs_1" hidden=""><span>Double-tap to search Google.</span></div>
                                </div>
                                <div jsname="hK5Dwd" class="dRYYxd" data-input-has-text="0" jsaction="Y2XUzc:Fs7Xzc">
                                    <style>
                                        .dRYYxd{display:flex;flex:0 0 auto;align-items:stretch;margin-right:0}.Sl6fgd .dRYYxd{margin-right:6px;height:48px}
                                    </style>
                                    <style>
                                        .RP0xob{display:flex;align-items:center;flex:1;color:var(--bbQxAb);cursor:pointer;padding:0 12px;margin:0;border:0;background:transparent}.Sl6fgd .RP0xob{color:#70757a}
                                    </style><button jscontroller="V73YV" jsname="RP0xob" class="RP0xob" id="tsuid_v8HjaMKuBpCvptQP9sqQyAs_2" tabindex="0" type="button" jsaction="AVsnlb" style="display:none"><span aria-label="Clear Search" class="z1asCe rzyADb"><svg focusable="false" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M19 6.41L17.59 5 12 10.59 6.41 5 5 6.41 10.59 12 5 17.59 6.41 19 12 13.41 17.59 19 19 17.59 13.41 12z"></path></svg></span></button>
                                    <style>
                                        .FJoqsb{flex:1;cursor:pointer;border:none;background:none;padding:0 12px}.FJoqsb:focus,.FJoqsb:active{outline-offset:-1px;z-index:1}.svprff{height:24px;width:24px;vertical-align:middle}
                                    </style><button jscontroller="D36Upd" jsname="Q7Ulpb" class="FJoqsb" data-iga="false" data-igafdl="" id="Q7Ulpb" tabindex="0" type="button" jsaction="cOuCgd;rcuQ6b:npT2md;KHxBOb:EANhx;hATt5e:AkD3se;CQJmec:efDZmf" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQvs8DCBA"><svg class="svprff" aria-label="Search by voice" role="img" focusable="false" viewbox="0 -960 960 960" xmlns="http://www.w3.org/2000/svg"><path fill="var(--bbQxAb)" d="M480-400q-50 0-85-35t-35-85v-240q0-50 35-85t85-35q50 0 85 35t35 85v240q0 50-35 85t-85 35Zm-40 280v-123q-104-14-172-93t-68-184h80q0 83 58.5 141.5T480-320q83 0 141.5-58.5T680-520h80q0 105-68 184t-172 93v123h-80Z"></path></svg></button>
                                    <style>
                                        .v9gBvd{display:flex;align-items:center;flex:1;cursor:pointer;border:0;background:transparent;padding:0 12px}.RcQUac{display:flex}html:not(.zAoYTe) .RcQUac:focus,.v9gBvd:focus{outline:0}.RcQUac:focus,.RcQUac:active,.v9gBvd:focus,.v9gBvd:active{outline-offset:-1px;z-index:1}
                                    </style>
                                    <div jscontroller="hywCzd" jsname="ormkb" class="RcQUac">
                                        <a class="v9gBvd" href="https://search.app.goo.gl/?ofl=https%3A%2F%2Flens.google&amp;al=googleapp%3A%2F%2Flens%3Flens_data%3DKAw&amp;apn=com.google.android.googlequicksearchbox&amp;amv=301204913&amp;isi=284815942&amp;ius=googleapp&amp;ibi=com.google.GoogleMobile&amp;link=https%3A%2F%2Fgoo.gl%2Fiosgoogleapp%2Fdefault%3Furl%3Dgoogleapp%253A%252F%252Flens%253Fmin-version%253D180%2526lens_data%253DKAw&amp;ifl=https%3A%2F%2Fapps.apple.com%2Fus%2Fapp%2Fgoogle%2Fid284815942%3Fppid%3D1ac8cc35-d99c-4a1d-b909-321c8968cc74%26pt%3D9008%26mt%3D8%26ct%3D4815459-oo-lens-isb-bar-lens-cam%26UTM_campaign%3Dgoogle_search_mweb&amp;efr=1&amp;ct=4815459-oo-lens-isb-bar-lens-cam&amp;utm_campaign=4815459-oo-lens-isb-bar-lens-cam&amp;utm_source=google_search_mweb&amp;utm_medium=owned&amp;pt=9008&amp;mt=8"
                                            aria-label="Search using your camera or photos" role="link" tabindex="0" data-jsarwt="1" data-usg="AOvVaw3lTdGuHCgQSsAbOkHGwVV8" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQhqEICBE">
                                            <style>
                                                .b4yCbb{height:24px;width:24px;vertical-align:middle}
                                            </style><svg class="b4yCbb" aria-hidden="true" role="img" focusable="false" viewbox="0 -960 960 960" xmlns="http://www.w3.org/2000/svg"><path fill="var(--bbQxAb)" d="M480-320q-50 0-85-35t-35-85q0-50 35-85t85-35q50 0 85 35t35 85q0 50-35 85t-85 35Zm240 160q-33 0-56.5-23.5T640-240q0-33 23.5-56.5T720-320q33 0 56.5 23.5T800-240q0 33-23.5 56.5T720-160Zm-440 40q-66 0-113-47t-47-113v-80h80v80q0 33 23.5 56.5T280-200h200v80H280Zm480-320v-160q0-33-23.5-56.5T680-680H280q-33 0-56.5 23.5T200-600v120h-80v-120q0-66 47-113t113-47h80l40-80h160l40 80h80q66 0 113 47t47 113v160h-80Z"></path></svg></a>
                                    </div>
                                </div>
                            </div>
                            <div jscontroller="JTdtMc" jsname="UUbT9" class="UUbT9 EyBRub" style="display:none" jsaction="ldyIye:CmVOgc;i58Tld:otTs8" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQ4tUDCBI">
                                <style>
                                    .UUbT9{background:#fff;position:relative;text-align:left;border-radius:0 0 20px 20px;border:1px solid #dadce0;border-radius:0;z-index:0;overflow-x:hidden;cursor:default;font-family:Google Sans,Roboto,sans-serif;}.UUbT9 .aajZCb span b{font-family:Google Sans,Roboto-Medium,sans-serif;font-weight:700}.aajZCb{padding:0;margin:0}.aajZCb ul{list-style:none}.Sl6fgd .UUbT9{border:none;margin:0 6px 0 10px;border-top:1px solid #dadce0}.sbsc li:not(:last-child){border-bottom:1px solid #e5e5e5}.sb7{background:url(/images/searchbox/searchbox_sprites317_hr.webp) no-repeat 0 -50px;background-size:24px;height:24px;width:24px}.sb27{background:url(/images/searchbox/searchbox_sprites317_hr.webp) no-repeat 0 -25px;background-size:24px;height:24px;width:24px}.sb43{background:url(/images/searchbox/searchbox_sprites317_hr.webp) no-repeat 0 0;background-size:24px;height:24px;width:24px}.sbqb{background:url(/images/searchbox/searchbox_sprites317_hr.webp) no-repeat 0 -75px;background-size:24px;height:24px;width:24px;align-self:center;flex:1}html[dir='rtl'] .sbqb{transform:scaleX(-1)}.sb33{background:url(//www.gstatic.com/images/icons/material/system/2x/trending_up_grey600_24dp.png) no-repeat;background-size:24px;height:24px;width:24px}.psPFyd{background-size:contain;height:36px;width:36px;border-radius:4px;background-repeat:no-repeat;background-position:center;margin-left:6px}
                                </style>
                                <div jsname="aajZCb" class="aajZCb" role="presentation"></div>
                                <style>
                                    .ftihzf{padding:0;margin:0}.CKtgbd{display:flex;flex-direction:column;min-width:0;padding:0;margin:0}.lPUhoe{flex-direction:row;flex-wrap:wrap}
                                </style>
                                <div jsname="E80e9e" class="ftihzf" role="presentation">
                                    <ul jsname="erkvQe" class="CKtgbd" role="listbox"></ul>
                                </div>
                                <style>
                                    #ynRric{display:none}.ynRric{display:flex;justify-content:center;flex-direction:column;padding-left:16px;color:var(--IXoxUe);font-family:Roboto,sans-serif;font-weight:500;font-size:14px;letter-spacing:0.8px;height:20px;width:100%;margin:8px 0 4px}
                                </style>
                                <div class="ynRric" id="ynRric" role="presentation"></div>
                                <style>
                                    #ymEu3b{display:none}.sbct{padding:0;display:flex;align-items:center;min-width:0;}.bRY3od{display:flex;flex:auto;align-items:center;min-width:0}.sbsc .sbct{padding:10px 3px 11px 10px;}.sbic{display:flex;align-items:center;margin:12px;margin-left:8px;}.GPcFIc{display:flex;flex:1;flex-direction:column;min-width:0;margin:5px 0 5px 0}.s49v2{display:flex;font-size:16px;line-height:22px;flex:1;align-items:center;word-break:break-word}.s49v2 span,.mus_il:first-child span{flex:1;font-weight:normal;color:#202124}.s49v2 span b,.mus_il:first-child span b{font-weight:bold;color:#202124}.cuyMBe{display:flex;font-size:12px;color:#767676;flex:1;align-items:center}.cuyMBe span.qC6o5d{flex:1;overflow:hidden;text-overflow:ellipsis}.ncRnnb{display:-webkit-box;-webkit-box-orient:vertical;-webkit-line-clamp:2}.rC1X7c{white-space:nowrap}.aZjrb{display:flex;align-self:stretch;padding:0 5px 0 11px;outline-offset:-1px;padding:0 12px}.FbzDTd{color:#70757a;font:27px/25px arial,sans-serif;padding:0 5px;cursor:pointer;align-self:center}.mus_pc{display:block}.mus_il{font-family:sans-serif;padding-top:7px;position:relative}.mus_il:first-child{padding-top:0}.mus_il_at{margin-left:10px}.mus_il_i{align:left;margin-right:10px}.mus_it3{margin-bottom:3px;max-height:24px;vertical-align:bottom}span.mus_tt3,div.mus_tt3,.mus_tt3{color:#767676 !important;font-size:12px !important;vertical-align:top !important}span.mus_tt5,div.mus_tt5,.mus_tt5{color:#d93025 !important;font-size:14px !important}span.mus_tt6,div.mus_tt6,.mus_tt6{color:#188038;font-size:14px !important}span.mus_tt8,div.mus_tt8,.mus_tt8{color:#000;font-size:16px !important;font-family:sans-serif !important}span.mus_tt17,div.mus_tt17,.mus_tt17{color:var(--YLNNHc) !important;font-size:20px !important}span.mus_tt18,div.mus_tt18,.mus_tt18{color:var(--YLNNHc);font-size:24px}span.mus_tt19,div.mus_tt19,.mus_tt19{color:var(--IXoxUe) !important;font-size:12px !important}span.mus_tt20,div.mus_tt20,.mus_tt20{color:var(--IXoxUe) !important;display:block;font-size:14px !important;margin-top:-7px !important}span.mus_il_at.mus_tt20{display:inline-block}span.mus_tt23,div.mus_tt23,.mus_tt23{color:var(--IXoxUe) !important;font-size:18px !important}.mus_tt3{color:#767676;font-size:12px;vertical-align:top}.mus_tt5{color:#d93025;font-size:14px}.mus_tt6{color:#188038;font-size:14px}.mus_tt8{color:#000;font-size:16px;font-family:sans-serif}.mus_tt17{color:var(--YLNNHc);font-size:20px}.mus_tt18{color:var(--YLNNHc);font-size:24px}.mus_tt19{color:var(--IXoxUe);font-size:12px}.mus_tt20{color:var(--IXoxUe);font-size:14px}.mus_tt23{color:var(--IXoxUe);font-size:18px}
                                </style>
                                <style>
                                    .globMc{background:var(--TMYS9);border-radius:50%;margin-right:6px;width:6px;height:6px}
                                </style>
                                <li class="sbct" data-view-type="1" jsname="IrQt0b" id="ymEu3b" role="presentation">
                                    <div class="bRY3od" role="option" tabindex="0">
                                        <div class="sbic"></div>
                                        <div class="GPcFIc">
                                            <div class="s49v2"><span></span></div>
                                            <div class="cuyMBe">
                                                <div class="globMc"></div><span class="qC6o5d"></span></div>
                                        </div>
                                        <div class="psPFyd"></div>
                                    </div>
                                    <div class="aZjrb" role="button" tabindex="0">
                                        <div class="sbai"></div>
                                    </div>
                                </li>
                                <style>
                                    #rdyYgb{display:none}.pqVAOb{border-radius:8px;display:inline-block;max-width:223px;width:28%;margin-left:2%;margin-right:2%;height:fit-content}.ywGxoc{padding:3px;width:100%;border-radius:8px;display:flex;flex-direction:column;margin-bottom:10px;margin-top:10px}.ywGxoc .GPcFIc{font-family:Google Sans,Roboto,sans-serif;margin-top:4px;max-height:60px;overflow:hidden;padding:0}.ywGxoc .GPcFIc .s49v2{font-size:11px;line-height:13px;margin-top:2px;font-family:Google Sans,Roboto,sans-serif;color:var(--IXoxUe)}.ywGxoc .GPcFIc .cuyMBe{font-size:11px;color:var(--YLNNHc);line-height:13px;font-weight:bold;margin-top:2px;margin-left:2px;display:flex}.ywGxoc .GPcFIc .s49v2 span,.ywGxoc .GPcFIc .cuyMBe span{overflow:hidden;text-overflow:ellipsis;white-space:nowrap}.ywGxoc .GPcFIc .s49v2 span{-webkit-box-orient:vertical;display:-webkit-box;-webkit-line-clamp:2;white-space:normal}.ywGxoc .GPcFIc .cuyMBe span{-webkit-box-orient:vertical;display:-webkit-box;-webkit-line-clamp:1;white-space:normal}.HsHf0c{background:#fff;border-radius:8px;width:100%}.HsHf0c .sbic{border-radius:8px;height:auto;margin:0;width:100%;padding-top:100%;background:center/contain no-repeat}
                                </style>
                                <li class="pqVAOb" data-view-type="6" id="rdyYgb" role="presentation">
                                    <div class="ywGxoc">
                                        <div class="HsHf0c">
                                            <div class="sbic"></div>
                                        </div>
                                        <div class="GPcFIc" role="option">
                                            <div class="s49v2"><span></span></div>
                                            <div class="cuyMBe"><span class="qC6o5d"></span></div>
                                        </div>
                                    </div>
                                </li>
                                <li class="X6FF1e" data-view-type="9" id="GVRszc" role="option" tabindex="0">
                                    <style>
                                        .fP1Wtb{background:#f1f3f4;border-radius:20px}.jhj0w{overflow-wrap:break-word;max-width:calc(100% - 2em)}.raMrp{color:var(--YLNNHc);display:-webkit-box;font-family:Google Sans,Roboto-Medium,sans-serif;font-size:32px;font-weight:400;line-height:40px;overflow:hidden;-webkit-line-clamp:2;-webkit-box-orient:vertical}.jDGOmc{font-size:24px}.KNLwef,.OOm3Jf{color:#5f6368;font-size:14px;font-weight:400;line-height:20px}.Id6Uac{display:inline-grid;color:var(--YLNNHc);font-size:16px;font-weight:400;line-height:22px}.bH5xgc{color:#5f6368;font-size:14px;font-weight:400;line-height:20px;margin-top:12px}.Cqbdne{border:none;border-radius:9999px;position:absolute;right:0;top:0;height:38px;padding:0;width:38px}.sNuPJc{background-color:var(--xhUGwc);border-radius:9999px;border:1px solid var(--mXZkqc);color:var(--amnPwe);cursor:pointer;height:24px;padding:6px;width:24px}.b23yLe{background:top right/contain no-repeat var(--xhUGwc);border-radius:12px;flex-shrink:0;height:64px;margin-left:12px;width:64px}.zHwC9b{font-style:italic}.RTjHVd{color:#146c2e}.qPHNse{color:#b3261e}#GVRszc{display:none}.X6FF1e{background:#f1f3f4;border-radius:12px 16px;display:flex}.DRCFcc{margin:12px 16px;width:100%;min-width:0}.TzqGLb{display:flex;justify-content:space-between}
                                    </style>
                                    <div class="DRCFcc" style="display:none" id="qwtsbf">
                                        <div class="TzqGLb">
                                            <div class="jhj0w">
                                                <div class="Gs0LAc" aria-atomic="true" role="option">
                                                    <div class="raMrp"></div>
                                                    <div class="KNLwef"></div>
                                                    <div class="Id6Uac"></div>
                                                    <div class="OOm3Jf"></div>
                                                </div><button class="Cqbdne" aria-atomic="true" type="button" style="display:none"><span class="sNuPJc z1asCe JKu1je"><svg focusable="false" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M3 9v6h4l5 5V4L7 9H3zm13.5 3c0-1.77-1.02-3.29-2.5-4.03v8.05c1.48-.73 2.5-2.25 2.5-4.02zM14 3.23v2.06c2.89.86 5 3.54 5 6.71s-2.11 5.85-5 6.71v2.06c4.01-.91 7-4.49 7-8.77s-2.99-7.86-7-8.77z"></path></svg></span></button></div>
                                            <div
                                                class="b23yLe" style="display:none"></div>
                                    </div>
                                    <div class="bH5xgc" style="display:none"></div>
                            </div>
                            </li>
                            <li class="gFhuhf" data-view-type="10" id="RyXi3" role="option" tabindex="0">
                                <style>
                                    #RyXi3{display:none}.gFhuhf{background:#f1f3f4;border-radius:12px 16px;display:flex}.hUlXyc{margin:12px 16px;width:100%}.yK3ynb{display:flex;flex-direction:column;position:relative}.isUbqc{display:flex;justify-content:space-between;padding-bottom:12px}.YNUCnf{display:flex;justify-content:space-between;border-top:1px solid var(--Aqn7xd);padding-top:12px}.YNUCnf .b23yLe{top:12px}
                                </style>
                                <div class="hUlXyc" style="display:none" id="RkIkbe">
                                    <div class="yK3ynb">
                                        <div class="isUbqc">
                                            <div class="Gs0LAc" aria-atomic="true" role="option">
                                                <div class="raMrp"></div>
                                                <div class="KNLwef"></div>
                                                <div class="Id6Uac"></div>
                                                <div class="OOm3Jf"></div>
                                            </div><button class="Cqbdne" aria-atomic="true" type="button" style="display:none"><span class="sNuPJc z1asCe JKu1je"><svg focusable="false" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M3 9v6h4l5 5V4L7 9H3zm13.5 3c0-1.77-1.02-3.29-2.5-4.03v8.05c1.48-.73 2.5-2.25 2.5-4.02zM14 3.23v2.06c2.89.86 5 3.54 5 6.71s-2.11 5.85-5 6.71v2.06c4.01-.91 7-4.49 7-8.77s-2.99-7.86-7-8.77z"></path></svg></span></button>
                                            <div
                                                class="b23yLe" style="display:none"></div>
                                    </div>
                                    <div class="YNUCnf">
                                        <div class="Gs0LAc" aria-atomic="true" role="option">
                                            <div class="raMrp"></div>
                                            <div class="KNLwef"></div>
                                            <div class="Id6Uac"></div>
                                            <div class="OOm3Jf"></div>
                                        </div><button class="Cqbdne" aria-atomic="true" type="button" style="display:none"><span class="sNuPJc z1asCe JKu1je"><svg focusable="false" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M3 9v6h4l5 5V4L7 9H3zm13.5 3c0-1.77-1.02-3.29-2.5-4.03v8.05c1.48-.73 2.5-2.25 2.5-4.02zM14 3.23v2.06c2.89.86 5 3.54 5 6.71s-2.11 5.85-5 6.71v2.06c4.01-.91 7-4.49 7-8.77s-2.99-7.86-7-8.77z"></path></svg></span></button>
                                        <div
                                            class="b23yLe" style="display:none"></div>
                                </div>
                        </div>
                        <div class="bH5xgc" style="display:none"></div>
                </div>
                </li>
                <div jsname="Sx9Kwc" jscontroller="jlvtuf" data-is-ios-gsa="false" jsaction="WlwJwc:FNFY6c">
                    <style>
                        .dgv .A8Ithf{background-color:#000;opacity:0.2;visibility:inherit}.dgv .MZbroc{background-color:#000;opacity:0.4;visibility:inherit}.dgv .Osn2G{background-color:#1f1f1f;opacity:0.7;visibility:inherit}.dgv .cfMOfb{background-color:var(--aYn2S);opacity:0.85;visibility:inherit}.dgv .o4qyNe,.dgv .qp8xxd{opacity:1;visibility:inherit}.n2wEib{position:fixed;z-index:1001;right:0;bottom:-200px;top:0;left:0;-webkit-transition:opacity 0.25s;opacity:0;visibility:hidden}.qp8xxd{border-radius:8px;position:relative;display:inline-block;z-index:1060;background-color:var(--xhUGwc);opacity:0;vertical-align:middle;white-space:normal;overflow:hidden;transform:translateZ(0);box-shadow:0px 5px 26px 0px rgba(0,0,0,.22),0px 20px 28px 0px rgba(0,0,0,.30)}.qp8xxd:focus{outline:none}.o4qyNe{position:fixed;right:0;bottom:0;top:0;left:0;z-index:1002;vertical-align:middle;visibility:hidden;white-space:nowrap;max-height:100%;max-width:100%;overflow:auto;transform:translateZ(0);-webkit-tap-highlight-color:transparent}.o4qyNe::after{content:'';display:inline-block;height:100%;vertical-align:middle}.nsc{box-sizing:border-box;overflow:hidden;position:fixed;width:100%}
                    </style>
                    <div class="n2wEib MZbroc"></div>
                    <div class="o4qyNe VDgVie">
                        <div jsname="bN97Pc" class="qp8xxd TUOsUe" role="dialog" tabindex="-1" jsaction="htU0tb:TvD9Pc">
                            <style>
                                .fp-f{bottom:0;height:auto;left:0;position:fixed !important;right:0;top:0;width:auto;z-index:127}.fp-h:not(.fp-nh){display:none !important}.fp-i .fp-c{display:block;min-height:100vh}li.fp-c{list-style:none}.fp-w{box-sizing:border-box;left:0;margin-left:auto;margin-right:auto;max-width:1181px;right:0}.Ndwy3e{max-width:300px;-webkit-user-select:none}.HGU8Ic{padding:24px;font-size:16px;overflow-wrap:break-word;color:var(--bbQxAb)}.HGU8Ic a{text-decoration:none;color:#4285f4}.EMErBb{display:flex;justify-content:flex-end;padding-bottom:8px}.KFwZGf{flex:0 0 auto;margin-right:8px;min-width:48px;padding:0 8px;line-height:36px !important;text-align:center;font-family:Roboto-Medium,sans-serif !important}
                            </style>
                            <div jscontroller="Sv8aof" class="Ndwy3e" jsdata="vST7rb;_;v8HjaMKuBpCvptQP9sqQyAs2" jsaction="TWGMlf:Z7TNWb;rcuQ6b:npT2md">
                                <div jsname="jVIZXb" class="HGU8Ic"></div>
                                <div class="EMErBb">
                                    <div>
                                        <div jsname="iOeE6e" class="KFwZGf Va3FIb TElO2c OSrXXb" style="display:none;color:#4285f4" role="button" jsaction="sUXdZe" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQtw8IEw">
                                            <style>
                                                .Va3FIb{font-size:14px;font-family:Roboto,sans-serif;line-height:20px}
                                            </style>
                                            <style>
                                                .Va3FIb{}
                                            </style>
                                            <style>
                                                .TElO2c{white-space:nowrap}
                                            </style>REPORT THIS</div>
                                    </div>
                                    <div jsname="jWDcpe" class="KFwZGf Va3FIb TElO2c OSrXXb" style="display:none;color:#4285f4" role="button" jsaction="r9DEDb">CANCEL</div>
                                    <div jsname="kcdgPe" class="KFwZGf Va3FIb TElO2c OSrXXb" style="display:none;color:#4285f4" role="button" jsaction="r9DEDb">OK</div>
                                    <div jsname="AFDdpc" class="KFwZGf Va3FIb TElO2c OSrXXb" style="display:none;color:#ea4335" role="button">DELETE</div>
                                </div>
                            </div>
                        </div>
                    </div>
                </div>
                <div jscontroller="Wo3n8" jsname="aadvhe" jsmodel=" d5EhJe" data-bkt="searchbox" data-fhs="" data-maindata='[null,null,null,"autocomplete_user_feedback_kp_id",null,11,null,null,null,null,null,5010715,"searchbox",null,"AutocompletePrediction",null,null,null,null,11]'
                    data-ofbl="114,120,134" data-pid="5010715" jsdata="vST7rb;_;v8HjaMKuBpCvptQP9sqQyAs2 zEIyGd;_;" jsaction="kPzEO:MlP2je;qjLxRc:FbhRG;w8f1fc:hRwSgb;kq2wxf:s5CUif;aIJAdf:UhDUnd;rwuG3b:KGQHJd;BqbTbe:naa5ve;kYAKrf:CqUGrf;hwhRRe:KyxjCd;rcuQ6b:npT2md">
                    <div id="_v8HjaMKuBpCvptQP9sqQyAs_4">
                        <style>
                            .z8gr9e{color:var(--bbQxAb)}
                        </style>
                        <style>
                            .i1eWpb .GTERze{display:none}.ky4hfd{display:none}.i1eWpb .ky4hfd{display:block}
                        </style>
                        <div jsname="GkjeIf" id="_v8HjaMKuBpCvptQP9sqQyAs_6" data-jiis="up" data-async-type="kp_feedback" class="yp" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQ68cECBQ"></div>
                    </div>
                </div>
            </div>
            <div jscontroller="kbpCEc" class="yx31rd opYLNd" jsaction="gVSUcb:kHt5yc;rcuQ6b:npT2md">
                <style>
                    .yx31rd{position:fixed;bottom:-48px;height:48px;width:100%;display:flex;background:rgba(50,50,50,.85);z-index:1000;overflow:hidden;transition:.4s ease-in-out}.QGtrY{color:#fff;padding:0 24px;align-self:center}.sWAGC{transform:translateY(-100%)}.opYLNd{transform:translateY(0);visibility:hidden}
                </style>
                <div jsname="r80sCc" class="QGtrY"></div>
            </div><span jsname="QDon1c" role="none" tabindex="-1" jsaction="focus:O01aAd"></span></div>
        <div style="background:url(/images/searchbox/searchbox_sprites317_hr.webp)"></div>
    </div>
    <div id="tophf"><input name="sca_esv" value="bbe3debe957613b7" type="hidden"><input name="source" type="hidden" value="hp"><input value="v8HjaMKuBpCvptQP9sqQyAs" name="ei" type="hidden"></div>
    </form>
    </div>
    <div id="belowsb">
        <div>
            <div jscontroller="ms4mZb" jsname="FAeNCb" data-aipm="1" data-eaip="true" data-ecom="true" data-endpoint="2" data-oaipm="0" id="_v8HjaMKuBpCvptQP9sqQyAs_8" data-jiis="up" data-async-type="hpba" class="yp" jsaction="rcuQ6b:npT2md" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQj-0KCBU"></div>
            <script nonce="IJ8UXppd3ETKm0PGR2OelA">
                (function() {
                    function d(a, b) {
                        if (a.wfpe && document.prerendering) document.addEventListener("prerenderingchange", () => {
                            b(a)
                        }, {
                            once: !0,
                            passive: !0
                        });
                        else if (a.wfve && document.hidden) {
                            function c() {
                                document.hidden || (b(a), document.removeEventListener("visibilitychange", c))
                            }
                            document.addEventListener("visibilitychange", c, {
                                passive: !0
                            })
                        } else b(a)
                    }

                    function e(a) {
                        a.ets = {};
                        var b = document.getElementById(a.id);
                        if (b = (a.t = b) && b.dataset.ved) a.url += `&vet=${`1${b}..i`}`;
                        if (b = (b = document.getElementById(a.trid)) && b.dataset.ved) a.url += `&ved=${b}`;
                        for (const c of google.ia.rf) c && c(a);
                        google.ia.r[a.id] = a;
                        google.ia.mi[a.m](a)
                    }
                    google.ia || (google.ia = {
                        q: e,
                        r: {},
                        mi: [],
                        rf: [],
                        was: d,
                        dq: [],
                        pf: [],
                        rids: new Set
                    }, google.iax && google.iax());
                    google.dp = !0;
                }).call(this);
                (function() {
                    google.cr = google.cr || new Set;
                }).call(this);
                (function() {
                    var rids = false;
                    (function() {
                        var l = {
                            push: a => void a()
                        };

                        function m(a, b) {
                            if (rids) {
                                b && a.id && google.ia.rids.add(a.id);
                                for (const c of a.children) m(c, !0)
                            }
                        };

                        function n() {
                            if (typeof google !== "undefined" && google && google.rdn) {
                                a: {
                                    if (window.performance && window.performance.getEntriesByType) {
                                        var a = window.performance.getEntriesByType("navigation");
                                        if (a.length !== 0) {
                                            a = a[0];
                                            break a
                                        }
                                    }
                                    a = void 0
                                }
                                return a ? a.type : null
                            }
                            return window.performance && window.performance.navigation && window.performance.navigation.type
                        };
                        const p = new Set("aaft aafit aafct irli sart scrt".split(" "));
                        var q = function(a) {
                                if (Array.isArray(a.g.cbvi))
                                    for (const b of a.g.cbvi) b();
                                a.g.cbvi = l
                            },
                            r = function(a, b = "all") {
                                const c = b === "all";
                                q(a);
                                var d, e;
                                let f;
                                const g = ((d = google.stvsc) == null ? void 0 : d.ns) || ((e = window.performance) == null ? void 0 : (f = e.timing) == null ? void 0 : f.navigationStart);
                                g && (a.i.ns = g);
                                a: {
                                    d = window.location.search;
                                    if (e = window._csc !== "agsa" || !window._cshid) typeof google !== "undefined" && google && google.rdn ? (e = n(), e = !e || e === "direct") : e = !n();
                                    if (e && (d = (d = d.match(RegExp("[?&]qsubts=(\\d+)"))) && Number(d[1]), d > 0)) break a;d = void 0
                                }
                                d && (a.i.gsans = d);
                                d = a.g.t.getAttribute("eid");
                                e = [];
                                for (const h of Object.keys(a.i)) !c && p.has(h) || e.push(`${h}.${a.i[h]-a.M}`);
                                for (const h of Object.keys(a.j)) !c && p.has(h.split("-")[0]) || e.push(`${h}.${a.j[h]-a.M}`);
                                b = `/gen_204?s=async&astyp=${a.g.astyp}&t=${b}&atyp=csi&ei=${d}&rt=${e.join(",")}`;
                                c && (a.l && (b += `&twt=${Math.round(a.l)}&mwt=${Math.round(a.D)}`), a.C && (b += `&lvhr=${a.C}`), b += `&${"imn"}=${a.H}`, b += `&${"ima"}=${a.I}`);
                                b += `&${"folid"}=${a.g.id}`;
                                google.log("", "", b)
                            },
                            t =
                            function(a) {
                                a.H !== a.B || a.J || (a.i.art = Date.now(), r(a))
                            },
                            u = function(a) {
                                a.I === a.N && q(a)
                            },
                            w = function(a, b, c, d) {
                                for (const e of a.g.t.querySelectorAll("[data-subtree]"))
                                    for (const f of v(e)) {
                                        if (!f) continue;
                                        const g = `${"saft"}-${f}`,
                                            h = `${"sart"}-${f}`;
                                        if (e.contains(b) || b.contains(e)) d && (a.j[g] = c), a.j[h] = c
                                    }
                            },
                            x = function(a, b, c, d) {
                                const e = google.cv(b, !1, a.g.t),
                                    f = !!(e & 1);
                                d = d || Date.now();
                                f && (c && (a.C = c), a.i.aaft = d, a.i.aafct = d);
                                w(a, b, d, f);
                                e & 4 && (a.O = !0, u(a))
                            },
                            y = function(a, b, c, d = !1) {
                                b.getAttribute("data-deferred") ===
                                    "1" ? google.rll(b, !0, () => {
                                        y(a, b, c, d)
                                    }) : d || (b.getAttribute("data-deferred") === "2" && b.setAttribute("data-deferred", "3"), d = !0, z(a, b, c))
                            },
                            A = function(a, b, c) {
                                const d = Date.now(),
                                    e = b.getElementsByTagName("img");
                                for (let f = 0; f < e.length; f++) {
                                    const g = e[f];
                                    ++a.H;
                                    const h = g.complete && !g.getAttribute("data-deferred"),
                                        k = g.hasAttribute("data-noaft");
                                    if (h || k) {
                                        ++a.B;
                                        continue
                                    }
                                    const E = !!(google.cv(g, !1, a.g.t) & 1);
                                    if (E) ++a.I;
                                    else if (g.loading === "lazy") {
                                        ++a.B;
                                        continue
                                    }
                                    google.rll(g, !0, () => {
                                        y(a, g, E)
                                    });
                                    a.i.irfi || (a.i.irfi = d);
                                    a.i.irli = d
                                }
                                x(a, b, c, d)
                            },
                            z = function(a, b, c) {
                                const d = Date.now();
                                a.i.irfie || (a.i.irfie = d);
                                ++a.B;
                                c && (++a.N, a.i.aaft = d, a.i.aafit = d);
                                w(a, b, d, c);
                                a.O && u(a);
                                a.T && t(a)
                            },
                            v = function(a) {
                                return (a = a.dataset.subtree) ? a.split(",") : []
                            },
                            B = function(a, b, c) {
                                for (const d of a.g.t.querySelectorAll("[data-subtree]"))
                                    for (const e of v(d)) {
                                        if (!e) continue;
                                        const f = `${"sirt"}-${e}`,
                                            g = `${"sart"}-${e}`,
                                            h = `${"scrt"}-${e}`;
                                        let k = !1;
                                        b.contains(d) ? (a.j[f] || (k = !0, a.j[f] = c), a.j[g] = c, a.j[h] = c) : d.contains(b) && (a.j[g] = c, a.j[h] = c);
                                        e === "mfl" &&
                                            k && r(a, "sirt-mfl")
                                    }
                            },
                            C = class {
                                constructor(a) {
                                    var b = Date.now();
                                    this.g = a;
                                    this.M = b;
                                    this.j = {};
                                    this.i = {};
                                    this.N = this.I = this.l = this.B = this.H = this.G = this.D = this.C = 0;
                                    if (this.g.t && google.cv(this.g.t, !1) & 1) {
                                        a = this.g.id;
                                        var c;
                                        let d;
                                        if (b = (c = window.google) == null ? void 0 : (d = c.timers) == null ? void 0 : d.load) c = b.e.folr, google.c.e("load", "folr", c ? `${c},${a}` : a)
                                    }
                                }
                            };
                        var D = class {
                            constructor(a) {
                                this.g = a
                            }
                            toString() {
                                return this.g
                            }
                        };

                        function F(a) {
                            if (a instanceof D) return a.g;
                            throw Error("a");
                        };

                        function G(a) {
                            var b = google.cr,
                                c = [];
                            const d = [];
                            for (var e = 0; e < a.length; e++) b.has(a[e].module_id) || (c.push(a[e]), d.push(a[e].module_id));
                            if (d.length !== 0) {
                                a = [];
                                for (e = 0; e < c.length; e++) {
                                    var f = c[e].style_sheet.private_do_not_access_or_else_safe_style_sheet_wrapped_value;
                                    if (f === void 0 || f === null) throw Error("b");
                                    f = new D(f);
                                    f !== null && a.push(f)
                                }
                                a = a.map(F).join("");
                                c = document.createElement("style");
                                c.textContent = F(new D(a));
                                c.setAttribute("data-mss-styles", d.join(","));
                                a = document.getElementsByTagName("head")[0];
                                if (!a) throw Error("c");
                                a.appendChild(c);
                                for (c = 0; c < d.length; c++) b.add(d[c])
                            }
                        };

                        function H(a) {
                            a.req = new XMLHttpRequest;
                            a.cbfd = [];
                            a.cbvi = []
                        }

                        function I(a) {
                            H(a);
                            const b = new J(a);
                            google.ia.was(a, () => {
                                b.g.req.addEventListener("readystatechange", b.K);
                                b.g.req.open("GET", b.g.url);
                                for (const c of Object.keys(b.g.h)) b.g.req.setRequestHeader(c, b.g.h[c]);
                                b.i.ipf = Date.now();
                                b.g.req.send()
                            })
                        }
                        var K = function(a) {
                                a.A === 0 && a.S && google.drty && google.drty()
                            },
                            L = function(a) {
                                if (!a.P) {
                                    a.P = !0;
                                    if (Array.isArray(a.g.cbfd))
                                        for (const b of a.g.cbfd) b();
                                    a.g.cbfd = l
                                }
                            },
                            M = function(a, b) {
                                a.J = !0;
                                a.i.ft = Date.now();
                                a.l = 0;
                                a.g.req.removeEventListener("readystatechange", a.K);
                                a.g.req.abort();
                                r(a);
                                google.fce(a.g.t, a.g.cee, b);
                                L(a)
                            },
                            T = function(a, b, c) {
                                var d = b[0];
                                b = b[1];
                                if (!a.U && d === 2 && b == null) return N(a, c), a.U = !0, a.g.t;
                                switch (d) {
                                    case 1:
                                        M(a, Error(c));
                                        break;
                                    case 2:
                                        return O(a, b, c);
                                    case 6:
                                        return P(a, b, c);
                                    case 3:
                                        Q(a, b, c, !0);
                                        break;
                                    case 4:
                                        a.A++;
                                        google.ia.dq.push({
                                            astyp: a.g.astyp,
                                            cb: () => {
                                                R(c);
                                                a.A--;
                                                K(a)
                                            }
                                        });
                                        break;
                                    case 7:
                                        S(a, c);
                                        break;
                                    case 5:
                                        google.ia.dq.push({
                                            astyp: a.g.astyp,
                                            md: c
                                        });
                                        break;
                                    case 12:
                                        d = JSON.parse(c), G(d.css_modules)
                                }
                            },
                            N = function(a, b) {
                                const c = Date.now();
                                a.i.st = c;
                                m(a.g.t);
                                a.g.t.innerHTML = b;
                                B(a, a.g.t, c);
                                U(a.g.t);
                                a.g.t.setAttribute("eid", a.L[0]);
                                A(a, a.g.t, ++a.v)
                            },
                            O = function(a, b, c) {
                                b = document.getElementById(b);
                                m(b);
                                b.innerHTML = c;
                                a.v++;
                                for (const d of b.children) x(a, d, a.v);
                                B(a, b, Date.now());
                                U(b);
                                return b
                            },
                            P = function(a, b, c) {
                                const d = a.g.t.querySelector(`[data-async-ph="${b}"]`);
                                if (d) return V(a, d, c);
                                a.A++;
                                google.ia.dq.push({
                                    astyp: a.g.astyp,
                                    cb: () => {
                                        var e = a.g.t.querySelector(`[data-async-ph="${b}"]`);
                                        e ? (V(a, e, c), a.A--, K(a)) : (e = Error(), e.message = "ia_no_ph", google.ml(e, !1, {
                                            t: b
                                        }))
                                    }
                                })
                            },
                            Q = function(a, b, c, d) {
                                let e;
                                const f = (e = document.getElementById(b)) != null ? e : a.g.t.querySelector(`img[data-iid="${b}"]`);
                                f ? (f.setAttribute("data-deferred", "2"), f.src = c) : d ? a.g.cbfd.push(() => {
                                    google.ia.dq.push({
                                        astyp: a.g.astyp,
                                        cb: () => {
                                            Q(a, b, c, !1)
                                        }
                                    })
                                }) : (d = Error(), d.message = "ia_no_img", google.ml(d, !1, {
                                    t: b,
                                    u: c
                                }))
                            },
                            R = function(a) {
                                const b = document.createElement("script");
                                b.textContent = a;
                                try {
                                    document.body.appendChild(b)
                                } catch (c) {
                                    google.ml(c, !1, {
                                        script: a.substring(0, 500)
                                    })
                                }
                            },
                            S = function(a, b) {
                                const c = document.createElement("style");
                                try {
                                    c.appendChild(document.createTextNode(b)), a.g.t.appendChild(c)
                                } catch (d) {
                                    google.ml(d, !1, {
                                        css: b.substring(0, 500)
                                    })
                                }
                            },
                            U = function(a) {
                                a = a.querySelectorAll('script:not([type]),script[type="text/javascript"]');
                                for (const b of a) R(b.text)
                            },
                            V = function(a, b, c) {
                                var d = document.createElement("div");
                                d.innerHTML = c;
                                const e = Array.from(d.getElementsByTagName("script"), g => g.text),
                                    f = document.createDocumentFragment();
                                for (; d.firstChild;) try {
                                    f.appendChild(d.firstChild)
                                } catch (g) {
                                    google.ml(g, !1, {
                                        script: c.substring(0, 500)
                                    });
                                    return
                                }
                                c = Array.from(f.children);
                                d = b.parentElement;
                                m(b, !0);
                                d.replaceChild(f, b);
                                b = Date.now();
                                a.v++;
                                for (const g of e) R(g);
                                for (const g of c) B(a, g, b), A(a, g, a.v);
                                return f
                            },
                            J = class extends C {
                                constructor(a) {
                                    super(a);
                                    this.K = this.R.bind(this);
                                    this.A = this.v = this.cursor = 0;
                                    a.cbvi.push(() => {
                                        this.S = !0;
                                        K(this)
                                    })
                                }
                                R() {
                                    const a = this.g.req.readyState;
                                    try {
                                        if (a >= 2 && (this.i.ipfr || (this.i.ttfb = this.i.ipfr = Date.now()), this.g.req.status !== 200)) {
                                            M(this, Error(`${this.g.req.status}`));
                                            return
                                        }
                                        if (a < 3) return;
                                        this.G = performance.now();
                                        const c = this.g.req.responseText,
                                            d = c.length;
                                        if (d >= 5) {
                                            if (this.cursor === 0) {
                                                if (c.substring(0, 5) !== ")]}'\n") {
                                                    M(this, Error("d"));
                                                    return
                                                }
                                                this.cursor = 5
                                            }
                                            let e;
                                            for (; !this.J;) {
                                                e = c.indexOf(";", this.cursor);
                                                if (e === -1) break;
                                                const g = parseInt(c.substring(this.cursor, e), 16);
                                                if (isNaN(g)) break;
                                                if (d < e + g) break;
                                                var b = c.substring(e + 1, e + g + 1);
                                                if (this.L)
                                                    if (this.F) {
                                                        const h = this.F;
                                                        this.F = void 0;
                                                        const k = T(this, h, b);
                                                        k && google.drty && google.drty(k)
                                                    } else this.F = JSON.parse(b);
                                                else this.L = JSON.parse(b);
                                                this.cursor = e + g + 1
                                            }
                                            const f = performance.now() - this.G;
                                            f > this.D && (this.D = f);
                                            this.l += f;
                                            this.G = 0
                                        }
                                    } catch (c) {
                                        M(this, c);
                                        return
                                    }
                                    a === 4 && (this.i.ipfrl = this.i.acrt = Date.now(), this.i.aaft || (this.i.aaft = this.i.acrt), this.T = !0, t(this), L(this), u(this))
                                }
                            };
                        google.ia.mi[1] == null && (google.ia.mi[1] = I);
                    }).call(this);
                })();
                (function() {
                    var p = {
                        astyp: 'hpba',
                        cee: 'kvZGKe',
                        ck: '_v8HjaMKuBpCvptQP9sqQyAs_8v8HjaMKuBpCvptQP9sqQyAs',
                        cp: 'x',
                        dshp: false,
                        id: '_v8HjaMKuBpCvptQP9sqQyAs_8',
                        m: 1,
                        trid: '',
                        url: '/async/hpba?yv\x3d3\x26cs\x3d0\x26ei\x3dv8HjaMKuBpCvptQP9sqQyAs\x26async\x3d_basejs:/xjs/_/js/k%3Dxjs.hm.en.u6SkHWMIN6A.2018.O/am%3DAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAACAAEAABAIAAAAAAAEAAAAAAAAAAAAAAAhAAEAAAAAAAAAAAACAAAAgAAAAAADBAAAAQgRAAIGAQAAAAAAAAAAAAIBCAAAAABwB8AIwAAAEgAwAWgAQAACAgAAAACAAAAAQAAAgAAAAAAAgAAAAAAAAAAIAAACAAAgAAAAAAAAAAAAAAAAAAAAAADAAAAgAAAAAAAAAAAAHAAgBAAAGAQRQIAAAAAAACQA4AHAkNqCgAAAAAAAAAAAAAAAAAAAOAAQ3AGAQQQAAAAAAAAAAAAAAAAAAAAAAAACWFXAwI/dg%3D0/br%3D1/rs%3DACT90oH0Z4SrkdR6Pg9nIpVk5_BiJAH_9w,_basecss:/xjs/_/ss/k%3Dxjs.hm.YxLWi9QlUZA.L.B1.O/am%3DIAQoAQgAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAQAAAAAAAAACAQAgDAACwAAAAAYAAChAAAAAAAAFAAAAEAAAAAgAAAAAAAQAAGIALggAAAAWDQAAgQgRAAAEBQAAAIAQIEAAQCYAGgAFAEgIAAAADCQYAAGAASAAQAAAAAegBECCAAXeRgUBiIdAPIAIiAAAAAAAAAQAhAACAAAEAAAAQFAuKZgAQAAAEYAgAgCIAAYgIAAAAAAEAAAADABgAEAMOAQRQIAAAAAAAAAAgAAAABgAAAAAAAAAAAAAAAAAAAAAOA/br%3D1/rs%3DACT90oElhrQ7usnLRpCaml2ZrYQR-0ylBQ,_basecomb:/xjs/_/js/k%3Dxjs.hm.en.u6SkHWMIN6A.2018.O/ck%3Dxjs.hm.YxLWi9QlUZA.L.B1.O/am%3DIAQoAQgAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAQAAAAAAAAACAQEgDBAKwAAAAAYEAChAAAAAAAAFAAAhEAEAAAgAAAAAAAQCAGIArggAAAAXDQAAgQgRAAIGBQAAAIAQIEAAQCYBGgAFAEh4B8AIzCQYEgGwAWgAQAACAgegBECCAAXeRgUBiIdAPIAIiAAAAAAAAAQIhAACAAAkAAAAQFAuKZgAQAAAEYAgAgDIAAYgIAAAAAAEAAAAHABgBEAMOAQRQIAAAAAAACQA4AHAkNqCgAAAAAAAAAAAAAAAAAAAOAAQ3AGAQQQAAAAAAAAAAAAAAAAAAAAAAAACWFXAwI/d%3D1/ed%3D1/dg%3D0/br%3D1/ujg%3D1/rs%3DACT90oElwdv2wJYloW7LcRi-dVh2LPlFXg,_fmt:prog,_id:_v8HjaMKuBpCvptQP9sqQyAs_8\x26sp_imghp\x3dfalse\x26sp_hpep\x3d2\x26sp_hpte\x3d0',
                        wfpe: false,
                        wfve: false
                    };
                    var h = {};
                    p.h = h;
                    google.ia.q(p);
                })();
            </script>
        </div>
    </div>
    <div jscontroller="OPwjEf" class="JwzvB" jsaction="rcuQ6b:npT2md">
        <style>
            .JwzvB{margin:24px 0}
        </style>
        <div jscontroller="V23Ql" jsname="QokYF" class="bLgP0e">
            <style>
                .Jvj09c{padding-right:8px;vertical-align:middle}.eTE8fd{text-decoration:none;color:var(--YLNNHc)}.bLgP0e{border:none;box-shadow:none;opacity:0;transition:opacity .18s ease-out}.WrKXc{display:none;background:none;border-radius:8px 8px 0 0;margin:0 0 10px 16px;padding:0 8px 4px 8px}.WrKXc.pne3Yd{display:flex}.VsEpQb{align-items:center;color:var(--YLNNHc);cursor:default;display:flex;font-family:Google Sans,Roboto,sans-serif;line-height:28px;text-align:left;width:100%;font-size:20px}.jh86Nb{align-items:center;display:flex;flex:1;max-width:-moz-available;max-width:-webkit-fill-available;max-width:fill-available;text-decoration:none;border-bottom:1px solid #dadce0;margin:0 16px;min-height:52px}.uVgSEd{height:20px;min-width:20px;padding:0 8px 0 14px;width:20px}.uVgSEd svg{fill:var(--bbQxAb)}#E80fqd{display:none}#g08KZb{display:none}.o5j3Ue{display:flex;flex-direction:column;margin:0;overflow:auto;padding:0;list-style:none}.o5j3Ue::-webkit-scrollbar{display:none}.yAOL0{background-position:center;background-repeat:no-repeat;background-size:contain;border-radius:4px;height:40px;margin:12px 0 8px 12px;width:40px}.MWnfue{display:flex;flex:1;flex-direction:column;min-width:0;margin:5px 0 5px 13px}.h5AwLb{color:var(--YLNNHc);text-align:left;word-break:break-word;font-size:16px;line-height:20px}.SwwBmb{color:var(--IXoxUe);font-size:14px;line-height:20px;overflow:hidden;text-overflow:ellipsis}.DiX1e{white-space:nowrap}.v4cbwf{display:-webkit-box;-webkit-box-orient:vertical;-webkit-line-clamp:2}
            </style>
            <div jsname="tJHJj" class="WrKXc">
                <div jsname="gYxgdc" class="VsEpQb" aria-level="1" role="heading">Trending searches</div>
                <g-dropdown-menu jsname="tcAyJf" class="xcNbEc adGN6" jscontroller="pFsdhd" jsdata="xJGXK;_;v8HjaMKuBpCvptQP9sqQyAs3" jsshadow="" jsaction="rcuQ6b:npT2md;mMf61e:GKlhgf;YraOve:cCpuJc;kNOP9c:bzkPEc;hmb6Ye:oyYkKb;KyPa0e:G8Ofmd;wjOG7e:rWoVB;ZvRO4b:L76sMb;LyWNEf:L76sMb">
                    <g-popup jsname="zpo2ue" jscontroller="DPreE" jsaction="A05xBd:IYtByb;EOZ57e:WFrRFb;" jsdata="mVjAjf;_;v8HjaMKuBpCvptQP9sqQyAs4">
                        <div jsname="oYxtQd" class="CcNe6e" aria-expanded="false" aria-haspopup="true" jsslot="" role="button" tabindex="0" aria-label="More options" jsaction="WFrRFb;keydown:uYT2Vb">
                            <div> <span style="height:18px;line-height:18px;width:18px" class="z1asCe SaPW2b"><svg focusable="false" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M12 8c1.1 0 2-.9 2-2s-.9-2-2-2-2 .9-2 2 .9 2 2 2zm0 2c-1.1 0-2 .9-2 2s.9 2 2 2 2-.9 2-2-.9-2-2-2zm0 6c-1.1 0-2 .9-2 2s.9 2 2 2 2-.9 2-2-.9-2-2-2z"></path></svg></span>                                </div>
                        </div>
                        <div jsname="V68bde" class="UjBGL pkWBse iRQHZe" style="display:none;z-index:1">
                            <g-menu jsname="iXXGtd" class="WNN1b yTik0 PBn44e iQXTJe wplJBd" jscontroller="WlNQGd" role="menu" tabindex="-1" jsaction="PSl28c;focus:h06R8;keydown:uYT2Vb;mouseenter:WOQqYb;mouseleave:Tx5Rb;mouseover:IgJl9c">
                                <g-menu-item jsname="NNJLud" jscontroller="CnSW2d" class="EpPYLd Zt0a5e" role="none" data-short-label="" jsdata="zPXzie;_;v8HjaMKuBpCvptQP9sqQyAs5">
                                    <div jsname="ibnC6b" class="YpcDnf OSrXXb" role="none">
                                        <div jscontroller="aBr2Mc" jsname="JUypV" jsaction="rcuQ6b:npT2md;swd3Zc:Qv1t5d" data-async-context="async_id:duf3-122;authority:0;card_id:;entry_point:0;feature_id:;ftoe:0;header:0;is_jobs_spam_form:0;open:0;preselect_answer_index:-1;suggestions:;suggestions_subtypes:;suggestions_types:;surface:0;title:;type:122">
                                            <style>
                                                a.oBa0Fe{color:var(--IXoxUe);float:right;font-style:italic;-webkit-tap-highlight-color:transparent;tap-highlight-color:transparent}a.aciXEb{padding:0;background:rgba(255,255,255,.9);color:var(--JKqx2) !important}.RTZ84b{color:var(--IXoxUe);cursor:pointer;padding-right:8px}.c2xzTb .RTZ84b{padding-top:1px;padding-right:4px}.XEKxtf{color:var(--IXoxUe);float:right;font-size:12px;padding-bottom:4px}
                                            </style>
                                            <div jscontroller="EkevXb" style="display:none" jsaction="rcuQ6b:npT2md"></div>
                                            <div id="duf3-122" data-jiis="up" data-async-type="duffy3" data-async-context-required="type,open,feature_id,async_id,entry_point,authority,card_id,ftoe,title,header,suggestions,surface,suggestions_types,suggestions_subtypes,preselect_answer_index,is_jobs_spam_form"
                                                class="yp" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQ-0EIGQ"></div>
                                            <a jsname="JUypV" jscontroller="XCToU" jsaction="MHYjYb" data-async-trigger="duf3-122" role="link" tabindex="0">
                                                <div> <span class="Jvj09c z1asCe dC6Tmc"><svg focusable="false" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M20 2H4c-1.1 0-1.99.9-1.99 2L2 22l4-4h14c1.1 0 2-.9 2-2V4c0-1.1-.9-2-2-2zm-7 12h-2v-2h2v2zm0-4h-2V6h2v4z"></path></svg></span>                                                    <span> Send feedback </span> </div>
                                            </a>
                                        </div>
                                    </div>
                                </g-menu-item>
                                <g-menu-item jsname="NNJLud" jscontroller="CnSW2d" class="EpPYLd Zt0a5e" role="none" data-short-label="" jsdata="zPXzie;_;v8HjaMKuBpCvptQP9sqQyAs5">
                                    <div jsname="ibnC6b" class="YpcDnf OSrXXb" role="none">
                                        <a class="eTE8fd" href="/preferences?hl=en&amp;prev=https://www.google.com/&amp;sa=X&amp;ved=0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQr8gICBo" role="link" tabindex="0">
                                            <div> <span class="Jvj09c z1asCe E9hVAb"><svg focusable="false" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M19.43 12.98c.04-.32.07-.64.07-.98s-.03-.66-.07-.98l2.11-1.65c.19-.15.24-.42.12-.64l-2-3.46c-.12-.22-.39-.3-.61-.22l-2.49 1c-.52-.4-1.08-.73-1.69-.98l-.38-2.65C14.46 2.18 14.25 2 14 2h-4c-.25 0-.46.18-.49.42l-.38 2.65c-.61.25-1.17.59-1.69.98l-2.49-1c-.23-.09-.49 0-.61.22l-2 3.46c-.13.22-.07.49.12.64l2.11 1.65c-.04.32-.07.65-.07.98s.03.66.07.98l-2.11 1.65c-.19.15-.24.42-.12.64l2 3.46c.12.22.39.3.61.22l2.49-1c.52.4 1.08.73 1.69.98l.38 2.65c.03.24.24.42.49.42h4c.25 0 .46-.18.49-.42l.38-2.65c.61-.25 1.17-.59 1.69-.98l2.49 1c.23.09.49 0 .61-.22l2-3.46c.12-.22.07-.49-.12-.64l-2.11-1.65zM12 15.5c-1.93 0-3.5-1.57-3.5-3.5s1.57-3.5 3.5-3.5 3.5 1.57 3.5 3.5-1.57 3.5-3.5 3.5z"></path></svg></span>                                                <span>Settings</span> </div>
                                        </a>
                                    </div>
                                </g-menu-item>
                                <g-menu-item jsname="hgDUwe" jscontroller="CnSW2d" class="EpPYLd Zt0a5e LGiluc" aria-hidden="true" role="separator" data-short-label="" jsdata="zPXzie;_;v8HjaMKuBpCvptQP9sqQyAs6"></g-menu-item>
                                <g-menu-item jsname="tqp7ud" jscontroller="CnSW2d" class="EpPYLd Zt0a5e" role="menuitem" tabindex="-1" data-short-label="" jsdata="zPXzie;_;v8HjaMKuBpCvptQP9sqQyAs7">
                                    <g-ripple jscontroller="KbYvUc" class="rLdWfe" jsdata="SMQQFe;_;v8HjaMKuBpCvptQP9sqQyAs8" jsaction="yYNR0b;rcuQ6b:npT2md">
                                        <div jsname="AHe6Kc" class="MDfoTd"></div>
                                        <div jsname="yA8yjc" class="hdqIFe"></div>
                                    </g-ripple>
                                    <div jsname="ibnC6b" class="YpcDnf OSrXXb">
                                        <div> <span> Close menu </span> </div>
                                    </div>
                                </g-menu-item>
                            </g-menu>
                        </div>
                    </g-popup>
                </g-dropdown-menu>
            </div>
            <ul jsname="rn6yge" class="o5j3Ue" data-num_pips_row="7"></ul>
            <li id="E80fqd" role="link" tabindex="0"> <a class="jh86Nb" href="#" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQ470GCBw"> <span class="uVgSEd z1asCe vdLE0b"><svg focusable="false" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M16 6l2.29 2.29-4.88 4.88-4-4L2 16.59 3.41 18l6-6 4 4 6.3-6.29L22 12V6z"></path></svg></span> <div class="MWnfue"> <span class="h5AwLb"></span> <span class="SwwBmb"></span> </div> <span class="yAOL0" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQzckICB0"></span> </a>                </li>
        </div>
    </div>
    </div>
    </div>
    <div>
        <div id="footer">
            <style>
                .fbar{margin-left:-27px;text-align:center;font-size:14px;line-height:35px;}.M6hT6{left:0;right:0}.fbar a{text-decoration:none;white-space:nowrap;color:var(--bbQxAb)}.fbar .EyBRub{text-decoration:none;white-space:nowrap;color:var(--bbQxAb);font-size:14px;line-height:35px;}.Fx4vi{padding-left:27px;margin:0 !important;display:inline-block}#fsl{white-space:nowrap}.f6F9Be{background:#f2f2f2;}.JQyAhb{border-top:1px solid var(--gS5jXb);padding-top:10px}.fbar p{color:#70757a}.fbar a:hover{color:var(--bbQxAb)}
            </style>
            <div class="M6hT6 f6F9Be" id="fbar" style="text-align:center;line-height:33px">
                <div class="b2hzT">
                    <style>
                        .b0KoTc{color:var(--IXoxUe)}.b0KoTc{margin:0 12px;text-align:left}
                    </style>
                    <div class="b0KoTc"></div>
                </div>
                <style>
                    .footer__homepage-mobile-settings-row-above-fold{padding-bottom:10px}
                </style>
                <div class="fbar footer__homepage-mobile-settings-row-above-fold JQyAhb">
                    <div data-spl="/setprefs?hl=en&amp;prev=https://www.google.com/?pccc%3D1&amp;sig=0_7QCvYJ2iJw40tBfcNgG7LYMrLU0%3D&amp;cs=2" id="YUIDDb" role="menuitem" tabindex="-1" style="display:none"></div><a class="Fx4vi" jscontroller="lllQlf" href="#" role="link" tabindex="0" jsaction="ok5gFc" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQqsEHCB8">Dark theme: off</a>
                    <style>
                        .j9xd9b{display:inline-block}.lA8P5e{color:var(--bbQxAb)}
                    </style><span class="j9xd9b" jscontroller="nabPbb" data-ffp="false" jsaction="KyPa0e:Y0y4c;"><style>.cF4V5c{background-color:var(--xhUGwc)}.cF4V5c g-menu-item{display:block;font-size:14px;line-height:23px;white-space:nowrap}.cF4V5c g-menu-item a,.cF4V5c .y0fQ9c{display:block;padding-top:4px;padding-bottom:4px;cursor:pointer}.cF4V5c g-menu-item a,.cF4V5c g-menu-item a:visited,.cF4V5c g-menu-item a:hover{text-decoration:inherit;color:inherit}</style><g-popup jsname="V68bde" jscontroller="DPreE" jsaction="A05xBd:IYtByb;EOZ57e:WFrRFb;" jsdata="mVjAjf;_;v8HjaMKuBpCvptQP9sqQyAs9"><div jsname="oYxtQd"><a jsname="LgbsSe" aria-haspopup="true" role="button" jsaction="WFrRFb;keydown:uYT2Vb" class="Fx4vi" href="https://www.google.com/preferences?hl=en" data-jsarwt="1" data-usg="AOvVaw0XvhSqpc_GFt9aG_mRLp1F" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQzq0CCCE">Settings</a></div><div jsname="V68bde" class="UjBGL pkWBse iRQHZe" style="display:none;z-index:200" id="_v8HjaMKuBpCvptQP9sqQyAs_9"></div></g-popup></span>
                    <a
                        class="Fx4vi" href="https://policies.google.com/privacy?hl=en&amp;fg=1" data-jsarwt="1" data-usg="AOvVaw25lN6TjyY2uszSImdhMejE" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQ8awCCCQ">Privacy</a><a class="Fx4vi" href="https://policies.google.com/terms?hl=en&amp;fg=1" data-jsarwt="1" data-usg="AOvVaw2d8Ahs2oc0jxVOgPfpr4Fe" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQ8qwCCCU">Terms</a></div>
            </div>
        </div>
    </div>
    </div>
    <div class="fbar M6hT6 As6eLe Lt2Ned">
        <style>
            .As6eLe{padding-bottom:10px;background:#f2f2f2}.Lt2Ned{margin-top:-10px}
        </style>
        <div><a class="Fx4vi" href="https://www.google.com/intl/en_us/ads/?subid=ww-ww-et-g-awa-a-g_hpafoot1_1!o2&amp;utm_source=google.com&amp;utm_medium=referral&amp;utm_campaign=google_hpafooter&amp;fg=1" data-jsarwt="1" data-usg="AOvVaw3XLzhicA5ZZW0afDbW0l2T"
                data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQkdQCCCc">Advertising</a><a class="Fx4vi" href="https://www.google.com/services/?subid=ww-ww-et-g-awa-a-g_hpbfoot1_1!o2&amp;utm_source=google.com&amp;utm_medium=referral&amp;utm_campaign=google_hpbfooter&amp;fg=1"
                data-jsarwt="1" data-usg="AOvVaw2SJ4zwRVXKyZr53qG9dm4K" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQktQCCCg">Business</a><a class="Fx4vi" href="https://about.google/?utm_source=google-US&amp;utm_medium=referral&amp;utm_campaign=hp-footer&amp;fg=1"
                data-jsarwt="1" data-usg="AOvVaw3D6_DogiQ_ppybHZyBMjRr" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQkNQCCCk">About</a></div>
        <script nonce="IJ8UXppd3ETKm0PGR2OelA">
            (function() {
                (function(b) {
                    var a = b.querySelector("#gb-main"),
                        c = b.body.clientWidth;
                    a && (a.style.minHeight = a.scrollHeight + "px", "fbar" in a.dataset || (a.dataset.fbar = "1", window.addEventListener("resize", function() {
                        b.body.clientWidth !== c && (a.style.minHeight = b.body.clientHeight + "px", c = b.body.clientWidth)
                    }, !1)))
                })(document);
            }).call(this);
        </script>
    </div>
    <div style="display:'none'" jscontroller="KHourd" jsdata="C4mkuf;_;v8HjaMKuBpCvptQP9sqQyAs10" jsaction="rcuQ6b:npT2md" data-ved="0ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQuqMJCCo"></div>
    <script nonce="IJ8UXppd3ETKm0PGR2OelA">
        this.gbar_ = this.gbar_ || {};
        (function(_) {
            var window = this;
            try {
                _.Cd = function(a, b, c) {
                    if (!a.j)
                        if (c instanceof Array)
                            for (var d of c) _.Cd(a, b, d);
                        else {
                            d = (0, _.z)(a.C, a, b);
                            const e = a.v + c;
                            a.v++;
                            b.dataset.eqid = e;
                            a.B[e] = d;
                            b && b.addEventListener ? b.addEventListener(c, d, !1) : b && b.attachEvent ? b.attachEvent("on" + c, d) : a.o.log(Error("t`" + b))
                        }
                };
            } catch (e) {
                _._DumpException(e)
            }
            try {
                var Dd = document.querySelector(".gb_J .gb_B"),
                    Ed = document.querySelector("#gb.gb_7c");
                Dd && !Ed && _.Cd(_.ld, Dd, "click");
            } catch (e) {
                _._DumpException(e)
            }
            try {
                _.mh = function(a) {
                    if (a.v) return a.v;
                    for (const b in a.i)
                        if (a.i[b].ha() && a.i[b].B()) return a.i[b];
                    return null
                };
                _.nh = function(a, b) {
                    a.i[b.J()] = b
                };
                var oh = new class extends _.P {
                    constructor() {
                        var a = _.Xc;
                        super();
                        this.B = a;
                        this.v = null;
                        this.o = {};
                        this.C = {};
                        this.i = {};
                        this.j = null
                    }
                    A(a) {
                        this.i[a] && (_.mh(this) && _.mh(this).J() == a || this.i[a].P(!0))
                    }
                    Sa(a) {
                        this.j = a;
                        for (const b in this.i) this.i[b].ha() && this.i[b].Sa(a)
                    }
                    lc(a) {
                        return a in this.i ? this.i[a] : null
                    }
                };
                _.od("dd", oh);
            } catch (e) {
                _._DumpException(e)
            }
            try {
                _.Fi = function(a, b) {
                    return _.J(a, 36, b)
                };
            } catch (e) {
                _._DumpException(e)
            }
            try {
                var Gi = document.querySelector(".gb_z .gb_B"),
                    Hi = document.querySelector("#gb.gb_7c");
                Gi && !Hi && _.Cd(_.ld, Gi, "click");
            } catch (e) {
                _._DumpException(e)
            }
        })(this.gbar_);
        // Google Inc.
    </script>
    <div>
        <div>
            <div class="gb_L">Google apps</div>
        </div>
    </div>
    <script nonce="IJ8UXppd3ETKm0PGR2OelA">
        this.gbar_ = this.gbar_ || {};
        (function(_) {
            var window = this;
            try {
                var Gd;
                Gd = class extends _.pd {};
                _.Hd = function(a, b) {
                    if (b in a.i) return a.i[b];
                    throw new Gd;
                };
                _.Id = function(a) {
                    return _.Hd(_.md.i(), a)
                };
            } catch (e) {
                _._DumpException(e)
            }
            try {
                /*

                 Copyright Google LLC
                 SPDX-License-Identifier: Apache-2.0
                */
                var Ld;
                _.Jd = function(a) {
                    const b = a.length;
                    if (b > 0) {
                        const c = Array(b);
                        for (let d = 0; d < b; d++) c[d] = a[d];
                        return c
                    }
                    return []
                };
                Ld = function(a) {
                    return new _.Kd(b => b.substr(0, a.length + 1).toLowerCase() === a + ":")
                };
                _.Md = globalThis.trustedTypes;
                _.Nd = class {
                    constructor(a) {
                        this.i = a
                    }
                    toString() {
                        return this.i
                    }
                };
                _.Od = new _.Nd("about:invalid#zClosurez");
                _.Kd = class {
                    constructor(a) {
                        this.Sh = a
                    }
                };
                _.Pd = [Ld("data"), Ld("http"), Ld("https"), Ld("mailto"), Ld("ftp"), new _.Kd(a => /^[^:]*([/?#]|$)/.test(a))];
                _.Qd = class {
                    constructor(a) {
                        this.i = a
                    }
                    toString() {
                        return this.i + ""
                    }
                };
                _.Rd = new _.Qd(_.Md ? _.Md.emptyHTML : "");
            } catch (e) {
                _._DumpException(e)
            }
            try {
                var Vd, fe, ie, Ud, Wd, ae;
                _.Sd = function(a) {
                    if (a == null) return a;
                    if (typeof a === "string" && a) a = +a;
                    else if (typeof a !== "number") return;
                    return (0, _.Na)(a) ? a | 0 : void 0
                };
                _.Td = function(a, b) {
                    return a.lastIndexOf(b, 0) == 0
                };
                Vd = function() {
                    let a = null;
                    if (!Ud) return a;
                    try {
                        const b = c => c;
                        a = Ud.createPolicy("ogb-qtm#html", {
                            createHTML: b,
                            createScript: b,
                            createScriptURL: b
                        })
                    } catch (b) {}
                    return a
                };
                _.Xd = function() {
                    Wd === void 0 && (Wd = Vd());
                    return Wd
                };
                _.Zd = function(a) {
                    const b = _.Xd();
                    a = b ? b.createScriptURL(a) : a;
                    return new _.Yd(a)
                };
                _.$d = function(a) {
                    if (a instanceof _.Yd) return a.i;
                    throw Error("x");
                };
                _.be = function(a) {
                    if (ae.test(a)) return a
                };
                _.ce = function(a) {
                    if (a instanceof _.Nd)
                        if (a instanceof _.Nd) a = a.i;
                        else throw Error("x");
                    else a = _.be(a);
                    return a
                };
                _.de = function(a, b = document) {
                    let c;
                    const d = (c = b.querySelector) == null ? void 0 : c.call(b, `${a}[nonce]`);
                    return d == null ? "" : d.nonce || d.getAttribute("nonce") || ""
                };
                _.S = function(a, b, c) {
                    return _.Ma(_.Lc(a, b, c, _.Kc))
                };
                _.ee = function(a, b) {
                    return _.Sd(_.Lc(a, b, void 0, _.Kc))
                };
                fe = class extends _.O {
                    constructor(a) {
                        super(a)
                    }
                    Xb(a) {
                        return _.L(this, 24, a)
                    }
                };
                _.ge = function() {
                    return _.C(_.hd, fe, 1)
                };
                _.he = function(a) {
                    var b = _.Ka(a);
                    return b == "array" || b == "object" && typeof a.length == "number"
                };
                Ud = _.Md;
                _.Yd = class {
                    constructor(a) {
                        this.i = a
                    }
                    toString() {
                        return this.i + ""
                    }
                };
                ae = /^\s*(?!javascript:)(?:[\w+.-]+:|[^:/?#]*(?:[/?#]|$))/i;
                var oe, se, je;
                _.le = function(a) {
                    return a ? new je(_.ke(a)) : ie || (ie = new je)
                };
                _.me = function(a, b) {
                    return typeof b === "string" ? a.getElementById(b) : b
                };
                _.T = function(a, b) {
                    var c = b || document;
                    c.getElementsByClassName ? a = c.getElementsByClassName(a)[0] : (c = document, a = a ? (b || c).querySelector(a ? "." + a : "") : _.ne(c, "*", a, b)[0] || null);
                    return a || null
                };
                _.ne = function(a, b, c, d) {
                    a = d || a;
                    return (b = b && b != "*" ? String(b).toUpperCase() : "") || c ? a.querySelectorAll(b + (c ? "." + c : "")) : a.getElementsByTagName("*")
                };
                _.pe = function(a, b) {
                    _.Bb(b, function(c, d) {
                        d == "style" ? a.style.cssText = c : d == "class" ? a.className = c : d == "for" ? a.htmlFor = c : oe.hasOwnProperty(d) ? a.setAttribute(oe[d], c) : _.Td(d, "aria-") || _.Td(d, "data-") ? a.setAttribute(d, c) : a[d] = c
                    })
                };
                oe = {
                    cellpadding: "cellPadding",
                    cellspacing: "cellSpacing",
                    colspan: "colSpan",
                    frameborder: "frameBorder",
                    height: "height",
                    maxlength: "maxLength",
                    nonce: "nonce",
                    role: "role",
                    rowspan: "rowSpan",
                    type: "type",
                    usemap: "useMap",
                    valign: "vAlign",
                    width: "width"
                };
                _.qe = function(a) {
                    return a ? a.defaultView : window
                };
                _.te = function(a, b) {
                    const c = b[1],
                        d = _.re(a, String(b[0]));
                    c && (typeof c === "string" ? d.className = c : Array.isArray(c) ? d.className = c.join(" ") : _.pe(d, c));
                    b.length > 2 && se(a, d, b);
                    return d
                };
                se = function(a, b, c) {
                    function d(e) {
                        e && b.appendChild(typeof e === "string" ? a.createTextNode(e) : e)
                    }
                    for (let e = 2; e < c.length; e++) {
                        const f = c[e];
                        !_.he(f) || _.Lb(f) && f.nodeType > 0 ? d(f) : _.bc(f && typeof f.length == "number" && typeof f.item == "function" ? _.Jd(f) : f, d)
                    }
                };
                _.ue = function(a) {
                    return _.re(document, a)
                };
                _.re = function(a, b) {
                    b = String(b);
                    a.contentType === "application/xhtml+xml" && (b = b.toLowerCase());
                    return a.createElement(b)
                };
                _.ve = function(a) {
                    let b;
                    for (; b = a.firstChild;) a.removeChild(b)
                };
                _.we = function(a) {
                    return a && a.parentNode ? a.parentNode.removeChild(a) : null
                };
                _.xe = function(a, b) {
                    return a && b ? a == b || a.contains(b) : !1
                };
                _.ke = function(a) {
                    return a.nodeType == 9 ? a : a.ownerDocument || a.document
                };
                je = function(a) {
                    this.i = a || _.t.document || document
                };
                _.n = je.prototype;
                _.n.H = function(a) {
                    return _.me(this.i, a)
                };
                _.n.Qa = function(a, b, c) {
                    return _.te(this.i, arguments)
                };
                _.n.appendChild = function(a, b) {
                    a.appendChild(b)
                };
                _.n.Je = _.ve;
                _.n.mg = _.we;
                _.n.lg = _.xe;
            } catch (e) {
                _._DumpException(e)
            }
            try {
                _.Mi = function(a) {
                    const b = _.de("script", a.ownerDocument);
                    b && a.setAttribute("nonce", b)
                };
                _.Ni = function(a) {
                    if (!a) return null;
                    a = _.I(a, 4);
                    var b;
                    a === null || a === void 0 ? b = null : b = _.Zd(a);
                    return b
                };
                _.Oi = function(a, b, c) {
                    a = a.fa;
                    return _.zb(a, a[_.v] | 0, b, c) !== void 0
                };
                _.Pi = class extends _.O {
                    constructor(a) {
                        super(a)
                    }
                };
                _.Qi = function(a, b) {
                    return (b || document).getElementsByTagName(String(a))
                };
            } catch (e) {
                _._DumpException(e)
            }
            try {
                var Si = function(a, b, c) {
                        a < b ? Ri(a + 1, b) : _.Xc.log(Error("W`" + a + "`" + b), {
                            url: c
                        })
                    },
                    Ri = function(a, b) {
                        if (Ti) {
                            const c = _.ue("SCRIPT");
                            c.async = !0;
                            c.type = "text/javascript";
                            c.charset = "UTF-8";
                            c.src = _.$d(Ti);
                            _.Mi(c);
                            c.onerror = _.Ob(Si, a, b, c.src);
                            _.Qi("HEAD")[0].appendChild(c)
                        }
                    },
                    Ui = class extends _.O {
                        constructor(a) {
                            super(a)
                        }
                    };
                var Vi = _.C(_.hd, Ui, 17) || new Ui,
                    Wi, Ti = (Wi = _.C(Vi, _.Pi, 1)) ? _.Ni(Wi) : null,
                    Xi, Yi = (Xi = _.C(Vi, _.Pi, 2)) ? _.Ni(Xi) : null,
                    Zi = function() {
                        Ri(1, 2);
                        if (Yi) {
                            const a = _.ue("LINK");
                            a.setAttribute("type", "text/css");
                            a.href = _.$d(Yi).toString();
                            a.rel = "stylesheet";
                            let b = _.de("style", document);
                            b && a.setAttribute("nonce", b);
                            _.Qi("HEAD")[0].appendChild(a)
                        }
                    };
                (function() {
                    const a = _.ge();
                    if (_.S(a, 18)) Zi();
                    else {
                        const b = _.ee(a, 19) || 0;
                        window.addEventListener("load", () => {
                            window.setTimeout(Zi, b)
                        })
                    }
                })();
            } catch (e) {
                _._DumpException(e)
            }
        })(this.gbar_);
        // Google Inc.
    </script>
    <script nonce="IJ8UXppd3ETKm0PGR2OelA">
        (function() {
            var s = '/images/nav_logo325_hr.webp';
            google.dclc(function() {
                document.images && ((new Image).src = s)
            });
        })();
        (function() {
            var a = this || self;
            var c = document.querySelector("form");
            if (c) {
                var d = function(b) {
                    b.key === "Enter" && (b.preventDefault(), c.submit && c.submit())
                };
                c.addEventListener("keydown", d);
                a.sbmlhf = d
            };
        }).call(this);
    </script><textarea class="csi" name="csi" style="display:none"></textarea>
    <script nonce="IJ8UXppd3ETKm0PGR2OelA">
        (function() {
            function a(c, d) {
                google.c.e("load", c, String(d))
            };
            window.google = window.google || {};
            google.c.iim = google.c.iim || {};
            var b = Date.now();
            google.tick("load", "prt", b, "SearchBodyEnd");
            a("imn", document.getElementsByTagName("img").length);
            a("dtc", document.getElementsByTagName("div").length);
            a("stc", document.getElementsByTagName("span").length);
            google.c.ub ? google.c.ub() : google.c.ubf && google.c.u("frt");
            google.c.cae || google.c.maft(b, null);
            google.c.miml(b);
            google.c.u("prt");
        }).call(this);
        (function() {
            window.google = window.google || {};
            window.google.ishk = [];

            function a() {
                return window.scrollY + window.document.documentElement.clientHeight >= Math.max(document.body.scrollHeight, document.body.offsetHeight)
            }

            function b() {
                a() && window.google.ishk.length === 0 && (window.google.bs = !0, window.removeEventListener("scroll", b))
            }
            a() ? window.google.bs = !0 : (window.google.bs = !1, window.addEventListener("scroll", b));
        }).call(this);
        (function() {
            google.jl = {
                bfl: 0,
                dw: false,
                eli: false,
                ine: false,
                ubm: false,
                uwp: true,
                vs: false
            };
        })();
        (function() {
            var pmc = '{\x22aa\x22:{},\x22abd\x22:{\x22abd\x22:false,\x22deb\x22:false,\x22det\x22:false},\x22async\x22:{},\x22bct\x22:{},\x22cdos\x22:{},\x22csi\x22:{},\x22d\x22:{},\x22foot\x22:{},\x22hsm\x22:{},\x22jsa\x22:{\x22csi\x22:true,\x22csir\x22:100},\x22pHXghd\x22:{},\x22qim\x22:{},\x22rr4R9e\x22:{},\x22sb_wiz\x22:{\x22rfs\x22:[],\x22scq\x22:\x22\x22,\x22stok\x22:\x22GexzxJrMbChTcBn5Zu0_YQy1OJ8\x22},\x22sonic\x22:{},\x22spch\x22:{\x22ae\x22:\x22Please check your microphone.  \\u003Ca href\x3d\\\x22https://support.google.com/chrome/?p\x3dui_voice_search\\\x22 target\x3d\\\x22_blank\\\x22\\u003ELearn more\\u003C/a\\u003E\x22,\x22ak\x22:\x22\x22,\x22cd\x22:0,\x22fp\x22:true,\x22hl\x22:\x22en-US\x22,\x22im\x22:\x22Click \\u003Cb\\u003EAllow\\u003C/b\\u003E to start voice search\x22,\x22iw\x22:\x22Waiting...\x22,\x22lm\x22:\x22Listening...\x22,\x22lu\x22:\x22%1$s voice search not available\x22,\x22mb\x22:true,\x22ne\x22:\x22No Internet connection\x22,\x22nt\x22:\x22Didn\x27t get that. \\u003Cspan\\u003ETry again\\u003C/span\\u003E\x22,\x22nv\x22:\x22Please check your microphone and audio levels.  \\u003Ca href\x3d\\\x22https://support.google.com/chrome/?p\x3dui_voice_search\\\x22 target\x3d\\\x22_blank\\\x22\\u003ELearn more\\u003C/a\\u003E\x22,\x22pe\x22:\x22Voice search has been turned off.  \\u003Ca href\x3d\\\x22https://support.google.com/chrome/?p\x3dui_voice_search\\\x22 target\x3d\\\x22_blank\\\x22\\u003EDetails\\u003C/a\\u003E\x22,\x22ri\x22:false,\x22rm\x22:\x22Speak now\x22,\x22s3\x22:false}}';
            google.pmc = JSON.parse(pmc);
        })();
        (function() {
            var r = ['sb_wiz', 'aa', 'abd', 'async', 'foot', 'pHXghd', 'sonic', 'spch'];
            google.plm(r);
        })();
        (function() {
            var m = {
                "v8HjaMKuBpCvptQP9sqQyAs1": ["mobile-gws-wiz-hp", "", "", "", 1, 0, null, null, null, "", "v8HjaMKuBpCvptQP9sqQyAs", 0, "en", null, null, 1, 3, 15, 8, null, "0", 0, 0, 1800000, 1, -1, 0, null, 0, 1, 1, null, null, 0, 0, 0, "", null, 0, null, 0, null, 0, null, 1, 0, 1, null, ["mobile-gws-modeless-local", "", ""], null, ["mobile-gws-modeless-products", "", "sh"],
                    ["mobile-gws-modeless-lodging", "", ""],
                    ["mobile-gws-modeless-flights", "", ""], 0, null, null, ["mobile-gws-modeless-vss-products", "", ""],
                    ["mobile-gws-modeless-vss-jobs", "", ""],
                    ["mobile-gws-modeless-vss-local-travel", "", ""],
                    ["mobile-gws-modeless-video", "", "v"],
                    ["mobile-gws-modeless-jobs", "", ""],
                    ["mobile-gws-modeless-vss-flights", "", ""],
                    ["mobile-gws-modeless", "", ""],
                    ["mobile-gws-modeless-shopping", "", "sh"], -1, null, null, ["multimodal-lens-web", "", ""], null, 0, 0, ["mobile-gws-modeless-forums", "", ""],
                    ["mobile-gws-modeless-short-videos", "", ""], null, ["mobile-gws-modeless-web-only", "", ""], null, ""
                ],
                "v8HjaMKuBpCvptQP9sqQyAs2": [null, null, null, "autocomplete_user_feedback_kp_id", null, 11, null, null, null, null, null, 5010715, "searchbox", null, "AutocompletePrediction", null, null, null, null, 11],
                "v8HjaMKuBpCvptQP9sqQyAs3": [2, 1, 0, 0, 0, 0, -145, 5, 0, 0],
                "v8HjaMKuBpCvptQP9sqQyAs4": [4, 1, null, null, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                "v8HjaMKuBpCvptQP9sqQyAs5": ["", 6, 0],
                "v8HjaMKuBpCvptQP9sqQyAs6": ["", 4, 0],
                "v8HjaMKuBpCvptQP9sqQyAs7": ["", 5, 0],
                "v8HjaMKuBpCvptQP9sqQyAs8": [0, 1],
                "v8HjaMKuBpCvptQP9sqQyAs9": [2, 5, null, null, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                "v8HjaMKuBpCvptQP9sqQyAs10": [null, null, null, null, null, null, null, null, null, null, 3, null, null, null, null, null, null, null, null, null, 0, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 1, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 1, 1, null, null, null, null, null, null, null, null, null, "UNKNOWN_BROWSER", null, null, null, null, null, null, null, null, null, null, 3]
            };
            var a = m;
            if (window.W_jd)
                for (var b in a) window.W_jd[b] = a[b];
            else window.W_jd = a;
        })();
        (function() {
            window.WIZ_global_data = {
                "Im6cmf": "/wizrpcui/_/WizRpcUi",
                "Yllh3e": "%.@.1759756735104258,76126096,3104056694]",
                "w2btAe": "%.@.null,null,\"0\",null,1]",
                "SNlM0e": "",
                "GWsdKe": "en-US",
                "d2zrDf": "%.@.]",
                "ocxFnb": "%.@.]",
                "zChJod": "%.@.]",
                "STfaRd": "{}",
                "LVIXXb": "2",
                "S06Grb": "",
                "S6lZl": "89978449",
                "QrtxK": "0",
                "oxN3nb": {
                    "1": false
                },
                "eptZe": "/wizrpcui/_/WizRpcUi/",
                "Ylvpqb": "%.@.\"multimodal-image-viewer\",null,null,null,1,null,null,null,null,null,null,null,\"en-US\",\"https://www.google.com\",null,1,null,null,null,null,null,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,null,null,null,null,null,0]"
            };
            window.IJ_values = {
                "A5tF3b": false,
                "zhkRO": "%.@.0,0,1,0,1,0,0,0,null,null,null,0,null,null,0,1,0,\"/setprefs?sig\\u003d0_7QCvYJ2iJw40tBfcNgG7LYMrLU0%3D\\u0026szl\\u003d0\",0,1,0]",
                "w2btAe": "%.@.null,null,\"0\",null,1]",
                "ea2VK": "%.@.1,1]",
                "pxO4Zd": "0",
                "mXOY5d": "%.@.null,2,2,null,[null,622,375]]",
                "SsQ4x": "IJ8UXppd3ETKm0PGR2OelA",
                "auIt8": "%.@.0,0]",
                "MuJWjd": false,
                "GWsdKe": "en-US",
                "N1ycab": "en_US",
                "BZUDzc": "%.@.1,\"14px\",\"500\",null,\"0 1px 1px rgba(0,0,0,.16)\",\"pointer\",null,null,null,\"#1f1f1f\",null,null,null,\"rgba(112,117,122,.40)\",\"#34a853\",\"#4285f4\",null,null,\"#fbbc04\",null,null,null,\"#34a853\",null,null,null,null,null,\"#1f1f1f\",null,null,0]",
                "MgUcDb": "US",
                "SIsrTd": false,
                "fyLpDc": "",
                "JPnTp": "%.@.\"#f1f3f4\",\"36px\"]",
                "ZxtPCd": "%.@.{\"100\":\"12px\",\"101\":\"8px\",\"102\":\"8px\",\"103\":\"12px\",\"104\":\"9px\"}]",
                "DwYRY": "%.@.\"#1967d2\",null,null,null,\"var(--bbQxAb)\",\"var(--mXZkqc)\",null,null,null,null,\"#47474761\",null,\"#1f1f1f33\",\"#1f1f1f66\",null,null,null,null,null,null,\"#0b57d015\",\"#0b57d015\",\"#0b57d039\",null,null,null,\"rgba(31,31,31,0.08)\",null,\"rgba(31,31,31,0.23921569)\"]",
                "NyzCwe": "%.@.null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,\"36px\",\"20px\",\"27px\",\"#202124\",\"16px\",\"var(--IXoxUe)\",\"var(--YLNNHc)\",\"var(--YLNNHc)\",\"var(--IXoxUe)\"]",
                "spz2q": "%.@.\"var(--xhUGwc)\",null,null,null,null,\"0px\"]",
                "geiQxc": "%.@.\"234px\",\"176px\",\"55vw\",\"172px\",\"128px\",\"40vw\",\"111px\",\"83px\",\"26vw\"]",
                "xFmcof": "%.@.null,null,null,\"16px\",null,\"8px\"]",
                "lDqiof": "%.@.\"var(--YLNNHc)\",\"var(--bbQxAb)\",\"var(--rrJJUc)\",null,\"var(--IXoxUe)\",\"var(--JKqx2)\",\"#681da8\",null,null,\"var(--xhUGwc)\",\"var(--Nsm0ce)\",\"var(--EpFNW)\",\"#e8f0fe\",\"#001d35\",\"#f1f3f4\",\"#001d35\",\"#fff\",\"#1f1f1f\",\"#1f1f1f\",\"#fff\",\"var(--ZXiaGf)\",\"var(--EoKObe)\",\"#146c2e\",\"#b3261e\",\"#b06000\",\"var(--mXZkqc)\",\"var(--cIehld)\",\"var(--gVUKcd)\",null,\"var(--gS5jXb)\",\"#d2e3fc\",null,\"var(--TMYS9)\",\"var(--amnPwe)\",null,\"transparent\",\"var(--ONhrGd)\",\"rgba(0,0,0,0.03)\",null,null,null,null,null,null,null,null,null,\"#ea4335\",\"#34a853\",\"#4285f4\",\"#fbbc04\",\"var(--ywz01c)\",\"#dadce0\",\"#f4f6f7\",null,null,null,null,\"#ebf1ff\",\"var(--XKMDxc)\",\"var(--aYn2S)\",null,\"var(--Aqn7xd)\",null,\"#b3261e\",\"#f9dedc\",\"#ffdf92\",\"#f4bf00\",\"#fff\",\"#410e0b\",\"#241a00\",\"#241a00\",\"#fff\",\"#072711\",\"#b3261e\",\"#146c2e\",\"#146c2e\",\"#c4eed0\",\"var(--nwXobb)\",\"var(--vZe0jb)\",\"var(--QWaaaf)\",\"var(--ZEpPmd)\",\"#a9acaa\",\"var(--Lm570b)\",\"#fff\",\"var(--jINu6c)\",\"var(--BRLwE)\",\"var(--DEeStf)\",\"var(--TSWZIb)\",\"var(--uLz37c)\",null,\"#545d7e\",\"var(--TyVYld)\",null,null,null,null,\"var(--VuZXBd)\",\"rgba(23,23,23,0.75)\",{\"100\":\"rgba(248,249,250,0.96)\",\"101\":\"rgba(255,255,255,0.8)\",\"102\":\"rgba(60,64,67,0.16)\",\"103\":\"#1558d6\",\"104\":\"#202124\",\"105\":\"#f1f3f4\",\"106\":\"#f1f3f4\",\"107\":\"var(--BRLwE)\",\"108\":\"var(--jINu6c)\",\"109\":\"var(--Xqboce)\",\"110\":\"var(--vdwxpe)\",\"111\":\"#1f1f1f\"}]",
                "Gpnz4c": "%.@.null,null,null,null,null,null,null,null,\"#001d35\",null,null,null,null,\"#1f1f1f\",null,\"#d2d2d2\",null,\"#0b57d0\",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,\"#0b57d0\",null,null,null,null,null,null,null,null,\"#e5edff\",\"#ecedee\"]",
                "kXVUzd": "%.@.\"40px\",\"48px\"]",
                "sCU50d": "%.@.null,\"none\",null,\"0px 1px 3px rgba(60,64,67,0.08)\",null,\"0px 2px 6px rgba(60,64,67,0.16)\",null,\"0px 4px 12px rgba(60,64,67,0.24)\",null,null,\"1px solid var(--mXZkqc)\",\"0\",\"0\",\"0\",null,\"0px 1px 3px rgba(60,64,67,0.24)\",\"0\"]",
                "w9Zicc": "%.@.\"#f1f3f4\",\"26px\",\"#e2eeff\",\"#0060f0\",\"#e2eeff\",\"1px\",\"#dadce0\",\"1px\",\"#fff\",\"#dadce0\",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,\"28px\",\"10px\",\"8px\",\"20px\",\"10px\",\"var(--XKMDxc)\",\"var(--mXZkqc)\",\"#fff\",\"var(--TSWZIb)\",\"var(--TMYS9)\",\"var(--TSWZIb)\",\"var(--TMYS9)\"]",
                "IkSsrf": "%.@.\"Google Sans,Roboto,sans-serif\",\"Google Sans,Roboto-Medium,sans-serif\",\"Roboto,sans-serif\",\"Roboto-Medium,sans-serif\",\"Roboto-Light,sans-serif\",\"sans-serif\",\"sans-serif\",\"Google Sans,Roboto,sans-serif\",\"sans-serif\",\"Roboto,sans-serif\"]",
                "MR0w4": "%.@.\"var(--google-fs,1)\",\"var(--google-screen-scale-ratio,1)\"]",
                "OItNqf": "%.@.null,\"20px\"]",
                "JMyuH": "%.@.]",
                "j2FoS": "%.@.null,null,null,null,null,null,null,null,\"20px\"]",
                "e2zoW": "%.@.\"16px\",\"12px\",\"0px\",\"8px\",\"4px\",\"2px\",\"20px\",\"24px\",\"48px\",\"8px\",null,null,\"16px\",\"8px\",\"36px\",\"8px\",null,\"83px\",\"52px\",\"6px\",\"20px\",\"18px\",\"16px\",\"24px\",\"12px\",\"6px\",\"75px\",\"52px\",\"799.98px\",\"684px\",\"683.98px\",\"1163.98px\",\"940px\",\"1560px\",\"1164px\",\"939.98px\",\"800px\",\"82px\"]",
                "W1Bte": "%.@.\"cubic-bezier(0.1,1,0.2,1)\",\"cubic-bezier(0.8,0,1,0.8)\",\"cubic-bezier(0.2,0.6,0.2,1)\",\"cubic-bezier(0.4,0,1,0.8)\",\"300ms\",\"100ms\",\"200ms\",\"250ms\",\"cubic-bezier(0.4,0,0.2,1)\",\"cubic-bezier(0.4,0,0.6,1)\",\"cubic-bezier(0.6,0,0,1)\",\"cubic-bezier(0,0,1,1)\",\"cubic-bezier(0.2,0,0,1)\",\"800ms\",\"1000ms\",\"400ms\",\"500ms\",\"600ms\",\"50ms\",\"400ms\",\"300ms\",\"250ms\",\"150ms\",\"250ms\",\"200ms\",\"150ms\",\"150ms\",\"300ms\",\"250ms\",\"200ms\",\"150ms\",\"450ms\",\"450ms\",\"300ms\",\"150ms\",\"300ms\",\"250ms\",\"200ms\",\"100ms\",\"250ms\",\"200ms\",\"150ms\",\"100ms\",\"250ms\",\"200ms\",\"150ms\",\"100ms\",\"300ms\",\"250ms\",\"200ms\",\"100ms\",\"null\",\"cubic-bezier(0.3,0,0.8,0.15)\",\"cubic-bezier(0.05,0.7,0.1,1)\",\"cubic-bezier(0,0,1,1)\",\"cubic-bezier(0.2,0,0,1)\",\"cubic-bezier(0.3,0,1,1)\",\"cubic-bezier(0,0,0,1)\",\"250ms\",\"200ms\",\"150ms\",\"50ms\",\"50ms\",\"50ms\",\"400ms\",\"350ms\",\"250ms\",\"50ms\",\"50ms\",\"50ms\",\"200ms\",\"150ms\",\"100ms\",\"50ms\",\"200ms\",\"150ms\",\"100ms\",\"50ms\",\"50ms\",\"50ms\",\"250ms\",\"200ms\",\"150ms\",\"50ms\",\"50ms\",\"50ms\",\"cubic-bezier(0.05,0.7,0.1,1)\",\"cubic-bezier(0.3,0,0.8,0.15)\"]",
                "pbvshf": "%.@.\"48px\"]",
                "u9mep": "%.@.\"var(--COEmY)\",null,\"var(--YLNNHc)\",\"var(--JKqx2)\"]",
                "mrqaQb": "%.@.null,null,null,null,\"2px\",\"12px\"]",
                "Umq10e": "%.@.\"#fff\"]",
                "k7Tqye": "%.@.null,null,null,null,null,null,null,\"16px\",\"12px\",\"8px\",\"20px\",\"4px\",\"9999px\",\"0px\",\"2px\"]",
                "y50LC": "%.@.null,null,\"#4d5156\",null,\"#3c4043\"]",
                "jfSEkd": "%.@.\"var(--Pa8Wlb)\",\"var(--KIZPne)\",\"var(--xPpiM)\",null,\"#d2e3fc\",\"#d2e3fc\",\"rgba(26,115,232,0.24)\",\"var(--pEa0Bc)\",\"var(--Yi4Nb)\",\"var(--kloG3)\",\"var(--YaIeMb)\",\"var(--izGsqb)\",\"var(--todMNcl)\",\"rgba(32,33,36,0.16)\",null,\"rgba(32,33,36,0.32)\",\"#ebf1ff\",\"#001d35\",\"rgba(32,33,36,0.08)\",\"rgba(32,33,36,0.08)\",\"rgba(32,33,36,0.24)\",\"transparent\",\"var(--rrJJUc)\",null,\"rgba(26,115,232,0.08)\",\"rgba(26,115,232,0.08)\",\"rgba(26,115,232,0.24)\",null,null,null,\"rgba(60,64,67,0.08)\",\"rgba(60,64,67,0.08)\",\"rgba(60,64,67,0.24)\",\"2px\",\"2px\",\"61\",\"var(--bbQxAb)\",\"var(--BRLwE)\",null,\"#0b57d0\",\"#f1f3f4\",\"var(--YLNNHc)\",\"var(--amnPwe)\",\"var(--YLNNHc)\",\"var(--YLNNHc)\",\"15\",\"0\",\"39\",\"var(--jINu6c)\",\"rgba(32,33,36,0.16)\",\"33\",\"39\",\"var(--EpFNW)\",\"66\",\"#001d35\",\"var(--amnPwe)\",\"0.08\",\"var(--bbQxAb)\",\"0.08\",\"var(--amnPwe)\",\"var(--bbQxAb)\",\"0.24\",\"var(--IXoxUe)\",\"transparent\",\"0.08\",\"0.08\",\"var(--IXoxUe)\",\"0.24\",\"var(--IXoxUe)\",\"transparent\",\"0.08\",\"0.08\",\"var(--uLz37c)\",\"0.24\",\"var(--uLz37c)\",\"var(--jINu6c)\",\"var(--jINu6c)\",\"15\",\"#1967d2\",\"15\",\"#0b57d0\",\"39\",\"#0b57d0\",\"15\",\"15\",\"#0b57d0\",\"#001d35\",\"39\",\"var(--p9J9c)\",\"var(--jINu6c)\",\"15\",\"15\",\"39\",\"#0b57d0\",\"rgba(0,0,0,.6)\",\"33\",\"33\",\"var(--EoKObe)\",\"66\",{\"100\":\"#fff\",\"101\":\"0.08\",\"102\":\"0.08\",\"103\":\"#1f1f1f\",\"104\":\"0.24\",\"105\":\"#1f1f1f\",\"106\":\"rgba(0,0,0,0.1)\",\"107\":\"0.08\",\"108\":\"#185abc\",\"109\":\"0.08\",\"110\":\"0.24\",\"111\":\"var(--Nsm0ce)\",\"112\":\"0\",\"113\":\"transparent\",\"114\":\"0.08\",\"115\":\"0.08\",\"116\":\"var(--YLNNHc)\",\"117\":\"0.24\",\"118\":\"var(--YLNNHc)\",\"119\":\"0.75\",\"120\":\"0.6\",\"121\":\"rgba(31,31,31,0.078431375)\",\"122\":\"rgba(31,31,31,0.08)\",\"123\":\"rgba(31,31,31,0.23921569)\",\"124\":\"rgba(11,87,208,0.078431375)\",\"125\":\"rgba(11,87,208,0.078431375)\",\"126\":\"rgba(11,87,208,0.23921569)\",\"127\":\"rgba(32,33,36,0.2)\",\"128\":\"rgba(32,33,36,0.2)\",\"129\":\"rgba(32,33,36,0.4)\",\"130\":\"rgba(32,33,36,0.078431375)\",\"131\":\"rgba(32,33,36,0.078431375)\",\"132\":\"rgba(32,33,36,0.23921569)\",\"133\":\"var(--Ehh4mf)\",\"134\":\"var(--Nsm0ce)\"}]",
                "GVtPm": "%.@.null,null,null,null,null,\"0 0 0 1px var(--mXZkqc)\",\"1px solid var(--mXZkqc)\",null,null,null,null,null,null,null,null,\"transparent\",\"rgba(23,23,23,0.3)\",null,null,\"16px\"]",
                "MexNte": "%.@.\"700\",\"400\",\"underline\",\"none\",\"capitalize\",\"none\",\"uppercase\",\"none\",\"500\",\"lowercase\",\"italic\",null,null,\"-1px\",\"0.3px\",\"20px\",\"12px\",null,\"12px\",\"14px\",\"16px\",\"18px\",\"22px\",\"24px\",\"26px\",\"28px\",\"32px\",\"36px\",\"40px\",\"48px\",\"52px\",\"56px\",\"60px\",\"14px\",\"16px\",\"18px\",\"20px\",\"22px\",\"24px\",\"28px\",\"36px\",\"40px\",\"45px\",\"48px\",\"56px\",\"0\",\"0.1px\"]",
                "Aahcnf": "%.@.\"24px\",\"32px\",\"500\",\"Google Sans,Roboto-Medium,sans-serif\",null,\"Roboto,sans-serif\",\"14px\",\"400\",\"20px\",null,\"18px\",\"24px\",\"500\",\"Google Sans,Roboto-Medium,sans-serif\",null,\"Google Sans,Roboto,sans-serif\",\"52px\",\"45px\",\"0\",null,\"400\",\"Google Sans,Roboto,sans-serif\",\"36px\",\"400\",\"48px\",null,\"Google Sans,Roboto,sans-serif\",\"32px\",\"24px\",null,\"400\",null,\"Roboto,sans-serif\",\"24px\",\"18px\",null,\"400\",\"Roboto,sans-serif\",\"16px\",\"12px\",null,\"400\",\"Roboto,sans-serif\",\"24px\",\"16px\",null,\"400\",\"Google Sans,Roboto,sans-serif\",\"26px\",\"20px\",null,\"400\",\"Google Sans,Roboto,sans-serif\",\"20px\",\"16px\",null,\"400\",\"Roboto,sans-serif\",\"18px\",\"14px\",null,\"400\",null,null,null,null,null,\"14px\",\"Google Sans,Roboto-Medium,sans-serif\",\"20px\",\"500\",\"Google Sans,Roboto,sans-serif\",\"32px\",\"24px\",\"400\",\"Google Sans,Roboto,sans-serif\",\"24px\",\"18px\",\"400\",\"Roboto,sans-serif\",\"16px\",\"12px\",\"Google Sans,Roboto-Medium,sans-serif\",\"26px\",\"20px\",\"500\",null,null,null,null,\"500\",\"0px\",\"0px\",\"0\"]",
                "PFhmed": "%.@.null,null,null,null,\"#f1f3f4\"]",
                "B4pZbd": "US",
                "o0P8Hf": "%.@.null,null,null,null,null,null,null,null,null,\"#f8f9fa\",\"#000\",null,\"#dadce0\",null,\"#fff\",null,null,\"rgba(0,0,0,0.26)\",null,null,null,null,\"rgba(0,0,0,0.1)\",\"#fff\",null,null,null,null,null,null,\"rgba(255,255,255,0.5)\",null,\"rgba(0,0,0,0.3)\",\"rgba(0,0,0,0.2)\",null,null,null,null,null,\"#70757a\",\"#70757a\",null,\"rgba(0,0,0,.30)\",null,null,\"#d2e3fc\",null,\"rgba(32,33,36,.7)\",null,null,null,null,null,null,null,null,null,\"rgba(0,0,0,.12)\",null,null,\"rgba(0,0,0,.24)\",\"rgba(0,0,0,.05)\",null,\"rgba(60,64,67,.3)\",null,\"rgba(0,0,0,.15)\",\"rgba(32,33,36,.28)\",null,\"#dadce0\",\"#fff\",null,\"#1a73e8\",\"#000\",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,\"rgba(248,249,250,0.85)\",\"#dadce0\",\"#ea4335\",{\"112\":\"#dadce0\",\"114\":\"#4285f4\",\"117\":\"#4285f4\",\"118\":\"#1a73e8\",\"127\":\"#d2e3fc\",\"130\":\"#fce8e6\",\"131\":\"#fad2cf\",\"140\":\"#d93025\",\"147\":\"#ea4335\",\"150\":\"#fef7e0\",\"187\":\"#34a853\",\"188\":\"#1e8e3e\",\"189\":\"#188038\",\"190\":\"#137333\",\"192\":\"rgba(0,0,0,.1)\",\"196\":\"rgba(255,255,255,0)\",\"198\":\"rgba(32,33,36,0)\",\"199\":\"rgba(32,33,36,.1)\",\"200\":\"rgba(0,0,0,.12)\",\"204\":\"rgba(255,255,255,.5)\",\"211\":\"rgba(0,0,0,.02)\",\"212\":\"#000\",\"214\":\"rgba(0,0,0,.7)\",\"218\":\"rgba(0,0,0,.15)\",\"219\":\"rgba(0,0,0,.05)\",\"221\":\"#dadce0\",\"227\":\"rgba(0,0,0,.05)\",\"231\":\"rgba(255,255,255,.8)\",\"232\":\"rgba(0,0,0,.05)\",\"236\":\"rgba(0,0,0,.87)\",\"244\":\"rgba(255,255,255,.54)\",\"246\":\"rgba(0,0,0,.26)\",\"249\":\"rgba(0,0,0,.03)\",\"253\":\"rgba(0,0,0,0)\",\"259\":\"#4285f4\",\"273\":\"rgba(60,64,67,.3)\"}]",
                "rkD25": "%.@.[[\"hl\",\"en-US\"]]]",
                "WiLzZe": "%.@.\"#202124\",\"#70757a\",null,null,\"#fff\",\"rgba(255,255,255,.70)\",28,24,20,20,16,-2,-1,-4,2,1,0,24,20,16,14,12]",
                "rNyuJc": "",
                "LU5fGb": false,
                "gXkHoe": "105250506097979753968",
                "hevonc": "%.@.2]"
            };
        })();
        (function() {
            google.xjs = {
                basecomb: '/xjs/_/js/k\x3dxjs.hm.en.u6SkHWMIN6A.2018.O/ck\x3dxjs.hm.YxLWi9QlUZA.L.B1.O/am\x3dIAQoAQgAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAQAAAAAAAAACAQEgDBAKwAAAAAYEAChAAAAAAAAFAAAhEAEAAAgAAAAAAAQCAGIArggAAAAXDQAAgQgRAAIGBQAAAIAQIEAAQCYBGgAFAEh4B8AIzCQYEgGwAWgAQAACAgegBECCAAXeRgUBiIdAPIAIiAAAAAAAAAQIhAACAAAkAAAAQFAuKZgAQAAAEYAgAgDIAAYgIAAAAAAEAAAAHABgBEAMOAQRQIAAAAAAACQA4AHAkNqCgAAAAAAAAAAAAAAAAAAAOAAQ3AGAQQQAAAAAAAAAAAAAAAAAAAAAAAACWFXAwI/d\x3d1/ed\x3d1/dg\x3d0/br\x3d1/ujg\x3d1/rs\x3dACT90oElwdv2wJYloW7LcRi-dVh2LPlFXg',
                basecss: '/xjs/_/ss/k\x3dxjs.hm.YxLWi9QlUZA.L.B1.O/am\x3dIAQoAQgAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAQAAAAAAAAACAQAgDAACwAAAAAYAAChAAAAAAAAFAAAAEAAAAAgAAAAAAAQAAGIALggAAAAWDQAAgQgRAAAEBQAAAIAQIEAAQCYAGgAFAEgIAAAADCQYAAGAASAAQAAAAAegBECCAAXeRgUBiIdAPIAIiAAAAAAAAAQAhAACAAAEAAAAQFAuKZgAQAAAEYAgAgCIAAYgIAAAAAAEAAAADABgAEAMOAQRQIAAAAAAAAAAgAAAABgAAAAAAAAAAAAAAAAAAAAAOA/br\x3d1/rs\x3dACT90oElhrQ7usnLRpCaml2ZrYQR-0ylBQ',
                basejs: '/xjs/_/js/k\x3dxjs.hm.en.u6SkHWMIN6A.2018.O/am\x3dAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAACAAEAABAIAAAAAAAEAAAAAAAAAAAAAAAhAAEAAAAAAAAAAAACAAAAgAAAAAADBAAAAQgRAAIGAQAAAAAAAAAAAAIBCAAAAABwB8AIwAAAEgAwAWgAQAACAgAAAACAAAAAQAAAgAAAAAAAgAAAAAAAAAAIAAACAAAgAAAAAAAAAAAAAAAAAAAAAADAAAAgAAAAAAAAAAAAHAAgBAAAGAQRQIAAAAAAACQA4AHAkNqCgAAAAAAAAAAAAAAAAAAAOAAQ3AGAQQQAAAAAAAAAAAAAAAAAAAAAAAACWFXAwI/dg\x3d0/br\x3d1/rs\x3dACT90oH0Z4SrkdR6Pg9nIpVk5_BiJAH_9w',
                excm: []
            };
        })();
    </script>
    <link href="/xjs/_/ss/k=xjs.hm.YxLWi9QlUZA.L.B1.O/am=IAQoAQgAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAQAAAAAAAAACAQAgDAACwAAAAAYAAChAAAAAAAAFAAAAEAAAAAgAAAAAAAQAAGIALggAAAAWDQAAgQgRAAAEBQAAAIAQIEAAQCYAGgAFAEgIAAAADCQYAAGAASAAQAAAAAegBECCAAXeRgUBiIdAPIAIiAAAAAAAAAQAhAACAAAEAAAAQFAuKZgAQAAAEYAgAgCIAAYgIAAAAAAEAAAADABgAEAMOAQRQIAAAAAAAAAAgAAAABgAAAAAAAAAAAAAAAAAAAAAOA/d=1/ed=1/br=1/rs=ACT90oElhrQ7usnLRpCaml2ZrYQR-0ylBQ/m=bct,cdos,hsm,jsa,qim,rr4R9e,cEt90b,SNUn3,qddgKe,sTsDMc,dtl0hd,eHDfl,YV5bee,d,csi"
        rel="stylesheet" nonce="IJ8UXppd3ETKm0PGR2OelA">
    <script nonce="IJ8UXppd3ETKm0PGR2OelA">
        (function() {
            google.kEXPI = '0,4003510,146403,104244,64701,68090891,7385,53222,52042,36709,26338,6748,190478,44166,393,22134,12654,1672,26806,16951,1270,8141,7355,5587,16207,410,2905,6762,7197,17292,2275,1899,21514,6378,1344,125,144,8,2766,10335,4822,5111,5474,1133,3705,2024,8603,1884,3608,1163,6721,6208,5990,1005,108,2988,856,3699,501,1070,2072,4128,451,170,619,591,535,55,846,481,1510,1156,1539,131,1010,1392,234,390,2641,2203,236,41,1212,1005,207,73,25,73,677,162,431,32,573,2106,1252,322,804,782,424,255,205,236,247,1777,958,68,1224,812,172,34,272,246,270,2712,98,170,156,485,129,950,799,20702051,421512,3181,15723,3851,4991,68,263,1721,8482225,573663';
        })();
        (function() {
            var u = '/xjs/_/js/k\x3dxjs.hm.en.u6SkHWMIN6A.2018.O/am\x3dAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAACAAEAABAIAAAAAAAEAAAAAAAAAAAAAAAhEAEAAAAAAAAAAAACAAAAgAAAAAADBAAAAQgRAAIGAQAAAAAAAAAAAAIBCAAAAABwB8AIwAAAEgAwAWgAQAACAgAAAACAAAAAQAAAgAAAAAAAgAAAAAAAAAAIAAACAAAgAAAAAAAAAAAAAAAAAAAAAADAAAAgAAAAAAAAAAAAHAAgBAAAGAQRQIAAAAAAACQA4AHAkNqCgAAAAAAAAAAAAAAAAAAAOAAQ3AGAQQQAAAAAAAAAAAAAAAAAAAAAAAACWFXAwI/d\x3d1/ed\x3d1/dg\x3d4/br\x3d1/rs\x3dACT90oGKVjbcRHAJ8zOyEr5u7XxHaDuKvg/ee\x3dALeJib:B8gLwd;AfeaP:TkrAjf;BMxAGc:E5bFse;BgS6mb:fidj5d;BjwMce:cXX2Wb;CIZTGb:Kqhykb;CxXAWb:YyRLvc;DQEued:Fevhcf;DULqB:RKfG5c;Dkk6ge:JZmW9e;DpcR3d:zL72xf;Du7NI:C6zLgf;EABSZ:MXZt9d;EVNhjf:pw70Gc;EjXHpb:pSHqh;EmZ2Bf:zr1jrb;EnlcNd:WeHg4;F9mqte:rlHKFc;FSxmUe:fiZR8b;Fmv9Nc:O1Tzwc;G0KhTb:LIaoZ;GEkGdd:e1RzQd;GleZL:J1A7Od;HMDDWe:G8QUdb;IBADCc:RYquRb;IoGlCf:b5lhvb;JXJSm:ii1RGf;JXS8fb:k0T3Ub;JqSq7d:y9ePhe;JsbNhc:Xd8iUd;K5nYTd:ZDZcre;KOxcK:OZqGte;KQzWid:ZMKkN;KcokUb:KiuZBf;KpRAue:Tia57b;LBgRLc:SdcwHb,XVMNvd;LEikZe:byfTOb,lsjVmc;LXA8b:q7OdKd;LsNahb:ucGLNb;NJ1rfe:qTnoBf;NPKaK:SdcwHb;NSEoX:lazG7b;Np8Qkd:Dpx6qc;Nyt6ic:jn2sGd;OgagBe:cNTe0;OiqE2c:TFpEK;OoK5v:Sp69O;OohIYe:mpEAQb;Pjplud:PoEs9b;PpTLXd:pJYjx;Q1Ow7b:x5CSu;QGR0gd:Mlhmy;QYLF2b:pAQYUd;Qw8Feb:jpavUe;R4IIIb:QWfeKf;R9Ulx:CR7Ufe;RCF5Sd:X1kBmd;SLtqO:Kh1xYe;SMDL4c:fTfGO,fTfGO;SNUn3:ZwDk9d,xD8Kp;ScI3Yc:e7Hzgb,e7Hzgb;ShpF6e:N0pvGc;SwCqAd:fXbCZc;SzQQ3e:dNhofb;TIUVQd:M8uRWb;TroZ1d:vVVzjb;U96pRd:FsR04;UBKJZ:LGDJGb;UDrY1c:eps46d;UVmjEd:EesRsb;UVzb9c:IvPZ6d;UYRIEb:HzTAQc;UyG7Kb:wQd0G;V2HTTe:RolTY;VGRfx:VFqbr;VN6jIc:ddQyuf;VOcgDe:YquhTb;VhA7bd:vAmQFf;VsAqSb:PGf2Re;WCEKNd:I46Hvd;WDGyFe:jcVOxd;Wfmdue:g3MJlb;Y3c5sd:FGbfLe;YIZmRd:A1yn5d;YV5bee:IvPZ6d;ZSH6tc:QAvyLe;ZWEUA:afR4Cf;ZlOOMb:P0I0Ec;a56pNe:JEfCwb;aAJE9c:WHW6Ef;aCJ9tf:qKftvc;aVZq3e:EMeVIb;aZ61od:arTwJ;bDXwRe:UsyOtc;bcPXSc:gSZLJb;cEt90b:ws9Tlc;cFTWae:gT8qnd;coJ8e:KvoW8;dIoSBb:ZgGg9b;dLlj2:Qqt3Gf;dowIGb:ebZ3mb,ebZ3mb;dtl0hd:lLQWFe;eBAeSb:Ck63tb;eBZ5Nd:audvde;eHDfl:ofjVkb;eJKchc:ATg1be;eO3lse:UefOmb;euOXY:OZjbQ;g8nkx:U4MzKc;gaub4:TN6bMe;gtVSi:ekUOYd;h3MYod:ws9Tlc;hK67qb:QWEO5b;hVic1b:Kqhykb;heHB1:sFczq;hjRo6e:F62sG;hlqGX:FWz1ic;hsLsYc:Vl118;hwoVHd:zw4U8c;iFQyKf:QIhFr;iySzae:a6xXfd;jJj2G:kF2o2b;k2Qxcb:XY51pe;kbAm9d:MkHyGd;lOO0Vd:OTA3Ae;lbfkyf:MqGdUd;liAz7d:kF2o2b;lkq0A:JyBE3e;mWzs9c:fz5ukf;nJw4Gd:dPFZH;nrDcw:SuEoDe;oGtAuc:sOXFj;oSUNyd:fTfGO,fTfGO;oUlnpc:BO43gd;okUaUd:wItadb;pKJiXd:VCenhc;pNsl2d:gsJLOc,j9Yuyc,kJ6Ooc,wJMPhe;pXdRYb:JKoKVe;pj82le:ww04Df;qGV2uc:HHi04c;qZx2Fc:j0xrE;qaS3gd:yiLg6e;qafBPd:sgY6Zb;qavrXe:zQzcXe;qddgKe:d7YSfd,x4FYXe;rmWaj:PMS6Sd;ropkZ:hjoqoe;sTsDMc:kHVSUb;sZmdvc:rdGEfc;slIQ5d:pnOULd;tH4IIe:Ymry6;tbg2ob:nMC03e;teSRSb:BMLai;tosKvd:ZCqP3;trZL0b:qY8PFe;uuQkY:u2V3ud;vEYCNb:FaqsVd;vRlMvf:Iw9Xo;vfVwPd:lcrkwe;w3bZCb:ZPGaIb;w4rSdf:XKiZ9;w9w86d:dt4g2b;wQlYve:aLUfP;wR5FRb:O1Gjze,TtcOte;wV5Pjc:L8KGxe;xBbsrc:NEW1Qc;ysNiMc:CpIBjd;yxTchf:KUM7Z;z97YGf:oug9te;zB3wie:WtgrP;zOsCQe:Ko78Df;zaIgPb:Sl0pxd/m\x3dbct,cdos,hsm,jsa,qim,rr4R9e,cEt90b,SNUn3,qddgKe,sTsDMc,dtl0hd,eHDfl,YV5bee,d,csi';
            var st = 1;
            var amd = 1000;
            var mmd = 0;
            var pop = true;
            var povp = false;
            var fp = '';
            var e = this || self;

            function f() {
                var a = google.ia && google.ia.r.B2Jtyd;
                return a && [1, 5, 6].indexOf(a.m) >= 0 && a.cbfd && a.cbvi ? a : void 0
            };

            function g(a) {
                var b = document.createElement("link");
                b.as = "script";
                b.href = a;
                b.rel = "preload";
                document.body.appendChild(b)
            }

            function k() {
                var a = [u];
                google.dp || (a.forEach(g), google.dp = !0)
            };
            google.ps = google.ps || [];

            function l() {
                var a = u,
                    b = function() {};
                google.lx = google.stvsc ? b : function() {
                    m(a);
                    google.lx = b
                };
                google.bx || google.lx()
            }

            function n(a, b) {
                b && (a.src = b);
                fp && google.caft && google.caft(function() {
                    a.fetchPriority = fp
                });
                var h = a.onload;
                a.onload = function(d) {
                    h && h(d);
                    google.ps = google.ps.filter(function(c) {
                        return a !== c
                    })
                };
                google.ps.push(a);
                document.body.appendChild(a)
            }
            google.as = n;

            function m(a) {
                google.tick && google.tick("load", "xjsls");
                var b = document.createElement("script");
                b.onerror = function() {
                    google.ple = 1
                };
                b.onload = function() {
                    google.ple = 0
                };
                google.xjsus = void 0;
                n(b, a);
                google.aple = -1;
                google.dp = !0
            };

            function p() {
                for (var a = document.getElementsByTagName("img"), b = 0, h = a.length; b < h; b++) {
                    var d = a[b],
                        c;
                    if (c = d.hasAttribute("data-lzy_") && Number(d.getAttribute("data-atf")) & 1) c = d.getAttribute("jscontroller"), c = !((c === "UBXHI" || c === "R3fhkb" || c === "TSZEqd") && d.hasAttribute("data-src"));
                    if (c) return !0
                }
                return !1
            };
            var q, r, t, w, x, y, z, A, B;

            function C() {
                google.xjsu = u;
                e._F_jsUrl = u;
                x = function() {
                    l()
                };
                q = !1;
                r = (st === 1 || st === 3) && !!google.caft && !p();
                t = f();
                w = (st === 2 || st === 3) && !!t && !p();
                y = pop;
                z = povp;
                A = y && document.prerendering || z && document.hidden;
                B = z ? "visibilitychange" : "prerenderingchange"
            }

            function D() {
                q || r || w || A || (x(), q = !0)
            }
            setTimeout(function() {
                google.tick && google.tick("load", "xjspls");
                C();
                if (r || w || A) {
                    if (r) {
                        var a = function() {
                            r = !1;
                            D()
                        };
                        google.caft(a);
                        setTimeout(a, amd)
                    }
                    w && (a = function() {
                        w = !1;
                        D()
                    }, t.cbvi.push(a), setTimeout(a, mmd));
                    if (A) {
                        var b = function() {
                            (z ? document.hidden : document.prerendering) || (A = !1, D(), document.removeEventListener(B, b))
                        };
                        document.addEventListener(B, b, {
                            passive: !0
                        })
                    }
                    q || k()
                } else x()
            }, 0);
        })();
        window._ = window._ || {};
        window._DumpException = _._DumpException = function(e) {
            throw e;
        };
        window._s = window._s || {};
        _s._DumpException = _._DumpException;
        window._qs = window._qs || {};
        _qs._DumpException = _._DumpException;
        (function() {
            var t = [19399712, 32, 0, 16384, 0, 0, 0, 0, 0, 270532608, 0, 0, 265224, 11342018, 0, 677381126, 533504, 5242894, 67649728, 134217729, 8192, 77611008, 192948352, 8, 132134268, 152076304, 21078720, 121669633, 331479872, 949411840, 403783766, 522248, 550547491, 386947361, 38844928, 128059904, 134361474, 568613888, 438329665, 16691223, 8226, 419561504, 16920832, 2162211, 16793232, 450953220, 5642, 1409024, 587401344, 34825232, 9224, 33555460, 50950162, 13107294, 2380046, 0, 61865984, 201858688, 46756, 0, 0, 0, 234881024, 27005124, 1049601, 0, 0, 0, 0, 69468160, 2637174];
            window._F_toggles = window._xjs_toggles = t;
        })();
        window._F_installCss = window._F_installCss || function(css) {};
    </script> <span jscontroller="oSkgIf" style="display:none" jsaction="rcuQ6b:npT2md"></span>
    <script nonce="IJ8UXppd3ETKm0PGR2OelA">
        (function() {
            var deia = false;
            window.jsl = window.jsl || {};
            window.jsl.dh = function(b, f, c) {
                try {
                    var a = document.getElementById(b),
                        d;
                    !a && ((d = google.stvsc) == null ? 0 : d.dds) && google.stvsc.dds.forEach(function(k) {
                        a || (a = k.getElementById(b))
                    });
                    if (!a) {
                        var l = document.querySelectorAll("template");
                        for (d = 0; d < l.length && !(a = l[d].content.getElementById(b)); d++);
                    }
                    if (a) a.innerHTML = f, c && c(), deia && (google.delt = google.delt || {}, google.delt[a.id] = Date.now());
                    else if (!(google.ia && google.ia.rids && google.ia.rids.has(b))) {
                        var e = {
                            id: b,
                            script: String(!!c),
                            milestone: String(google.jslm || 0)
                        };
                        google.jsla && (e.async = google.jsla);
                        var m = b.indexOf("_"),
                            g = m > 0 ? b.substring(0, m) : "",
                            n = document.createElement("div");
                        n.innerHTML = f;
                        var h = n.children[0];
                        if (h && (e.tag = h.tagName, e["class"] = String(h.className || null), e.name = String(h.getAttribute("jsname")), g)) {
                            f = [];
                            var p = document.querySelectorAll('[id^="' + g + '_"]');
                            for (c = 0; c < p.length; ++c) f.push(p[c].id);
                            e.ids = f.join(",")
                        }
                        google.ml(Error(g ? "Missing ID with prefix " + g : "Missing ID"), !1, e)
                    }
                } catch (k) {
                    google.ml(k, !0, {
                        "jsl.dh": !0
                    })
                }
            };
        })();
        (function() {
            var x = true;
            google.jslm = x ? 2 : 1;
        })();
        google.x(null, function() {
            (function() {
                function d(c, b) {
                    var a = document.createElement("div");
                    for (a.innerHTML = b; a.firstChild;) b = a.firstChild, a.firstChild.tagName && a.firstChild.tagName.toLowerCase() == "script" && (b = document.createElement("script"), b.innerHTML = a.firstChild.innerHTML, a.removeChild(a.firstChild)), c.appendChild(b)
                }
                google.mhp = {};
                google.mhp.processMobileHpJson = function(c) {
                    var b = c.promo;
                    if (b) {
                        var a = document.getElementById("promo");
                        a && d(a, b)
                    }(c = c.pushdown_promo) && (b = document.getElementById("mpd")) && d(b, c)
                };
            })();
            (function() {
                (function() {
                    var mhpd_json = {
                        show_feed: true
                    };
                    google.mhp.processMobileHpJson(mhpd_json);
                })();
            })();
            (function() {
                window.jsl = window.jsl || {};
                window.jsl.dh = window.jsl.dh || function(i, c, d) {
                    try {
                        var e = document.getElementById(i);
                        if (e) {
                            e.innerHTML = c;
                            if (d) {
                                d();
                            }
                        } else {
                            if (window.jsl.el) {
                                window.jsl.el(new Error('Missing ID.'), {
                                    'id': i
                                });
                            }
                        }
                    } catch (e) {
                        if (window.jsl.el) {
                            window.jsl.el(new Error('jsl.dh'));
                        }
                    }
                };
            })();
            (function() {
                window.jsl.dh('_v8HjaMKuBpCvptQP9sqQyAs_9', '\x3cg-menu jsname\x3d\x22xl07Ob\x22 class\x3d\x22cF4V5c lA8P5e yTik0 PBn44e iQXTJe wplJBd\x22 jscontroller\x3d\x22WlNQGd\x22 role\x3d\x22menu\x22 tabindex\x3d\x22-1\x22 jsaction\x3d\x22PSl28c;focus:h06R8;keydown:uYT2Vb;mouseenter:WOQqYb;mouseleave:Tx5Rb;mouseover:IgJl9c\x22\x3e\x3cg-menu-item jsname\x3d\x22NNJLud\x22 jscontroller\x3d\x22CnSW2d\x22 class\x3d\x22EpPYLd Zt0a5e\x22 role\x3d\x22none\x22 data-short-label\x3d\x22\x22 jsdata\x3d\x22zPXzie;_;v8HjaMKuBpCvptQP9sqQyAs5\x22\x3e\x3cdiv jsname\x3d\x22ibnC6b\x22 class\x3d\x22YpcDnf OSrXXb HG1dvd\x22 role\x3d\x22none\x22\x3e\x3ca href\x3d\x22https://www.google.com/preferences?hl\x3den\x26amp;fg\x3d1\x22 role\x3d\x22menuitem\x22\x3eSearch settings\x3c/a\x3e\x3c/div\x3e\x3c/g-menu-item\x3e\x3cg-menu-item jsname\x3d\x22NNJLud\x22 jscontroller\x3d\x22CnSW2d\x22 class\x3d\x22EpPYLd Zt0a5e\x22 role\x3d\x22none\x22 data-short-label\x3d\x22\x22 jsdata\x3d\x22zPXzie;_;v8HjaMKuBpCvptQP9sqQyAs5\x22\x3e\x3cdiv jsname\x3d\x22ibnC6b\x22 class\x3d\x22YpcDnf OSrXXb HG1dvd\x22 role\x3d\x22none\x22\x3e\x3ca href\x3d\x22/advanced_search?hl\x3den\x26amp;fg\x3d1\x22 role\x3d\x22menuitem\x22\x3eAdvanced search\x3c/a\x3e\x3c/div\x3e\x3c/g-menu-item\x3e\x3cg-menu-item jsname\x3d\x22NNJLud\x22 jscontroller\x3d\x22CnSW2d\x22 class\x3d\x22EpPYLd Zt0a5e\x22 role\x3d\x22none\x22 data-short-label\x3d\x22\x22 jsdata\x3d\x22zPXzie;_;v8HjaMKuBpCvptQP9sqQyAs5\x22\x3e\x3cdiv jsname\x3d\x22ibnC6b\x22 class\x3d\x22YpcDnf OSrXXb HG1dvd\x22 role\x3d\x22none\x22\x3e\x3ca href\x3d\x22/history/privacyadvisor/search/unauth?utm_source\x3dgooglemenu\x26amp;fg\x3d1\x26amp;cctld\x3dcom\x22 role\x3d\x22menuitem\x22\x3eYour data in Search\x3c/a\x3e\x3c/div\x3e\x3c/g-menu-item\x3e\x3cg-menu-item jsname\x3d\x22NNJLud\x22 jscontroller\x3d\x22CnSW2d\x22 class\x3d\x22EpPYLd Zt0a5e\x22 role\x3d\x22none\x22 data-short-label\x3d\x22\x22 jsdata\x3d\x22zPXzie;_;v8HjaMKuBpCvptQP9sqQyAs5\x22\x3e\x3cdiv jsname\x3d\x22ibnC6b\x22 class\x3d\x22YpcDnf OSrXXb HG1dvd\x22 role\x3d\x22none\x22\x3e\x3ca href\x3d\x22/history/optout?hl\x3den\x26amp;fg\x3d1\x22 role\x3d\x22menuitem\x22\x3eSearch history\x3c/a\x3e\x3c/div\x3e\x3c/g-menu-item\x3e\x3cg-menu-item jsname\x3d\x22NNJLud\x22 jscontroller\x3d\x22CnSW2d\x22 class\x3d\x22EpPYLd Zt0a5e\x22 role\x3d\x22none\x22 data-short-label\x3d\x22\x22 jsdata\x3d\x22zPXzie;_;v8HjaMKuBpCvptQP9sqQyAs5\x22\x3e\x3cdiv jsname\x3d\x22ibnC6b\x22 class\x3d\x22YpcDnf OSrXXb HG1dvd\x22 role\x3d\x22none\x22\x3e\x3ca href\x3d\x22https://support.google.com/websearch/?p\x3dws_results_help\x26amp;hl\x3den\x26amp;fg\x3d1\x22 role\x3d\x22menuitem\x22\x3eSearch help\x3c/a\x3e\x3c/div\x3e\x3c/g-menu-item\x3e\x3cg-menu-item jsname\x3d\x22NNJLud\x22 jscontroller\x3d\x22CnSW2d\x22 class\x3d\x22EpPYLd Zt0a5e\x22 role\x3d\x22none\x22 data-short-label\x3d\x22\x22 jsdata\x3d\x22zPXzie;_;v8HjaMKuBpCvptQP9sqQyAs5\x22\x3e\x3cdiv jsname\x3d\x22ibnC6b\x22 class\x3d\x22YpcDnf OSrXXb HG1dvd\x22 role\x3d\x22none\x22\x3e\x3ca jscontroller\x3d\x22nRwWne\x22 href\x3d\x22#\x22 data-bucket\x3d\x22typeId:84156\x22 data-psd-ssc\x3d\x220\x22 role\x3d\x22menuitem\x22 target\x3d\x22_blank\x22 id\x3d\x22dk2qOd\x22 jsaction\x3d\x22IevYAd\x22 data-ved\x3d\x220ahUKEwiCitnY1I-QAxWQl4kEHXYlBLkQLggj\x22\x3eFeedback\x3c/a\x3e\x3c/div\x3e\x3c/g-menu-item\x3e\x3c/g-menu\x3e');
            })();
            (function() {
                google.drty && google.drty(undefined, true);
            })();
        });
        if (!google.stvsc) {
            google.drty && google.drty(undefined, true);
        }
    </script>
</body>

</html>

In [ ]:

import subprocess, uuid, time, json, os

log_path = "/var/log/malware_process_log.json"
suspicious_keywords = ['vmtoolsd', 'VBoxService', 'powershell.exe', 'cmd.exe', 'php', 'java', 'curl', 'wget']

def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    os.makedirs(os.path.dirname(log_path), exist_ok=True)
    with open(log_path, "a") as f:
        f.write(json.dumps(entry) + "\n")
    print(f"📜 {event_type}: {details}")

def scan_processes():
    output = subprocess.check_output(['ps', '-eo', 'comm'], text=True)
    for proc in output.splitlines()[1:]:
        proc_name = proc.strip().lower()
        for keyword in suspicious_keywords:
            if keyword in proc_name:
                log_event("SuspiciousProcess", f"Detected: {proc_name}")

scan_processes()

In [ ]:
(import subprocess, uuid, time, json, os

log_path = "/var/log/system_update_log.json"

def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    os.makedirs(os.path.dirname(log_path), exist_ok=True)
    with open(log_path, "a") as f:
        f.write(json.dumps(entry) + "\n")
    print(f"📜 {event_type}: {details}")

def update_system():
    print("🔧 Starting system update...")
    try:
        subprocess.run(["apt", "update", "-y"], check=True)
        log_event("SystemUpdate", "apt update completed.")
        subprocess.run(["apt", "upgrade", "-y"], check=True)
        log_event("SystemUpgrade", "apt upgrade completed.")
    except subprocess.CalledProcessError as e:
        log_event("UpdateError", f"Update failed: {e}")
    except Exception as e:
        log_event("GeneralError", f"Unexpected error: {e}")

if __name__ == "__main__":
    update_system()

In [ ]:

import subprocess, uuid, time, json, os

log_path = "/var/log/program_block_log.json"

# 🔍 คำที่ต้องบล็อกหรือแจ้งเตือน
blocked_keywords = [
    'vmtoolsd', 'VBoxService', 'qemu', 'virt', 'powershell.exe', 'cmd.exe',
    'bash', 'sh', 'zsh', 'ksh', 'dev', 'php', 'java', 'javac',
    'xml', 'json', 'deb', 'apt', 'dpkg'
]

def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    try:
        os.makedirs(os.path.dirname(log_path), exist_ok=True)
        with open(log_path, "a") as f:
            f.write(json.dumps(entry) + "\n")
        print(f"📜 {event_type}: {details}")
    except Exception as e:
        print(f"⚠️ Failed to write log: {e}")

def scan_and_block_programs():
    print("🔍 Scanning for risky programs...")
    try:
        output = subprocess.check_output(['ps', '-eo', 'comm'], text=True)
        for proc in output.splitlines()[1:]:
            proc_name = proc.strip().lower()
            for keyword in blocked_keywords:
                if keyword in proc_name:
                    log_event("BlockedProgramDetected", f"Process matched: {proc_name}")
                    # Optional: kill process (ใช้ด้วยความระมัดระวัง)
                    # subprocess.run(['pkill', '-f', proc_name])
    except Exception as e:
        log_event("ScanError", f"Failed to scan processes: {e}")

def run_program_blocker():
    print("\n🔐 Program Access Blocker System")
    scan_and_block_programs()
    print("✅ Scan complete.\n")

if __name__ == "__main__":
    run_program_blocker()

In [ ]:
import os
import subprocess
import time
import uuid
import json

# 🛡️ ตั้งค่าระบบ
bypass = False  # True = ข้ามการแจ้งเตือนและบล็อก
log_path = "defense_audit_log.json"
threat_processes = ['nmap', 'tcpdump', 'wireshark', 'aircrack-ng', 'metasploit', 'strace', 'netcat', 'ettercap']
danger_ports = [22, 23, 4444, 8080, 31337]

# 📜 บันทึก log พร้อม UUID
def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()),
        "event": event_type,
        "details": details
    }
    with open(log_path, "a") as f:
        f.write(json.dumps(entry) + "\n")
    print(f"📜 {event_type}: {details}")

# 🔍 ตรวจจับโปรเซสต้องสงสัย
def scan_processes():
    print("🔍 Scanning processes...")
    output = subprocess.check_output(['ps', 'aux'], text=True)
    for line in output.splitlines():
        for proc in threat_processes:
            if proc in line:
                log_event("ThreatProcess", f"Detected '{proc}' in: {line.strip()}")
                if not bypass:
                    try:
                        pid = int(line.split()[1])
                        subprocess.run(['kill', '-9', str(pid)])
                        log_event("ProcessTerminated", f"PID {pid} ({proc}) killed.")
                    except Exception as e:
                        log_event("KillFailed", f"Failed to kill {proc}: {e}")

# 🚫 ตรวจจับพอร์ตอันตราย
def scan_ports():
    print("🚫 Scanning open ports...")
    output = subprocess.check_output(['ss', '-tuln'], text=True)
    for line in output.splitlines():
        for port in danger_ports:
            if f":{port} " in line:
                log_event("DangerPort", f"Port {port} open: {line.strip()}")
                if not bypass:
                    log_event("Alert", f"Port {port} flagged for lockdown.")

# 🧠 วิเคราะห์ความเสี่ยงเบื้องต้น
def ai_risk_analysis():
    print("🧠 Running AI risk analysis...")
    risk_score = 0
    with open(log_path, "r") as f:
        logs = f.readlines()
    for entry in logs[-50:]:
        data = json.loads(entry)
        if "Threat" in data["event"]:
            risk_score += 2
        if "Port" in data["event"]:
            risk_score += 1
    level = "LOW"
    if risk_score > 10:
        level = "MEDIUM"
    if risk_score > 20:
        level = "HIGH"
    log_event("RiskAssessment", f"Risk score: {risk_score}, Level: {level}")

# 🔐 ระบบล็อกอัตโนมัติเมื่อความเสี่ยงสูง
def auto_lockdown():
    with open(log_path, "r") as f:
        logs = f.readlines()
    recent = [json.loads(e) for e in logs[-10:]]
    high_threats = [e for e in recent if "Threat" in e["event"]]
    if len(high_threats) >= 5 and not bypass:
        log_event("Lockdown", "System entering lockdown mode due to high threat activity.")
        subprocess.run(["iptables", "-P", "INPUT", "DROP"])
        subprocess.run(["iptables", "-P", "FORWARD", "DROP"])
        subprocess.run(["iptables", "-P", "OUTPUT", "DROP"])

# 🚀 เริ่มระบบป้องกัน
def run_defense_ai():
    print(f"\n🛡️ Defense AI System — Hacker Level 5 Protection")
    print(f"  - Bypass mode: {'ON' if bypass else 'OFF'}")
    scan_processes()
    scan_ports()
    ai_risk_analysis()
    auto_lockdown()
    print("✅ Defense complete.\n")

if __name__ == "__main__":
    run_defense_ai()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
mount -o noexec,nosuid,nodev -t tmpfs tmpfs /sandbox

In [ ]:
!firejail --net=none python3 delete_logs.py

In [ ]:

print("Hello world")

In [ ]:
def test():
   print("Hello")

In [ ]:
value = 5

In [ ]:
for i in range(5):
    print(i)

In [ ]:
print ("Hello")  # Python2, syntax ใน Python3

Chrome_Log_paths

In [ ]:
import os
import shutil

# ตัวอย่าง path ของ Chrome logs
chrome_log_paths = [
    os.path.expanduser("~/.config/google-chrome/Default/chrome_debug.log"),
    os.path.expanduser("~/.cache/google-chrome/"),
    os.path.expanduser("~/.config/google-chrome/Crash Reports/")
]

def secure_delete_logs(paths):
    for path in paths:
        if os.path.isfile(path):
            try:
                os.remove(path)
                print(f"Deleted file: {path}")
            except Exception as e:
                print(f"Error deleting file {path}: {e}")
        elif os.path.isdir(path):
            try:
                shutil.rmtree(path)
                print(f"Deleted directory: {path}")
            except Exception as e:
                print(f"Error deleting directory {path}: {e}")

secure_delete_logs(chrome_log_paths)

 ✅ ตรวจสอบชื่อไฟล์/โฟลเดอร์ทั่วระบบ  เฉพาะพื้นที่ปลอดภัย ไม่แตะต้องระบบหลัก
| 🔐 ใช้หลัก “assess by simple”  ไม่ลบ ไม่เปลี่ยนสิทธิ์ แค่ตรวจสอบและบันทึก
| 📜 บันทึก audit log พร้อม UUID  ลงไฟล์ /var/log/simpleassesslog.json
| 🛡️ ป้องกัน SyntaxError  โค้ดสมบูรณ์ ไม่มี input ที่ไม่ครบ
| 📦 ไม่มีการเชื่อมต่อ cloud  เหมาะสำหรับระบบปิดหรือ audit gateway

In [ ]:
import os
import fnmatch
import uuid
import time
import json

# 🛡️ ตั้งค่าระบบ
search_paths = ['/home', '/tmp', '/var', '/etc']  # จำกัดพื้นที่ปลอดภัย
keywords = ['madia', 'link', 'input', 'output']
audit_log = "/var/log/simple_assess_log.json"

# 📜 ฟังก์ชันบันทึก log
def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    try:
        os.makedirs(os.path.dirname(audit_log), exist_ok=True)
        with open(audit_log, "a") as f:
            f.write(json.dumps(entry) + "\n")
        print(f"📜 {event_type}: {details}")
    except Exception as e:
        print(f"⚠️ Failed to write log: {e}")

# 🔍 ตรวจสอบชื่อไฟล์/โฟลเดอร์ที่เข้าข่าย
def assess_files_simple():
    print("🔍 Starting simple assessment...")
    matches = []
    for path in search_paths:
        for root, dirs, files in os.walk(path):
            for name in files + dirs:
                if any(keyword in name.lower() for keyword in keywords):
                    full_path = os.path.join(root, name)
                    matches.append(full_path)
                    log_event("KeywordMatch", f"Found: {full_path}")
    log_event("AssessmentComplete", f"Total matches: {len(matches)}")
    print("✅ Simple assessment complete.\n")

# 🚀 เริ่มระบบ
if __name__ == "__main__":
    assess_files_simple()

Log_event, Block_keywords

In [ ]:
import os
import fnmatch
import uuid
import time
import json

# 🛡️ ตั้งค่าระบบ
bypass = False  # True = ข้ามการบล็อก, False = บล็อกทันที
search_paths = ['/home', '/tmp', '/var', '/etc']  # ปรับตามระบบของคุณ
block_keywords = ['madia', 'link', 'input', 'output']
audit_log = "/var/log/block_keyword_access.json"

# 📜 ฟังก์ชันบันทึก log
def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    try:
        os.makedirs(os.path.dirname(audit_log), exist_ok=True)
        with open(audit_log, "a") as f:
            f.write(json.dumps(entry) + "\n")
        print(f"📜 {event_type}: {details}")
    except Exception as e:
        print(f"⚠️ Failed to write log: {e}")

# 🔍 ค้นหาไฟล์หรือโฟลเดอร์ที่เข้าข่าย
def find_block_targets():
    targets = []
    for path in search_paths:
        for root, dirs, files in os.walk(path):
            for name in files + dirs:
                if any(keyword in name.lower() for keyword in block_keywords):
                    full_path = os.path.join(root, name)
                    targets.append(full_path)
    return targets

# 🚫 บล็อกการเข้าถึง (เปลี่ยนสิทธิ์หรือแจ้งเตือน)
def block_items(item_list):
    for item in item_list:
        try:
            if not bypass:
                os.chmod(item, 0o000)  # ปิดสิทธิ์ทั้งหมด
                log_event("AccessBlocked", f"Blocked access to: {item}")
            else:
                log_event("AccessDetected", f"Bypass active — found: {item}")
        except Exception as e:
            log_event("BlockFailed", f"Failed to block {item}: {e}")

# 🚀 เริ่มระบบ
def run_keyword_blocker():
    print(f"\n🔐 Keyword-Based Access Blocker")
    print(f"  - Bypass mode: {'ON' if bypass else 'OFF'}")
    items = find_block_targets()
    log_event("ScanComplete", f"Found {len(items)} items with risky keywords.")
    block_items(items)
    print("✅ Blocking complete.\n")

if __name__ == "__main__":
    run_keyword_blocker()

In [ ]:

import subprocess
import shutil
import time
import uuid
import json
import os

log_path = "/var/log/router_access_log.json"
bypass = False

authorized_devices = {
    "00:11:22:33:44:55": "Owner Laptop",
    "AA:BB:CC:DD:EE:FF": "Audit Terminal"
}

def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    try:
        os.makedirs(os.path.dirname(log_path), exist_ok=True)
        with open(log_path, "a") as f:
            f.write(json.dumps(entry) + "\n")
        print(f"📜 {event_type}: {details}")
    except Exception as e:
        print(f"⚠️ Failed to write log: {e}")

# 🔍 ตรวจสอบ MAC/IP ที่เชื่อมต่อ
def scan_connected_devices():
    print("🔍 Scanning connected devices...")
    arp_cmd = shutil.which("ip") or shutil.which("arp")
    if not arp_cmd:
        log_event("ScanError", "No ARP or IP command available.")
        return

    try:
        if "ip" in arp_cmd:
            output = subprocess.check_output(["ip", "neigh"], text=True)
            for line in output.splitlines():
                parts = line.split()
                if len(parts) >= 5:
                    ip = parts[0]
                    mac = parts[4]
                    handle_mac(ip, mac)
        else:
            output = subprocess.check_output(["arp", "-n"], text=True)
            for line in output.splitlines():
                parts = line.split()
                if len(parts) >= 3:
                    ip = parts[0]
                    mac = parts[2]
                    handle_mac(ip, mac)
    except Exception as e:
        log_event("ScanError", f"Failed to scan devices: {e}")

def handle_mac(ip, mac):
    if mac not in authorized_devices:
        log_event("UnauthorizedAccess", f"MAC {mac} IP {ip} not authorized.")
        if not bypass:
            block_mac(mac)
    else:
        log_event("AuthorizedAccess", f"{authorized_devices[mac]} connected with IP {ip}.")

# 🔐 บล็อก MAC ด้วย iptables หรือ fallback
def block_mac(mac):
    if shutil.which("iptables"):
        try:
            subprocess.run(["iptables", "-A", "INPUT", "-m", "mac", "--mac-source", mac, "-j", "DROP"])
            log_event("AccessBlocked", f"Blocked MAC {mac} via iptables.")
        except Exception as e:
            log_event("BlockError", f"Failed to block MAC {mac}: {e}")
    else:
        log_event("BlockSkipped", f"iptables not available — MAC {mac} not blocked.")

# 🔐 ปิดบริการ cloud ด้วย fallback
def disable_third_party_services():
    print("🔐 Disabling third-party services...")
    if shutil.which("systemctl"):
        try:
            subprocess.run(["systemctl", "stop", "systemd-resolved"], stderr=subprocess.DEVNULL)
            subprocess.run(["systemctl", "disable", "systemd-resolved"], stderr=subprocess.DEVNULL)
            log_event("ServiceStopped", "systemd-resolved stopped and disabled.")
        except Exception as e:
            log_event("SystemctlError", f"Failed to run systemctl: {e}")
    else:
        log_event("SystemctlMissing", "systemctl not found — skipping service disable.")

    if shutil.which("iptables"):
        subprocess.run(["iptables", "-A", "OUTPUT", "-d", "cloudflare.com", "-j", "DROP"])
        subprocess.run(["iptables", "-A", "OUTPUT", "-d", "googleapis.com", "-j", "DROP"])
        log_event("CloudBlocked", "Blocked outbound access to cloud services.")
    else:
        log_event("CloudBlockSkipped", "iptables not available — cloud access not blocked.")

# 🧱 ปิด remote provisioning ด้วย fallback
def block_remote_control():
    print("🧱 Blocking remote router control...")
    if shutil.which("iptables"):
        subprocess.run(["iptables", "-A", "INPUT", "-p", "tcp", "--dport", "7547", "-j", "DROP"])
        subprocess.run(["iptables", "-A", "INPUT", "-p", "tcp", "--dport", "443", "-j", "DROP"])
        log_event("RemoteBlocked", "Blocked TR-069 and remote HTTPS access.")
    else:
        log_event("RemoteBlockSkipped", "iptables not available — remote access not blocked.")

# 🚀 เริ่มระบบ
def run_private_router():
    print(f"\n🔐 Private Router Firewall System")
    print(f"  - Bypass mode: {'ON' if bypass else 'OFF'}")
    disable_third_party_services()
    block_remote_control()
    scan_connected_devices()
    print("✅ Router secured as private network.\n")

if __name__ == "__main__":
    run_private_router()

In [ ]:

import os
import fnmatch
import time
import uuid

# 🛡️ ตั้งค่าระบบ
bypass = False  # True = ข้ามการแจ้งเตือน, False = เตือนก่อนลบ
log_patterns = ['*.log', '*.logging', '*.lock']
search_paths = ['/var/log', '/tmp', '/run', '/home', '/etc']  # ปรับตามระบบของคุณ
audit_log = "secure_cleanup_audit.json"

# 📜 บันทึก audit log
def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()),
        "event": event_type,
        "details": details
    }
    with open(audit_log, "a") as f:
        f.write(str(entry) + "\n")
    print(f"📜 {event_type}: {details}")

# 🔍 ค้นหาไฟล์ที่เข้าข่าย
def find_risky_files():
    risky_files = []
    for path in search_paths:
        for root, dirs, files in os.walk(path):
            for pattern in log_patterns:
                for name in fnmatch.filter(files, pattern):
                    full_path = os.path.join(root, name)
                    risky_files.append(full_path)
    return risky_files

# 🚫 ลบไฟล์ที่พบ
def delete_files(file_list):
    for file_path in file_list:
        try:
            if not bypass:
                print(f"⚠️ Preparing to delete: {file_path}")
            os.remove(file_path)
            log_event("FileDeleted", f"Removed: {file_path}")
        except Exception as e:
            log_event("DeleteFailed", f"Failed to delete {file_path}: {e}")

# 🚀 เริ่มระบบ
def run_cleanup():
    print(f"\n🔐 Secure Log/Lock Cleanup System")
    print(f"  - Bypass mode: {'ON' if bypass else 'OFF'}")
    risky_files = find_risky_files()
    log_event("ScanComplete", f"Found {len(risky_files)} risky files.")
    delete_files(risky_files)
    print("✅ Cleanup complete.\n")

if __name__ == "__main__":
    run_cleanup()

🧩 จุดเด่นของเวอร์ชันนี้:

| ฟีเจอร์ | รายละเอียด |
|--------|-------------|
| ✅ รองรับทั้ง ip, arp, iptables, systemctl | ตรวจสอบก่อนใช้ทุกคำสั่ง |
| 🔐 มี fallback หากคำสั่งไม่พร้อมใช้งาน | ไม่ล้ม, ไม่กระทบระบบอื่น |
| 📜 บันทึก log พร้อม UUID และ timestamp | ลงไฟล์ /var/log/routeraccesslog.json |
| 🛡️ ป้องกัน cloud และ remote provisioning | แม้ไม่มี systemd หรือ iptables |
| 📦 เหมาะสำหรับ Termux, Alpine, Docker, VM | ใช้งานได้ในระบบปิดหรือเบา |

In [ ]:
import subprocess
import shutil
import time
import uuid
import json
import os

log_path = "/var/log/router_access_log.json"
bypass = False

authorized_devices = {
    "00:11:22:33:44:55": "Owner Laptop",
    "AA:BB:CC:DD:EE:FF": "Audit Terminal"
}

def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    try:
        os.makedirs(os.path.dirname(log_path), exist_ok=True)
        with open(log_path, "a") as f:
            f.write(json.dumps(entry) + "\n")
        print(f"📜 {event_type}: {details}")
    except Exception as e:
        print(f"⚠️ Failed to write log: {e}")

# 🔍 ตรวจสอบ MAC/IP ที่เชื่อมต่อ
def scan_connected_devices():
    print("🔍 Scanning connected devices...")
    arp_cmd = shutil.which("ip") or shutil.which("arp")
    if not arp_cmd:
        log_event("ScanError", "No ARP or IP command available.")
        return

    try:
        if "ip" in arp_cmd:
            output = subprocess.check_output(["ip", "neigh"], text=True)
            for line in output.splitlines():
                parts = line.split()
                if len(parts) >= 5:
                    ip = parts[0]
                    mac = parts[4]
                    handle_mac(ip, mac)
        else:
            output = subprocess.check_output(["arp", "-n"], text=True)
            for line in output.splitlines():
                parts = line.split()
                if len(parts) >= 3:
                    ip = parts[0]
                    mac = parts[2]
                    handle_mac(ip, mac)
    except Exception as e:
        log_event("ScanError", f"Failed to scan devices: {e}")

def handle_mac(ip, mac):
    if mac not in authorized_devices:
        log_event("UnauthorizedAccess", f"MAC {mac} IP {ip} not authorized.")
        if not bypass:
            block_mac(mac)
    else:
        log_event("AuthorizedAccess", f"{authorized_devices[mac]} connected with IP {ip}.")

# 🔐 บล็อก MAC ด้วย iptables หรือ fallback
def block_mac(mac):
    if shutil.which("iptables"):
        try:
            subprocess.run(["iptables", "-A", "INPUT", "-m", "mac", "--mac-source", mac, "-j", "DROP"])
            log_event("AccessBlocked", f"Blocked MAC {mac} via iptables.")
        except Exception as e:
            log_event("BlockError", f"Failed to block MAC {mac}: {e}")
    else:
        log_event("BlockSkipped", f"iptables not available — MAC {mac} not blocked.")

# 🔐 ปิดบริการ cloud ด้วย fallback
def disable_third_party_services():
    print("🔐 Disabling third-party services...")
    if shutil.which("systemctl"):
        try:
            subprocess.run(["systemctl", "stop", "systemd-resolved"], stderr=subprocess.DEVNULL)
            subprocess.run(["systemctl", "disable", "systemd-resolved"], stderr=subprocess.DEVNULL)
            log_event("ServiceStopped", "systemd-resolved stopped and disabled.")
        except Exception as e:
            log_event("SystemctlError", f"Failed to run systemctl: {e}")
    else:
        log_event("SystemctlMissing", "systemctl not found — skipping service disable.")

    if shutil.which("iptables"):
        subprocess.run(["iptables", "-A", "OUTPUT", "-d", "cloudflare.com", "-j", "DROP"])
        subprocess.run(["iptables", "-A", "OUTPUT", "-d", "googleapis.com", "-j", "DROP"])
        log_event("CloudBlocked", "Blocked outbound access to cloud services.")
    else:
        log_event("CloudBlockSkipped", "iptables not available — cloud access not blocked.")

# 🧱 ปิด remote provisioning ด้วย fallback
def block_remote_control():
    print("🧱 Blocking remote router control...")
    if shutil.which("iptables"):
        subprocess.run(["iptables", "-A", "INPUT", "-p", "tcp", "--dport", "7547", "-j", "DROP"])
        subprocess.run(["iptables", "-A", "INPUT", "-p", "tcp", "--dport", "443", "-j", "DROP"])
        log_event("RemoteBlocked", "Blocked TR-069 and remote HTTPS access.")
    else:
        log_event("RemoteBlockSkipped", "iptables not available — remote access not blocked.")

# 🚀 เริ่มระบบ
def run_private_router():
    print(f"\n🔐 Private Router Firewall System")
    print(f"  - Bypass mode: {'ON' if bypass else 'OFF'}")
    disable_third_party_services()
    block_remote_control()
    scan_connected_devices()
    print("✅ Router secured as private network.\n")

if __name__ == "__main__":
    run_private_router()

🔧 1. รันเป็น Background Service (Linux Systemd)

สร้างไฟล์ service

In [ ]:
!Python3 within /etc/systemd/system/private-router.service

In [ ]:
!python3 with-in /etc/systemd/system/private-router.service

In [ ]:

import smtplib
from email.message import EmailMessage

def send_alert(subject, body):
    try:
        msg = EmailMessage()
        msg.set_content(body)
        msg['Subject'] = subject
        msg['From'] = "router@yourdomain.local"
        msg['To'] = "admin@yourdomain.local"

        with smtplib.SMTP('localhost') as s:
            s.send_message(msg)
        print("📬 Email alert sent.")
    except Exception as e:
        print(f"⚠️ Failed to send email: {e}")

In [ ]:
if event_type in ["UnauthorizedAccess", "AccessBlocked", "RemoteBlocked"]:
    send_alert(f"Router Alert: {event_type}", details)

In [ ]:
import requests

def send_alert(subject, body):
    try:
        payload = {"event": subject, "details": body}
        requests.post("http://localhost:5000/api/router-alert", json=payload)
    except Exception as e:
        print(f"⚠️ Failed to send dashboard alert: {e}")

In [ ]:
def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    try:
        os.makedirs(os.path.dirname(log_path), exist_ok=True)
        with open(log_path, "a") as f:
            f.write(json.dumps(entry) + "\n")
        print(f"📜 {event_type}: {details}")
        # ส่งไปยัง audit trail
        requests.post("http://localhost:5000/api/audit", json=entry)
    except Exception as e:
        print(f"⚠️ Failed to write log or send audit: {e}")

In [ ]:

import subprocess, time, uuid, json, shutil, os, smtplib
from email.message import EmailMessage
import requests

bypass = False
log_path = "/var/log/router_access_log.json"
authorized_devices = {
    "00:11:22:33:44:55": "Owner Laptop",
    "AA:BB:CC:DD:EE:FF": "Audit Terminal"
}

def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    try:
        os.makedirs(os.path.dirname(log_path), exist_ok=True)
        with open(log_path, "a") as f:
            f.write(json.dumps(entry) + "\n")
        print(f"📜 {event_type}: {details}")
        if event_type in ["UnauthorizedAccess", "AccessBlocked", "RemoteBlocked"]:
            send_email_alert(event_type, details)
            send_dashboard_alert(event_type, details)
    except Exception as e:
        print(f"⚠️ Failed to log: {e}")

def send_email_alert(subject, body):
    try:
        msg = EmailMessage()
        msg.set_content(body)
        msg['Subject'] = f"[Router Alert] {subject}"
        msg['From'] = "router@localhost"
        msg['To'] = "admin@localhost"
        with smtplib.SMTP('localhost') as s:
            s.send_message(msg)
        print("📬 Email alert sent.")
    except Exception as e:
        print(f"⚠️ Email failed: {e}")

def send_dashboard_alert(subject, body):
    try:
        payload = {"event": subject, "details": body}
        requests.post("http://localhost:5000/api/router-alert", json=payload)
        print("📊 Dashboard alert sent.")
    except Exception as e:
        print(f"⚠️ Dashboard alert failed: {e}")

def scan_connected_devices():
    print("🔍 Scanning connected devices...")
    if not shutil.which("arp"):
        log_event("ScanError", "ARP command not found.")
        return
    try:
        output = subprocess.check_output(['arp', '-n'], text=True)
        for line in output.splitlines():
            parts = line.split()
            if len(parts) >= 3:
                ip = parts[0]
                mac = parts[2]
                if mac not in authorized_devices:
                    log_event("UnauthorizedAccess", f"MAC {mac} IP {ip} not authorized.")
                    if not bypass and shutil.which("iptables"):
                        subprocess.run(["iptables", "-A", "INPUT", "-m", "mac", "--mac-source", mac, "-j", "DROP"])
                        log_event("AccessBlocked", f"Blocked MAC {mac} from accessing router.")
                else:
                    log_event("AuthorizedAccess", f"{authorized_devices[mac]} connected with IP {ip}.")
    except Exception as e:
        log_event("ScanError", f"ARP scan failed: {e}")

def disable_third_party_services():
    print("🔐 Disabling third-party services...")
    if shutil.which("systemctl"):
        subprocess.run(["systemctl", "stop", "systemd-resolved"], stderr=subprocess.DEVNULL)
        subprocess.run(["systemctl", "disable", "systemd-resolved"], stderr=subprocess.DEVNULL)
    if shutil.which("iptables"):
        subprocess.run(["iptables", "-A", "OUTPUT", "-d", "cloudflare.com", "-j", "DROP"])
        subprocess.run(["iptables", "-A", "OUTPUT", "-d", "googleapis.com", "-j", "DROP"])
    log_event("CloudBlocked", "Blocked outbound access to cloud services.")

def block_remote_control():
    print("🧱 Blocking remote router control...")
    if shutil.which("iptables"):
        subprocess.run(["iptables", "-A", "INPUT", "-p", "tcp", "--dport", "7547", "-j", "DROP"])
        subprocess.run(["iptables", "-A", "INPUT", "-p", "tcp", "--dport", "443", "-j", "DROP"])
        log_event("RemoteBlocked", "Blocked TR-069 and remote HTTPS access.")
    else:
        log_event("FirewallError", "iptables not available.")

def run_private_router():
    print(f"\n🔐 Private Router Firewall System")
    print(f"  - Bypass mode: {'ON' if bypass else 'OFF'}")
    disable_third_party_services()
    block_remote_control()
    scan_connected_devices()
    print("✅ Router secured as private network.\n")

if __name__ == "__main__":
    run_private_router()

- 🔍 ระบบสแกนจาก cloud หรือ remote audit
- 🧠 การประเมินจาก AI หรือ script ที่ไม่ได้รับอนุญาต
- 🌐 การเชื่อมต่อจาก endpoint ที่ไม่อยู่ใน whitelist

ผมขอเสนอระบบ Python 3 ที่บล็อกการประเมินทั้งหมดจากภายนอก โดยใช้หลักการ:

- ✅ ตรวจสอบ process และ network ที่เข้าข่าย
- 🔐 บล็อกการเชื่อมต่อหรือการรัน script ที่ไม่อยู่ในรายการอนุญาต
- 📜 บันทึก audit log ทุกการบล็อก
- 🧱 ปิดพอร์ตหรือบริการที่เปิดให้ assess ภายนอก

In [ ]:
import subprocess
import shutil
import time
import uuid
import json
import os

log_path = "/var/log/assess_block_log.json"
whitelist_process = ['python3', 'sshd', 'bash']
whitelist_ports = [22, 80, 443]  # อนุญาตเฉพาะ SSH และ Web

def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    try:
        os.makedirs(os.path.dirname(log_path), exist_ok=True)
        with open(log_path, "a") as f:
            f.write(json.dumps(entry) + "\n")
        print(f"📜 {event_type}: {details}")
    except Exception as e:
        print(f"⚠️ Failed to write log: {e}")

# 🔍 ตรวจสอบ process ที่ไม่อยู่ใน whitelist
def block_unapproved_processes():
    print("🔍 Checking running processes...")
    try:
        output = subprocess.check_output(['ps', '-eo', 'comm'], text=True)
        for proc in output.splitlines()[1:]:
            proc_name = proc.strip()
            if proc_name not in whitelist_process:
                log_event("ProcessBlocked", f"Blocked or flagged process: {proc_name}")
    except Exception as e:
        log_event("ProcessScanError", f"Failed to scan processes: {e}")

# 🔐 บล็อกพอร์ตที่ไม่อยู่ใน whitelist
def block_unapproved_ports():
    print("🧱 Checking open ports...")
    try:
        output = subprocess.check_output(['ss', '-tuln'], text=True)
        for line in output.splitlines():
            if ':' in line:
                parts = line.split()
                for part in parts:
                    if ':' in part:
                        port_str = part.split(':')[-1]
                        if port_str.isdigit():
                            port = int(port_str)
                            if port not in whitelist_ports:
                                subprocess.run(['iptables', '-A', 'INPUT', '-p', 'tcp', '--dport', str(port), '-j', 'DROP'])
                                log_event("PortBlocked", f"Blocked port: {port}")
    except Exception as e:
        log_event("PortScanError", f"Failed to scan ports: {e}")

# 🚀 เริ่มระบบ
def run_assess_blocker():
    print(f"\n🔐 Assess Blocker System — Prevent External Evaluation")
    block_unapproved_processes()
    block_unapproved_ports()
    print("✅ All unauthorized assess attempts blocked.\n")

if __name__ == "__main__":
    run_assess_blocker()

- ✅ ตรวจสอบแบบง่ายและปลอดภัย
- 🔍 ประเมินชื่อไฟล์และโฟลเดอร์ที่เข้าข่ายคำว่า madia, link, input, output
- 📜 บันทึกผลการตรวจสอบลง log พร้อม UUID
- 🧠 ไม่กระทบระบบหลักหรือไฟล์สำคัญ

🔧 Python 3: ตรวจสอบระบบทั้งหมดแบบ simple assess

In [ ]:

import os
import fnmatch
import uuid
import time
import json

# 🛡️ ตั้งค่าระบบ
search_paths = ['/home', '/tmp', '/var', '/etc']  # จำกัดพื้นที่ปลอดภัย
keywords = ['madia', 'link', 'input', 'output']
audit_log = "/var/log/simple_assess_log.json"

# 📜 ฟังก์ชันบันทึก log
def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    try:
        os.makedirs(os.path.dirname(audit_log), exist_ok=True)
        with open(audit_log, "a") as f:
            f.write(json.dumps(entry) + "\n")
        print(f"📜 {event_type}: {details}")
    except Exception as e:
        print(f"⚠️ Failed to write log: {e}")

# 🔍 ตรวจสอบชื่อไฟล์/โฟลเดอร์ที่เข้าข่าย
def assess_files_simple():
    print("🔍 Starting simple assessment...")
    matches = []
    for path in search_paths:
        for root, dirs, files in os.walk(path):
            for name in files + dirs:
                if any(keyword in name.lower() for keyword in keywords):
                    full_path = os.path.join(root, name)
                    matches.append(full_path)
                    log_event("KeywordMatch", f"Found: {full_path}")
    log_event("AssessmentComplete", f"Total matches: {len(matches)}")
    print("✅ Simple assessment complete.\n")

# 🚀 เริ่มระบบ
if __name__ == "__main__":
    assess_files_simple()

- ✅ ป้องกันการเข้าถึงจากบุคคลที่สาม
- 🔐 บล็อก cloud และ remote provisioning
- 📜 บันทึก audit log พร้อม UUID
- 🧠 ใช้งานได้ในหลายสภาพแวดล้อมโดยไม่ล้ม


🔧 Python 3: ปรับปรุงระบบด้วย fallback สำหรับ environment ที่ไม่มี systemctl หรือ iptables

In [ ]:
import subprocess
import shutil
import time
import uuid
import json
import os

log_path = "/var/log/router_access_log.json"
bypass = False

authorized_devices = {
    "00:11:22:33:44:55": "Owner Laptop",
    "AA:BB:CC:DD:EE:FF": "Audit Terminal"
}

def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    try:
        os.makedirs(os.path.dirname(log_path), exist_ok=True)
        with open(log_path, "a") as f:
            f.write(json.dumps(entry) + "\n")
        print(f"📜 {event_type}: {details}")
    except Exception as e:
        print(f"⚠️ Failed to write log: {e}")

# 🔍 ตรวจสอบ MAC/IP ที่เชื่อมต่อ
def scan_connected_devices():
    print("🔍 Scanning connected devices...")
    arp_cmd = shutil.which("ip") or shutil.which("arp")
    if not arp_cmd:
        log_event("ScanError", "No ARP or IP command available.")
        return

    try:
        if "ip" in arp_cmd:
            output = subprocess.check_output(["ip", "neigh"], text=True)
            for line in output.splitlines():
                parts = line.split()
                if len(parts) >= 5:
                    ip = parts[0]
                    mac = parts[4]
                    handle_mac(ip, mac)
        else:
            output = subprocess.check_output(["arp", "-n"], text=True)
            for line in output.splitlines():
                parts = line.split()
                if len(parts) >= 3:
                    ip = parts[0]
                    mac = parts[2]
                    handle_mac(ip, mac)
    except Exception as e:
        log_event("ScanError", f"Failed to scan devices: {e}")

def handle_mac(ip, mac):
    if mac not in authorized_devices:
        log_event("UnauthorizedAccess", f"MAC {mac} IP {ip} not authorized.")
        if not bypass:
            block_mac(mac)
    else:
        log_event("AuthorizedAccess", f"{authorized_devices[mac]} connected with IP {ip}.")

# 🔐 บล็อก MAC ด้วย iptables หรือ fallback
def block_mac(mac):
    if shutil.which("iptables"):
        try:
            subprocess.run(["iptables", "-A", "INPUT", "-m", "mac", "--mac-source", mac, "-j", "DROP"])
            log_event("AccessBlocked", f"Blocked MAC {mac} via iptables.")
        except Exception as e:
            log_event("BlockError", f"Failed to block MAC {mac}: {e}")
    else:
        log_event("BlockSkipped", f"iptables not available — MAC {mac} not blocked.")

# 🔐 ปิดบริการ cloud ด้วย fallback
def disable_third_party_services():
    print("🔐 Disabling third-party services...")
    if shutil.which("systemctl"):
        try:
            subprocess.run(["systemctl", "stop", "systemd-resolved"], stderr=subprocess.DEVNULL)
            subprocess.run(["systemctl", "disable", "systemd-resolved"], stderr=subprocess.DEVNULL)
            log_event("ServiceStopped", "systemd-resolved stopped and disabled.")
        except Exception as e:
            log_event("SystemctlError", f"Failed to run systemctl: {e}")
    else:
        log_event("SystemctlMissing", "systemctl not found — skipping service disable.")

    if shutil.which("iptables"):
        subprocess.run(["iptables", "-A", "OUTPUT", "-d", "cloudflare.com", "-j", "DROP"])
        subprocess.run(["iptables", "-A", "OUTPUT", "-d", "googleapis.com", "-j", "DROP"])
        log_event("CloudBlocked", "Blocked outbound access to cloud services.")
    else:
        log_event("CloudBlockSkipped", "iptables not available — cloud access not blocked.")

# 🧱 ปิด remote provisioning ด้วย fallback
def block_remote_control():
    print("🧱 Blocking remote router control...")
    if shutil.which("iptables"):
        subprocess.run(["iptables", "-A", "INPUT", "-p", "tcp", "--dport", "7547", "-j", "DROP"])
        subprocess.run(["iptables", "-A", "INPUT", "-p", "tcp", "--dport", "443", "-j", "DROP"])
        log_event("RemoteBlocked", "Blocked TR-069 and remote HTTPS access.")
    else:
        log_event("RemoteBlockSkipped", "iptables not available — remote access not blocked.")

# 🚀 เริ่มระบบ
def run_private_router():
    print(f"\n🔐 Private Router Firewall System")
    print(f"  - Bypass mode: {'ON' if bypass else 'OFF'}")
    disable_third_party_services()
    block_remote_control()
    scan_connected_devices()
    print("✅ Router secured as private network.\n")

if __name__ == "__main__":
    run_private_router()

- ✅ ความเข้ากันได้กับระบบ Linux สมัยใหม่ (Debian, Ubuntu, Kali)
- 🔐 การใช้งานจริงอย่างมีประสิทธิภาพ
- 📜 การจัดการข้อผิดพลาดอย่างปลอดภัย
- 🧠 การตรวจสอบ MAC/IP แบบยืดหยุ่น
- 🛡️ ไม่ส่งผลกระทบต่อระบบอื่นโดยไม่จำเป็น

In [ ]:
import subprocess
import time
import uuid
import json
import shutil

# 🛡️ ตั้งค่าระบบ
bypass = False
log_path = "/var/log/router_access_log.json"  # ใช้ path มาตรฐาน
authorized_devices = {
    "00:11:22:33:44:55": "Owner Laptop",
    "AA:BB:CC:DD:EE:FF": "Audit Terminal"
}

# 📜 บันทึก log พร้อม UUID
def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    try:
        with open(log_path, "a") as f:
            f.write(json.dumps(entry) + "\n")
        print(f"📜 {event_type}: {details}")
    except Exception as e:
        print(f"⚠️ Failed to write log: {e}")

# 🔍 ตรวจสอบ MAC/IP ที่เชื่อมต่อ
def scan_connected_devices():
    print("🔍 Scanning connected devices...")
    if not shutil.which("arp"):
        log_event("ScanError", "ARP command not found.")
        return
    try:
        output = subprocess.check_output(['arp', '-n'], text=True)
        for line in output.splitlines():
            parts = line.split()
            if len(parts) >= 3:
                ip = parts[0]
                mac = parts[2]
                if mac not in authorized_devices:
                    log_event("UnauthorizedAccess", f"MAC {mac} IP {ip} not authorized.")
                    if not bypass and shutil.which("iptables"):
                        subprocess.run(["iptables", "-A", "INPUT", "-m", "mac", "--mac-source", mac, "-j", "DROP"])
                        log_event("AccessBlocked", f"Blocked MAC {mac} from accessing router.")
                else:
                    log_event("AuthorizedAccess", f"{authorized_devices[mac]} connected with IP {ip}.")
    except Exception as e:
        log_event("ScanError", f"Failed to scan ARP table: {e}")

# 🔐 ปิดบริการที่เชื่อมกับบุคคลที่สาม
def disable_third_party_services():
    print("🔐 Disabling third-party services...")
    if shutil.which("systemctl"):
        subprocess.run(["systemctl", "stop", "systemd-resolved"], stderr=subprocess.DEVNULL)
        subprocess.run(["systemctl", "disable", "systemd-resolved"], stderr=subprocess.DEVNULL)
    if shutil.which("iptables"):
        subprocess.run(["iptables", "-A", "OUTPUT", "-d", "cloudflare.com", "-j", "DROP"])
        subprocess.run(["iptables", "-A", "OUTPUT", "-d", "googleapis.com", "-j", "DROP"])
    log_event("CloudBlocked", "Blocked outbound access to cloud services.")

# 🧱 ปิด remote management และ auto provisioning
def block_remote_control():
    print("🧱 Blocking remote router control...")
    if shutil.which("iptables"):
        subprocess.run(["iptables", "-A", "INPUT", "-p", "tcp", "--dport", "7547", "-j", "DROP"])
        subprocess.run(["iptables", "-A", "INPUT", "-p", "tcp", "--dport", "443", "-j", "DROP"])
        log_event("RemoteBlocked", "Blocked TR-069 and remote HTTPS access.")
    else:
        log_event("FirewallError", "iptables not available.")

# 🚀 เริ่มระบบ
def run_private_router():
    print(f"\n🔐 Private Router Firewall System")
    print(f"  - Bypass mode: {'ON' if bypass else 'OFF'}")
    disable_third_party_services()
    block_remote_control()
    scan_connected_devices()
    print("✅ Router secured as private network.\n")

if __name__ == "__main__":
    run_private_router()

In [ ]:
import datetime
import json
import os
import paramiko  # ใช้เชื่อมต่อ SSH

# 🔐 กำหนดข้อมูล router สำหรับเชื่อมต่อ SSH
ROUTER_CONFIG = {
    "hostname": "192.168.1.1",
    "port": 22,
    "username": "admin",
    "password": "your_password_here",
    "service_name": "router.service"
}

# 📁 กำหนด path สำหรับ log และ audit
LOG_FILE = "router_status.log"
AUDIT_FILE = "router_audit.json"

# 📡 เชื่อมต่อ router ผ่าน SSH และตรวจสอบสถานะ systemd
def check_router_status_via_ssh(config):
    try:
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(config["hostname"], port=config["port"],
                    username=config["username"], password=config["password"])
        command = f"systemctl is-active {config['service_name']}"
        stdin, stdout, stderr = ssh.exec_command(command)
        status = stdout.read().decode().strip()
        ssh.close()
    except Exception as e:
        status = f"error: {str(e)}"
    return status

# 🧾 สร้าง metadata และ audit log
def generate_audit_entry(status, config):
    timestamp = datetime.datetime.now().isoformat()
    return {
        "timestamp": timestamp,
        "service": config["service_name"],
        "status": status,
        "source": config["hostname"],
        "method": "SSH",
        "uuid": os.urandom(8).hex()
    }

# 📝 บันทึก log แบบข้อความ
def log_to_file(entry):
    with open(LOG_FILE, "a") as f:
        f.write(f"[{entry['timestamp']}] {entry['service']} on {entry['source']} → {entry['status']}\n")

# 📦 บันทึก audit log แบบ JSON
def save_audit(entry):
    if not os.path.exists(AUDIT_FILE):
        with open(AUDIT_FILE, "w") as f:
            json.dump([], f)
    with open(AUDIT_FILE, "r+") as f:
        data = json.load(f)
        data.append(entry)
        f.seek(0)
        json.dump(data, f, indent=2)

# 🚀 เรียกใช้งานทั้งหมด
if __name__ == "__main__":
    status = check_router_status_via_ssh(ROUTER_CONFIG)
    audit_entry = generate_audit_entry(status, ROUTER_CONFIG)
    log_to_file(audit_entry)
    save_audit(audit_entry)
    print(json.dumps(audit_entry, indent=2, ensure_ascii=False))

In [ ]:

# ติดตั้ง paramiko สำหรับเชื่อมต่อ SSH
!pip install paramiko

# ติดตั้ง requests สำหรับเรียก API
!pip install requests

In [ ]:
# ติดตั้ง tcpdump หรือเครื่องมือ network บนระบบ Linux
!apt update
!apt install tcpdump

In [ ]:
import datetime
import json
import os
import paramiko  # ใช้สำหรับเชื่อมต่อ SSH

# 🔐 กำหนดข้อมูล router สำหรับเชื่อมต่อ SSH
ROUTER_CONFIG = {
    "hostname": "192.168.1.1",        # IP ของ router
    "port": 22,                       # พอร์ต SSH
    "username": "admin",             # ชื่อผู้ใช้
    "password": "your_password_here",# รหัสผ่าน
    "service_name": "router.service" # ชื่อ systemd service ที่ต้องตรวจสอบ
}

# 📁 กำหนด path สำหรับ log และ audit
LOG_FILE = "router_status.log"
AUDIT_FILE = "router_audit.json"

# 📡 เชื่อมต่อ router ผ่าน SSH และตรวจสอบสถานะ systemd
def check_router_status_via_ssh(config):
    try:
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(config["hostname"], port=config["port"],
                    username=config["username"], password=config["password"])
        command = f"systemctl is-active {config['service_name']}"
        stdin, stdout, stderr = ssh.exec_command(command)
        status = stdout.read().decode().strip()
        ssh.close()
    except Exception as e:
        status = f"error: {str(e)}"
    return status

# 🧾 สร้าง metadata และ audit log
def generate_audit_entry(status, config):
    timestamp = datetime.datetime.now().isoformat()
    return {
        "timestamp": timestamp,
        "service": config["service_name"],
        "status": status,
        "source": config["hostname"],
        "method": "SSH",
        "uuid": os.urandom(8).hex()
    }

# 📝 บันทึก log แบบข้อความ
def log_to_file(entry):
    with open(LOG_FILE, "a") as f:
        f.write(f"[{entry['timestamp']}] {entry['service']} on {entry['source']} → {entry['status']}\n")

# 📦 บันทึก audit log แบบ JSON
def save_audit(entry):
    if not os.path.exists(AUDIT_FILE):
        with open(AUDIT_FILE, "w") as f:
            json.dump([], f)
    with open(AUDIT_FILE, "r+") as f:
        data = json.load(f)
        data.append(entry)
        f.seek(0)
        json.dump(data, f, indent=2)

# 🚀 เรียกใช้งานทั้งหมด
if __name__ == "__main__":
    status = check_router_status_via_ssh(ROUTER_CONFIG)
    audit_entry = generate_audit_entry(status, ROUTER_CONFIG)
    log_to_file(audit_entry)
    save_audit(audit_entry)
    print(json.dumps(audit_entry, indent=2, ensure_ascii=False))

In [ ]:

if status == "inactive":
    log_to_file({"note": "Bypassed inactive check for testing"})

In [ ]:
try:
    status = check_router_status_via_ssh(config)
except Exception:
    status = "fallback: using cached status"

แบล็คเอน

In [ ]:

import subprocess
import time
import uuid
import json
import shutil

# 🛡️ ตั้งค่าระบบ
bypass = False
log_path = "/var/log/router_access_log.json"  # ใช้ path มาตรฐาน
authorized_devices = {
    "00:11:22:33:44:55": "Owner Laptop",
    "AA:BB:CC:DD:EE:FF": "Audit Terminal"
}

# 📜 บันทึก log พร้อม UUID
def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
        "event": event_type,
        "details": details
    }
    try:
        with open(log_path, "a") as f:
            f.write(json.dumps(entry) + "\n")
        print(f"📜 {event_type}: {details}")
    except Exception as e:
        print(f"⚠️ Failed to write log: {e}")

# 🔍 ตรวจสอบ MAC/IP ที่เชื่อมต่อ
def scan_connected_devices():
    print("🔍 Scanning connected devices...")
    if not shutil.which("arp"):
        log_event("ScanError", "ARP command not found.")
        return
    try:
        output = subprocess.check_output(['arp', '-n'], text=True)
        for line in output.splitlines():
            parts = line.split()
            if len(parts) >= 3:
                ip = parts[0]
                mac = parts[2]
                if mac not in authorized_devices:
                    log_event("UnauthorizedAccess", f"MAC {mac} IP {ip} not authorized.")
                    if not bypass and shutil.which("iptables"):
                        subprocess.run(["iptables", "-A", "INPUT", "-m", "mac", "--mac-source", mac, "-j", "DROP"])
                        log_event("AccessBlocked", f"Blocked MAC {mac} from accessing router.")
                else:
                    log_event("AuthorizedAccess", f"{authorized_devices[mac]} connected with IP {ip}.")
    except Exception as e:
        log_event("ScanError", f"Failed to scan ARP table: {e}")

# 🔐 ปิดบริการที่เชื่อมกับบุคคลที่สาม
def disable_third_party_services():
    print("🔐 Disabling third-party services...")
    if shutil.which("systemctl"):
        subprocess.run(["systemctl", "stop", "systemd-resolved"], stderr=subprocess.DEVNULL)
        subprocess.run(["systemctl", "disable", "systemd-resolved"], stderr=subprocess.DEVNULL)
    if shutil.which("iptables"):
        subprocess.run(["iptables", "-A", "OUTPUT", "-d", "cloudflare.com", "-j", "DROP"])
        subprocess.run(["iptables", "-A", "OUTPUT", "-d", "googleapis.com", "-j", "DROP"])
    log_event("CloudBlocked", "Blocked outbound access to cloud services.")

# 🧱 ปิด remote management และ auto provisioning
def block_remote_control():
    print("🧱 Blocking remote router control...")
    if shutil.which("iptables"):
        subprocess.run(["iptables", "-A", "INPUT", "-p", "tcp", "--dport", "7547", "-j", "DROP"])
        subprocess.run(["iptables", "-A", "INPUT", "-p", "tcp", "--dport", "443", "-j", "DROP"])
        log_event("RemoteBlocked", "Blocked TR-069 and remote HTTPS access.")
    else:
        log_event("FirewallError", "iptables not available.")

# 🚀 เริ่มระบบ
def run_private_router():
    print(f"\n🔐 Private Router Firewall System")
    print(f"  - Bypass mode: {'ON' if bypass else 'OFF'}")
    disable_third_party_services()
    block_remote_control()
    scan_connected_devices()
    print("✅ Router secured as private network.\n")

if __name__ == "__main__":
    run_private_router()

In [ ]:
import os
import fnmatch
import time
import uuid
import json

# 🛡️ ตั้งค่าระบบ
bypass = False  # True = ข้ามการแจ้งเตือน, False = เตือนก่อนลบ
search_paths = ['/home', '/var', '/usr', '/etc', '/tmp']  # ปรับตามระบบของคุณ
danger_patterns = ['*node*', '*note*', '*lib*', '*global*', '*share*', '*doc*', '*cloud*']
audit_log = "dangerous_file_cleanup_log.json"

# 📜 บันทึก audit log
def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()),
        "event": event_type,
        "details": details
    }
    with open(audit_log, "a") as f:
        f.write(json.dumps(entry) + "\n")
    print(f"📜 {event_type}: {details}")

# 🔍 ค้นหาไฟล์/โฟลเดอร์ที่เข้าข่าย
def find_dangerous_items():
    targets = []
    for path in search_paths:
        for root, dirs, files in os.walk(path):
            for pattern in danger_patterns:
                for name in fnmatch.filter(files + dirs, pattern):
                    full_path = os.path.join(root, name)
                    targets.append(full_path)
    return targets

# 🚫 ลบไฟล์/โฟลเดอร์ที่พบ
def delete_items(item_list):
    for item in item_list:
        try:
            if os.path.isfile(item):
                os.remove(item)
                log_event("FileDeleted", f"Removed file: {item}")
            elif os.path.isdir(item):
                # ลบโฟลเดอร์แบบ recursive
                for root, dirs, files in os.walk(item, topdown=False):
                    for name in files:
                        try:
                            os.remove(os.path.join(root, name))
                        except Exception as e:
                            log_event("DeleteFailed", f"Failed to delete file in {root}: {e}")
                    for name in dirs:
                        try:
                            os.rmdir(os.path.join(root, name))
                        except Exception as e:
                            log_event("DeleteFailed", f"Failed to delete folder in {root}: {e}")
                os.rmdir(item)
                log_event("FolderDeleted", f"Removed folder: {item}")
        except Exception as e:
            log_event("DeleteFailed", f"Failed to delete {item}: {e}")

# 🚀 เริ่มระบบ
def run_cleanup():
    print(f"\n🔐 Dangerous File/Folder Cleanup System")
    print(f"  - Bypass mode: {'ON' if bypass else 'OFF'}")
    items = find_dangerous_items()
    log_event("ScanComplete", f"Found {len(items)} risky items.")
    delete_items(items)
    print("✅ Cleanup complete.\n")

if __name__ == "__main__":
    run_cleanup()

In [ ]:
import os
import subprocess
import time
import uuid
import json

# 🛡️ ตั้งค่าระบบ
bypass = False  # True = ข้ามการแจ้งเตือนและบล็อก
log_path = "ai_recall_defense_log.json"
keywords = ['backpack', 'backup', 'background', 'restore', 'mount', 'sync', 'remote']
restricted_paths = ['/mnt/', '/media/', '/run/user/', '/home/*/.cache/', '/tmp/']

# 📜 ฟังก์ชันบันทึก log
def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()),
        "event": event_type,
        "details": details
    }
    with open(log_path, "a") as f:
        f.write(json.dumps(entry) + "\n")
    print(f"📜 {event_type}: {details}")

# 🔍 ตรวจจับโปรเซสต้องสงสัย
def scan_processes():
    print("🔍 Scanning processes for recall activity...")
    output = subprocess.check_output(['ps', 'aux'], text=True)
    for line in output.splitlines():
        for keyword in keywords:
            if keyword in line.lower():
                log_event("RecallProcessDetected", f"Keyword '{keyword}' found in: {line.strip()}")
                if not bypass:
                    try:
                        pid = int(line.split()[1])
                        subprocess.run(['kill', '-9', str(pid)])
                        log_event("ProcessTerminated", f"PID {pid} ({keyword}) killed.")
                    except Exception as e:
                        log_event("KillFailed", f"Failed to kill {keyword}: {e}")

# 🚫 ตรวจสอบการ mount หรือ sync จากเครื่องอื่น
def scan_mounts():
    print("🚫 Checking for external mounts or sync...")
    output = subprocess.check_output(['mount'], text=True)
    for line in output.splitlines():
        for path in restricted_paths:
            if path in line:
                log_event("ExternalMountDetected", f"Mount path '{path}' found: {line.strip()}")
                if not bypass:
                    subprocess.run(['umount', path], stderr=subprocess.DEVNULL)
                    log_event("MountBlocked", f"Unmounted: {path}")

# 🔐 ล็อกระบบเมื่อพบการละเมิด
def auto_lockdown():
    with open(log_path, "r") as f:
        logs = f.readlines()
    recent = [json.loads(e) for e in logs[-10:]]
    threats = [e for e in recent if "Detected" in e["event"]]
    if len(threats) >= 3 and not bypass:
        log_event("Lockdown", "System entering lockdown mode due to recall threats.")
        subprocess.run(["pkill", "-u", os.getlogin()])
        subprocess.run(["iptables", "-P", "INPUT", "DROP"])
        subprocess.run(["iptables", "-P", "OUTPUT", "DROP"])

# 🚀 เริ่มระบบ
def run_recall_defense_ai():
    print(f"\n🔐 AI Recall Defense System")
    print(f"  - Bypass mode: {'ON' if bypass else 'OFF'}")
    scan_processes()
    scan_mounts()
    auto_lockdown()
    print("✅ Defense complete.\n")

if __name__ == "__main__":
    run_recall_defense_ai

In [ ]:
import os
import time
import uuid
import json
import fnmatch

# 🛡️ ตั้งค่าระบบ
bypass = False  # True = ข้ามการแจ้งเตือนและการลบ
log_path = "fustrack_audit_log.json"
search_root = "/"  # หรือกำหนดเฉพาะโฟลเดอร์ เช่น "/home/user/"
fustrack_patterns = ['*fustrack*', '*.fustrack.conf', '*.fustrack.cache', '*.fustrack.tmp']
essential_files = ['fustrack_main.py', 'fustrack_core.bin']  # ไฟล์หลักที่ไม่ควรถูกลบ

# 📜 บันทึก log พร้อม UUID
def log_event(event_type, details):
    entry = {
        "uuid": str(uuid.uuid4()),
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()),
        "event": event_type,
        "details": details
    }
    with open(log_path, "a") as f:
        f.write(json.dumps(entry) + "\n")
    print(f"📜 {event_type}: {details}")

# 🔍 ตรวจจับไฟล์ที่เกี่ยวข้องกับ fustrack
def scan_fustrack_files():
    print("🔍 Scanning for fustrack-related files...")
    suspicious_files = []
    for root, dirs, files in os.walk(search_root):
        for name in files:
            for pattern in fustrack_patterns:
                if fnmatch.fnmatch(name, pattern) and name not in essential_files:
                    full_path = os.path.join(root, name)
                    suspicious_files.append(full_path)
    return suspicious_files

# 🚫 ลบไฟล์ต้องสงสัย
def delete_files(file_list):
    for path in file_list:
        try:
            if not bypass:
                os.remove(path)
                log_event("FileDeleted", f"Removed: {path}")
            else:
                log_event("FileDetected", f"Bypass active — skipped deletion: {path}")
        except Exception as e:
            log_event("DeleteFailed", f"Failed to delete {path}: {e}")

# 🧠 วิเคราะห์ความเสี่ยงเบื้องต้น
def ai_risk_analysis(file_list):
    print("🧠 Running AI risk analysis...")
    risk_score = len(file_list)
    level = "LOW"
    if risk_score > 10:
        level = "MEDIUM"
    if risk_score > 25:
        level = "HIGH"
    log_event("RiskAssessment", f"Risk score: {risk_score}, Level: {level}")

# 🚀 เริ่มระบบ
def run_fustrack_defense_ai():
    print(f"\n🔐 Fustrack Defense AI System")
    print(f"  - Bypass mode: {'ON' if bypass else 'OFF'}")
    files = scan_fustrack_files()
    ai_risk_analysis(files)
    delete_files(files)
    print("✅ Scan and cleanup complete.\n")

if __name__ == "__main__":
    run_fustrack_defense_ai()

This code transforms or splits the dictionary column into many columns.

E.g. The output DataFrame of this cell will have columns named [`date, letter, fruit, weather`].

In [ ]:
import pandas as pd
df = pd.DataFrame({
  'date': ['2022-09-14', '2022-09-15', '2022-09-16'],
  'letter': ['A', 'B', 'C'],
  'dict' : [{ 'fruit': 'apple', 'weather': 'aces'},
            { 'fruit': 'banana', 'weather': 'bad'},
            { 'fruit': 'cantaloupe', 'weather': 'cloudy'}],
})

pd.concat([df.drop(['dict'], axis=1), df['dict'].apply(pd.Series)], axis=1)

In [ ]:
# ตัวอย่างการอ่านไฟล์อย่างปลอดภัย
from pathlib import Path

def read_file_securely(filepath: str) -> str:
    path = Path(filepath)
    if not path.is_file():
        raise FileNotFoundError(f"File not found: {filepath}")
    return path.read_text(encoding='utf-8')

In [ ]:
def calculate_discount(price: float, rate: float) -> float:
    """คำนวณส่วนลดจากราคาและอัตราส่วนลด"""
    return price * (1 - rate)

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def process_data(data: list[int]) -> int:
    logger.info("Processing data...")
    return sum(data)

In [ ]:
import os
import hashlib
import logging
import sqlite3
import mimetypes
from datetime import datetime
from pathlib import Path
from typing import Dict
from werkzeug.utils import secure_filename
from flask import Flask, request, jsonify

app = Flask(__name__)
UPLOAD_DIR = Path("uploads")
UPLOAD_DIR.mkdir(exist_ok=True)

# ✅ ตั้งค่า Logging
logging.basicConfig(
    filename='audit.log',
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s'
)

# ✅ สร้างฐานข้อมูล SQLite สำหรับ metadata
DB_PATH = "metadata.db"
def init_db():
    with sqlite3.connect(DB_PATH) as conn:
        conn.execute("""
        CREATE TABLE IF NOT EXISTS uploads (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user TEXT,
            filename TEXT,
            mime TEXT,
            size_bytes INTEGER,
            sha256 TEXT,
            created TEXT,
            disclaimer TEXT
        )
        """)
init_db()

# ✅ ตรวจสอบ MIME type
def validate_mime(file_path: Path) -> str:
    mime_type, _ = mimetypes.guess_type(file_path)
    return mime_type or "application/octet-stream"

# ✅ ตรวจสอบ metadata
def extract_metadata(file_path: Path) -> Dict[str, str]:
    stat = file_path.stat()
    return {
        "filename": file_path.name,
        "size_bytes": stat.st_size,
        "created": datetime.fromtimestamp(stat.st_ctime).isoformat(),
        "modified": datetime.fromtimestamp(stat.st_mtime).isoformat(),
        "sha256": hashlib.sha256(file_path.read_bytes()).hexdigest(),
        "mime": validate_mime(file_path)
    }

# ✅ บันทึกลงฐานข้อมูล
def save_to_db(user: str, metadata: Dict[str, str], disclaimer: str):
    with sqlite3.connect(DB_PATH) as conn:
        conn.execute("""
        INSERT INTO uploads (user, filename, mime, size_bytes, sha256, created, disclaimer)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (
            user,
            metadata["filename"],
            metadata["mime"],
            metadata["size_bytes"],
            metadata["sha256"],
            metadata["created"],
            disclaimer
        ))

# ✅ ตรวจสอบ sandbox (จำลอง)
def sandbox_check(file_path: Path) -> bool:
    # ตรวจสอบขนาดไฟล์และ MIME type เบื้องต้น
    if file_path.stat().st_size > 10_000_000:
        return False
    if validate_mime(file_path).startswith("application/x-msdownload"):
        return False
    return True

# ✅ บันทึก audit log
def log_upload_event(user: str, metadata: Dict[str, str]):
    logging.info(f"UPLOAD by {user} | {metadata}")

# ✅ API อัปโหลดไฟล์
@app.route("/upload", methods=["POST"])
def upload_file():
    user = request.form.get("user", "anonymous")
    file = request.files.get("file")
    if not file:
        return jsonify({"error": "No file provided"}), 400

    filename = secure_filename(file.filename)
    save_path = UPLOAD_DIR / filename
    file.save(save_path)

    metadata = extract_metadata(save_path)

    # ✅ ตรวจสอบ sandbox
    if not sandbox_check(save_path):
        return jsonify({"error": "File rejected by sandbox"}), 403

    # ✅ เพิ่ม disclaimer
    disclaimer = "This file was uploaded by user and may contain sensitive or regulated content. Use with caution."

    # ✅ บันทึก log และฐานข้อมูล
    log_upload_event(user, metadata)
    save_to_db(user, metadata, disclaimer)

    return jsonify({
        "message": "File uploaded successfully",
        "metadata": metadata,
        "disclaimer": disclaimer
    })

if __name__ == "__main__":
    app.run(debug=False, port=8080)

Project1

In [ ]:
def generate_compliance_disclaimer(metadata: Dict[str, str]) -> str:
    if metadata["size_bytes"] > 5_000_000:
        return "ไฟล์นี้มีขนาดใหญ่และอาจมีเนื้อหาที่ต้องตรวจสอบก่อนใช้งาน"
    if metadata["mime"].startswith("application/"):
        return "ไฟล์นี้เป็นประเภท application ซึ่งอาจมีสคริปต์หรือโค้ดฝังอยู่"
    return "ไฟล์นี้ผ่านการตรวจสอบเบื้องต้นแล้ว แต่ควรใช้งานด้วยความระมัดระวัง"

def deep_inspection(file_path: Path) -> Dict[str, str]:
    content = file_path.read_bytes()
    suspicious = b"<script>" in content or b"eval(" in content
    return {
        "contains_script": suspicious,
        "binary_signature": hashlib.md5(content[:64]).hexdigest()
    }

In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import hashlib, os
from stegano import lsb
from oletools.olevba import VBA_Parser
import magic
from datetime import datetime
import pygame

pygame.mixer.init()
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

def compute_file_hash(file_path):
    sha256 = hashlib.sha256()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            sha256.update(chunk)
    return sha256.hexdigest()

def detect_mime_type(file_path):
    return magic.Magic(mime=True).from_file(file_path)

def detect_steganography(file_path):
    try:
        if not file_path.lower().endswith((".png", ".bmp")):
            return False, "Unsupported image format for stego analysis (use PNG or BMP)."

        img = Image.open(file_path)
        rgb_path = file_path + "_temp_rgb.png"
        img.convert("RGB").save(rgb_path)

        secret = lsb.reveal(rgb_path)
        os.remove(rgb_path)

        return (True, secret) if secret else (False, "No hidden data found.")
    except Exception as e:
        return False, f"Stego error: {e}"

def ransomware_trace(secret):
    keywords = ["ransom", "bitcoin", "wallet", "key=", "decrypt", "aes", ".onion"]
    for word in keywords:
        if word in secret.lower():
            return True, f"⚠️ Ransomware indicator: {word}"
    return False, "✅ No ransomware indicators."

def scan_macros(file_path):
    logs = []
    try:
        vba = VBA_Parser(file_path)
        if vba.detect_vba_macros():
            logs.append("✅ Macros found. Extracting code...")
            for (_, stream_path, filename, code) in vba.extract_macros():
                logs.append(f"\n--- Macro: {filename or 'Unnamed'} ---\n{code[:1000]}")
                if any(k in code.lower() for k in ["shell", "powershell", "autoopen", "createobject"]):
                    logs.append("⚠️ Suspicious macro behavior detected.")
            return logs, "Threat Detected"
        else:
            logs.append("✅ No macros found.")
            return logs, "Clean"
    except Exception as e:
        return [f"Macro scan failed: {e}"], "Scan Error"

def scan_log_file(path):
    indicators = ["bitcoin", "wallet", "decrypt", ".onion", "tor", "aes", "key="]
    lolbins = ["powershell.exe", "certutil.exe", "regsvr32.exe", "mshta.exe"]
    log = []
    try:
        with open(path, "r", errors="ignore") as f:
            content = f.read().lower()
            found = False
            for ioc in indicators + lolbins:
                if ioc in content:
                    found = True
                    log.append(f"⚠️ IOC: {ioc}")
            if not found:
                log.append("✅ No suspicious content.")
            return log, "Threat Detected" if found else "Clean"
    except Exception as e:
        return [f"Log read failed: {e}"], "Scan Error"

def full_scan_pipeline(path):
    logs = [f"Scanning: {os.path.basename(path)}", f"SHA256: {compute_file_hash(path)}"]
    mime = detect_mime_type(path)
    logs.append(f"MIME: {mime}")
    ext = os.path.splitext(path)[-1].lower()

    if mime.startswith("image"):
        steg_found, steg_msg = detect_steganography(path)
        logs.append("✅ Stego detected!" if steg_found else "❌ No stego found.")
        if steg_found:
            logs.append(f"Stego Message: {steg_msg[:100]}..." if len(steg_msg) > 100 else steg_msg)
            r_found, r_msg = ransomware_trace(steg_msg)
            logs.append(r_msg)
            return logs, "Threat Detected" if r_found else "Clean"
        else:
            logs.append(steg_msg)
            return logs, "Clean"

    elif ext in [".xlsm", ".docm"]:
        macro_logs, status = scan_macros(path)
        logs.extend(macro_logs)
        return logs, status

    elif ext in [".log", ".txt"]:
        log_lines, status = scan_log_file(path)
        logs.extend(log_lines)
        return logs, status

    logs.append("❌ Unsupported file type.")
    return logs, "Unsupported"

class RansomwareScanner(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Ransomware Forensic Scanner")
        self.geometry("1000x700")
        self.file_path = ""
        self.scan_history = []
        self.build_gui()

    def build_gui(self):
        tabs = ctk.CTkTabview(self)
        tabs.pack(padx=20, pady=20, expand=True, fill="both")
        scan_tab = tabs.add("Scanner")
        preview_tab = tabs.add("Preview")
        history_tab = tabs.add("History")

        ctk.CTkLabel(scan_tab, text="Select and Scan Files", font=("Segoe UI", 22, "bold")).pack(pady=10)
        ctk.CTkButton(scan_tab, text="Choose File", command=self.select_file).pack(pady=10)
        self.file_label = ctk.CTkLabel(scan_tab, text="No file selected")
        self.file_label.pack()
        ctk.CTkButton(scan_tab, text="Start Scan", fg_color="green", command=self.run_scan).pack(pady=10)
        self.progress = ctk.CTkProgressBar(scan_tab, width=500)
        self.progress.set(0)
        self.progress.pack(pady=10)
        self.status_label = ctk.CTkLabel(scan_tab, text="Status: Idle", font=("Segoe UI", 16))
        self.status_label.pack(pady=5)
        self.log_box = ctk.CTkTextbox(scan_tab, width=850, height=300)
        self.log_box.pack(pady=10)

        self.preview_box = ctk.CTkTextbox(preview_tab, width=850, height=500)
        self.preview_box.pack(pady=10)

        self.history_box = ctk.CTkTextbox(history_tab, width=850, height=500)
        self.history_box.pack(pady=10)
        ctk.CTkButton(history_tab, text="Export Last Report", command=self.export_report).pack(pady=10)

    def select_file(self):
        path = filedialog.askopenfilename(filetypes=[("Supported", "*.png *.jpg *.jpeg *.bmp *.xlsm *.docm *.log *.txt")])
        if path:
            self.file_path = path
            self.file_label.configure(text=os.path.basename(path))
            self.preview_box.delete("0.0", "end")
            if path.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
                try:
                    img = Image.open(path).resize((250, 250), Image.Resampling.LANCZOS)
                    img_tk = ImageTk.PhotoImage(img)
                    self.preview_box.image_create("end", image=img_tk)
                    self.preview_box.image = img_tk
                except:
                    self.preview_box.insert("0.0", "Image preview failed.")
            else:
                with open(path, 'r', errors='ignore') aory_box.delete("0.0", "end")
        for entry in self.scan_history:
            self.history_box.insert("end", f"{entry}\n")

    def export_report(self):
        if not self.file_path:
            messagebox.showinfo("Export", "No scan to export.")
            return
        name = f"scan_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
        path = filedialog.asksaveasfilename(defaultextension=".txt", initialfile=name)
        if path:
            with open(path, "w") as f:
                f.write(self.log_box.get("0.0", "end"))
            messagebox.showinfo("Export", f"Report saved as {path}")

if __name__ == "__main__":
    app = RansomwareScanner()
    app.mainloop

In [ ]:
def write_audit_log(file_path: str, status: str, details: list[str]):
    log_path = "audit.txt"
    with open(log_path, "a") as f:
        f.write(f"\n[{datetime.now().isoformat()}] File: {file_path} | Status: {status}\n")
        for line in details:
            f.write(f"  {line}\n")

In [ ]:
import json

def export_json_report(file_path: str, logs: list[str], status: str):
    report = {
        "timestamp": datetime.now().isoformat(),
        "filename": os.path.basename(file_path),
        "sha256": compute_file_hash(file_path),
        "status": status,
        "logs": logs
    }
    name = f"report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(name, "w") as f:
        json.dump(report, f, indent=2)
    return name

In [ ]:

def safe_open(path: str, mode: str = "rb"):
    try:
        return open(path, mode)
    except Exception as e:
        raise RuntimeError(f"Cannot open file: {e}")

In [ ]:
ransomware_scanner/
├── scanner_gui.py           # GUI ด้วย CustomTkinter
├── scanner_cli.py           # CLI สำหรับ batch scan
├── modules/
│   ├── stego.py             # ตรวจ steganography
│   ├── macro.py             # ตรวจ VBA macro
│   ├── mime.py              # ตรวจ MIME type
│   ├── audit.py             # บันทึก log และ export JSON
│   ├── ai_classifier.py     # จัดกลุ่มไฟล์ด้วย AI
│   ├── sandbox.py           # ตรวจ sandbox และ quarantine
├── quarantine/              # โฟลเดอร์เก็บไฟล์ต้องสงสัย
├── reports/                 # รายงาน JSON
├── audit.txt                # Log การสแกนทั้งหมด

In [ ]:

import sys
from modules.audit import write_audit_log, export_json_report
from modules.macro import scan_macros
from modules.stego import detect_steganography
from modules.mime import detect_mime_type
from modules.sandbox import sandbox_check
from modules.ai_classifier import classify_file

def full_scan(path):
    logs = []
    mime = detect_mime_type(path)
    logs.append(f"MIME: {mime}")
    logs.append(f"SHA256: {compute_file_hash(path)}")

    if not sandbox_check(path):
        logs.append("⚠️ Sandbox rejected this file.")
        status = "Quarantined"
    elif mime.startswith("image"):
        steg_found, steg_msg = detect_steganography(path)
        logs.append("✅ Stego found!" if steg_found else "❌ No stego.")
        status = "Threat Detected" if steg_found else "Clean"
    elif path.endswith((".docm", ".xlsm")):
        macro_logs, status = scan_macros(path)
        logs.extend(macro_logs)
    else:
        logs.append("❌ Unsupported file type.")
        status = "Unsupported"

    write_audit_log(path, status, logs)
    json_path = export_json_report(path, logs, status)
    print(f"Scan complete. Status: {status}. Report: {json_path}")

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: python scanner_cli.py <file_path>")
    else:
        full_scan(sys.argv[1]()

In [ ]:

def generate_disclaimer(metadata):
    if metadata["mime"].startswith("application/") and metadata["size_bytes"] > 5_000_000:
        return "⚠️ ไฟล์นี้อาจมีเนื้อหาที่ต้องได้รับอนุญาตก่อนใช้งาน"
    return "✅ ไฟล์นี้ผ่านการตรวจสอบเบื้องต้นแล้ว"

In [ ]:
import sys
import subprocess

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# ติดตั้งแพ็กเกจทีละตัว
for pkg in ["oletools", "stegano", "python-magic", "Pillow", "pygame", "customtkinter"]:
    try:
        install(pkg)
    except Exception as e:
        print(f"❌ Failed to install {pkg}: {e}")

In [ ]:

import sys
print(sys.executable)

In [ ]:

import subprocess
import sys

def install_package(pkg_name):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_name])
        print(f"✅ Installed: {pkg_name}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install {pkg_name}: {e}")

In [ ]:

import os
print(os.environ.get("SHELL"))  # อาจคืนค่าเป็น /bin/sh หรือ None

In [ ]:
!python main_cli.py /path/to/file.docm

In [ ]:

✅ MIME: application/vnd.ms-word.document.macroEnabled.12
✅ SHA256: a1b2c3...
✅ Macros found. Suspicious behavior: powershell
⚠️ Threat Detected. File quarantined.
📄 Report saved: reports/report_20251007_0539.json